In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
df = pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
#importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train=pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')
test=pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv')


In [3]:
df=pd.concat([train,test]) #merging the train and test dataset

In [4]:
df.head()

In [5]:
df.shape

In [6]:
df.info()

In [7]:
df.isnull().sum()

In [8]:
df.describe()

In [9]:
df.corr(numeric_only=True).abs()

In [11]:
# Visualize the relationships
plt.figure(figsize=(12, 6))
sns.barplot(x='MonsoonIntensity', y='FloodProbability', data=train)
plt.title('Monsoon Intensity vs Flood Probability')
plt.xlabel('Monsoon Intensity')
plt.ylabel('Flood Probability')
plt.show()

In [12]:
plt.figure(figsize=(12, 6))
sns.lineplot(x='TopographyDrainage', y='FloodProbability', data=train)
plt.title('Topography Drainage vs Flood Probability')
plt.xlabel('Topography Drainage')
plt.ylabel('Flood Probability')
plt.show()


In [13]:
plt.figure(figsize=(12, 6))
sns.lineplot(x='RiverManagement', y='FloodProbability', data=train)
plt.title('River Management vs Flood Probability')
plt.xlabel('River Management')
plt.ylabel('Flood Probability')
plt.show()

In [15]:
df.corr(numeric_only=True)['FloodProbability'].sort_values(ascending=False)

In [16]:
x=train.drop(['FloodProbability'],axis=1)
y=train['FloodProbability']

In [17]:
train.drop(['id'],axis=1,inplace=True)


In [19]:
# Best Performing Model.
lr=LinearRegression()
model=lr.fit(x,y)
pred=lr.predict(test)
submission=pd.DataFrame({'id':test['id'],'FloodProbability':pred})
submission.to_csv('submission.csv',index=False)

In [20]:
import pickle
pickle.dump(model,open('flood.pkl','wb'))
pickle.dump(df,open('floodprediction.h5','wb'))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score

from tqdm import tqdm

random_seed = 42

In [1]:
df = pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv")

df

In [3]:
train_columns = list(df.columns[: -1])

len(train_columns)

In [4]:
dataset_size = df.shape[0]

for feature in train_columns:
    feat_val_counts = df[feature].value_counts()
    const_count = feat_val_counts[0]
    if const_count > 0.1 * dataset_size:
        print(f"feature {feature} has {const_count / dataset_size} constants")
    print(f"{feature} dtype: {df[feature].dtype}; count of values: {len(feat_val_counts)}")

In [5]:
df.isna().sum()

In [6]:
df.shape

In [8]:
df[train_columns].describe()

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[train_columns], df["FloodProbability"], test_size=0.33, random_state=random_seed)

In [12]:
r2_score(y_test, model_lr.predict(X_test))

In [13]:
model_lr.coef_

In [14]:
df_wsum = df[train_columns].sum(axis=1) * 0.00528

In [15]:
plt.figure(figsize=(9, 5))
plt.hist(df["FloodProbability"], label="FloodProbability")
plt.hist(df_wsum, alpha=0.5, label="Weighted sum of features")

plt.legend()

plt.show()

In [16]:
from catboost import CatBoostClassifier

model_catboost_base = CatBoostClassifier(random_seed=random_seed, loss_function="CrossEntropy")

model_catboost_base.fit(X_train, y_train, verbose=False)

In [17]:
r2_score(y_test, model_catboost_base.predict_proba(X_test)[:, -1])

In [19]:
model_catboost_feat_select = CatBoostClassifier(random_seed=random_seed, loss_function="CrossEntropy")

model_catboost_feat_select.select_features(X_train, y_train, features_for_select=train_columns, num_features_to_select=10, algorithm="RecursiveByShapValues", verbose=False)

In [20]:
r2_score(y_test, model_catboost_feat_select.predict_proba(X_test)[:, -1])

In [21]:
df['sum'] = df[train_columns].sum(axis=1)

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[train_columns + ['sum']], df["FloodProbability"], test_size=0.33, random_state=random_seed)

In [23]:
from catboost import CatBoostClassifier

model_catboost_sum = CatBoostClassifier(random_seed=random_seed, loss_function="CrossEntropy")

model_catboost_sum.fit(X_train, y_train, verbose=False)

In [24]:
r2_score(y_test, model_catboost_sum.predict_proba(X_test)[:, -1])

In [25]:
model_catboost_sum.feature_importances_

In [ ]:
# Ignore Warnings
import warnings

# Data Manipulation
import pandas as pd
import numpy as np

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Data Transformation
from sklearn.preprocessing import  RobustScaler, StandardScaler, MinMaxScaler, FunctionTransformer

# Feature Selection
from sklearn.model_selection import train_test_split, GridSearchCV

# Pipeline
from sklearn.pipeline import Pipeline, FeatureUnion  # Импорт классов Pipeline и make_pipeline из scikit-learn
from sklearn.base import BaseEstimator, TransformerMixin  # Импорт базовых классов BaseEstimator и TransformerMixin

# Metrics
from sklearn.metrics import (mean_squared_error, make_scorer, r2_score)

# Feature Selection
from sklearn.linear_model import LassoCV

# Metrics
import math


# Regression Algorithms
from lightgbm import LGBMRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


%config InlineBackend.figure_format = 'svg'
warnings.filterwarnings('ignore')  # Disabling warning outputs

In [1]:
# Path to the CSV data file
TRAIN_DATAPATH = '/kaggle/input/playground-series-s4e5/train.csv'
TEST_DATAPATH = '/kaggle/input/playground-series-s4e5/test.csv'

# # Reading data from the CSV file into a DataFrame
train = pd.read_csv(TRAIN_DATAPATH) 
test = pd.read_csv(TEST_DATAPATH) 

# Concatenating train and test dataframes along axis 0 (rows) to combine them into a single dataframe.
df = pd.concat([train, test], axis=0)


# Displaying the first few rows of the DataFrame
df.head()

In [2]:
df.columns

In [3]:
# Dropping the 'id' column from the dataframe.
df.drop(columns=['id'], inplace=True)

In [4]:
# Initialize empty lists to store object and non-object columns
obj = []
ints = []

# Loop through DataFrame columns
for col in df.columns:
    # Check if column data type is object
    if df[col].dtype == 'object':
        # If object, append column name, unique values count, and count of missing values to 'obj' list
        obj.append((col, df[col].nunique(), df[col].isna().sum()))
    else:
        # If non-object, append column name, unique values count, and count of missing values to 'ints' list
        ints.append((col, df[col].nunique(), df[col].isna().sum(), df[col].skew()))

# Determine the maximum length of 'obj' and 'ints' lists
max_len = max(len(obj), len(ints))

# Extend 'obj' and 'ints' lists with empty tuples to match the maximum length
obj.extend([('', '', '')] * (max_len - len(obj)))
ints.extend([('', '', '', '')] * (max_len - len(ints)))

# Create a dictionary with keys representing column categories and values representing lists of corresponding data
data = {
    'Numeric_columns': [x[0] for x in ints],
    'int_cols_uniques': [x[1] for x in ints],
    'int_cols_missing': [x[2] for x in ints],
    'int_cols_skew': [x[3] for x in ints]
}

# Convert the dictionary into a pandas DataFrame
pd.DataFrame(data)




In [5]:
# Printing out the column names of the dataframe.
print(df.columns)


In [6]:
# This line of code retrieves the shape of the DataFrame 'df'
shape = df.shape
print(shape)

In [7]:
# Displaying concise summary information about the DataFrame, including
# data types, non-null values, and memory usage
df.info()


In [8]:
# Displaying the data types of each column in the DataFrame
df.dtypes

In [9]:
# Calculate the sum of missing values for each column and reset the index, storing the result in a new DataFrame called 'isna'
isna = df.isna().sum().reset_index(name="missing_values")

# Calculate the percentage of missing values for each column and add it as a new column called 'percentage' in the 'isna' DataFrame
isna['percentage'] = round((isna['missing_values'] / df.shape[0]) * 100, 2)

# Sort the 'isna' DataFrame by the 'missing_values' column in descending order and display the top 35 rows
isna.sort_values(by='missing_values', ascending=False)


In [10]:
# Counting the number of duplicated rows in the DataFrame 
df.duplicated().value_counts()


In [11]:
df.isnull().sum()

In [12]:
# Initialize an empty list to store the transformed column names
new_columns = []

# Iterate through the current column names in the DataFrame
for j in df.columns.tolist():
    # Convert each column name to a list of characters
    col = list(j)
    
    # Find the indices of uppercase characters (excluding the first character) to insert underscores
    upper_case_indices = [index for index, char in enumerate(j) if char.isupper() and index != 0]
    
    # Iterate through the uppercase indices in reverse order and insert underscores
    for snake in reversed(upper_case_indices):
        col.insert(snake, '_')
    
    # Join the characters back together to form the transformed column name and append it to the new_columns list
    new_columns.append(''.join(col))

# Assign the transformed column names to the DataFrame's columns attribute
df.columns = new_columns

# Convert all column names to lowercase
df.columns = df.columns.str.lower()


# Displaying the first few rows of the dataframe.
df.head()


In [13]:
# Generate descriptive statistics for all numerical columns
df.describe()

In [15]:
# Select numerical columns from the DataFrame
numerics = df.select_dtypes(include=np.number)

# Calculate the number of plots, rows, and columns for subplots
num_plots = len(numerics.columns)
num_columns = 3
num_rows = num_plots // num_columns + (1 if num_plots % num_columns > 0 else 0)



# Set the figure size based on the number of rows
plt.figure(figsize=(11, 4 * num_rows))

# Iterate over each numerical column and create a histogram subplot
for i, col in enumerate(numerics, 1):
    plt.subplot(num_rows, num_columns, i)  # Create subplot
    mean_values = numerics[col].mean()
    median = numerics[col].median()

    sns.kdeplot(numerics[col], fill=True, color='#638889')  # Plot histogram using seaborn
    plt.axvline(x=mean_values, color='#F28585', linestyle='--', label='Mean')
    plt.axvline(x=median, color='#747264', linestyle='--', label='Median')
    plt.grid(True, alpha=0.8)  # Add grid lines to the plot
    plt.title(f'{" ".join(list(map(str.title, col.split("_"))))} Distribution')  # Set title for the subplot
    plt.savefig('/kaggle/working/kdeplot.png')
    plt.legend()

plt.tight_layout()  # Adjust layout to prevent overlapping
plt.show()  # Display the plots

In [16]:
# Create a figure with custom size
plt.figure(figsize=(11, 10))

# Generate a heatmap of correlations for numerical columns in the DataFrame
sns.heatmap(
    df.select_dtypes(include=np.number).corr(),  # DataFrame correlation matrix for numerical columns
    annot=True,  # Show correlation values in cells
    cmap=['#638889', '#678788', '#6c8788', '#718788', '#768788', '#7b8788', '#808788', '#858788', '#8a8787', '#8f8787', 
          '#948687', '#998687', '#9e8687', '#a38687', '#a88687', '#ac8686', '#b18686', '#b68686', '#bb8686', '#c08686',
          '#c58586', '#ca8586', '#cf8585', '#d48585', '#d98585', '#de8585', '#e38585', '#e88585', '#ed8585', '#f28585'],
    annot_kws={"fontsize": 7}  # Annotation font size
)

# Display the heatmap
plt.show()

In [17]:

# Get list of columns excluding 'flood_probability'
cols = df.drop(columns='flood_probability').columns.tolist()

# Define statistical functions to apply
funcs = [np.sum, np.mean, np.std, np.max, np.min, np.median]

# Apply each statistical function to each row in the DataFrame
for func in funcs:
    df[f'features_{func.__name__}'] = df[cols].agg(func, axis=1)    

In [18]:
df.head()

In [19]:
# Define feature groups with respective features
feature_list = {'env_degradation_index' : ['deforestation', 'urbanization', 'wetland_loss', 'siltation'], 
                'infrastructure_vulnerability_score' : ['dams_quality', 'drainage_systems', 'deteriorating_infrastructure',
                                                        'ineffective_disaster_preparedness'],
                'human_impact_factor' : ['deforestation', 'urbanization', 'agricultural_practices', 'encroachments'], 
                'climate_stress_index' : ['monsoon_intensity', 'climate_change'], 
                'governance_planning_efficiency' : ['river_management', 'inadequate_planning', 'political_factors'], 
                'disaster_preparedness_index' : ['ineffective_disaster_preparedness', 'drainage_systems', 'dams_quality'], 
                'population_pressure_index' : ['population_score', 'urbanization', 'encroachments'], 
                'natural_hazard_risk_score' : ['landslides', 'coastal_vulnerability'], 
                'water_management_effectiveness' : ['watersheds', 'river_management', 'dams_quality'], 
                'developmental_impact_index' : ['urbanization', 'agricultural_practices', 'deforestation']}

In [23]:
# Extract features and encode categorical variables using one-hot encoding
X = df[:len(train)].drop(columns='flood_probability')
TEST = df[len(train):]

# Extract the target variable
y = df[:len(train)]['flood_probability']


In [24]:
# Create an instance of StandardScaler
scaler = StandardScaler()

# Scale the features in postlasso_df using StandardScaler
X_scaled = scaler.fit_transform(X)

lasso_cv = LassoCV(alphas=[0.0001, 0.0003, 0.0006, 0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1, 0.3, 0.6, 1],
                   max_iter=1000, cv=5, random_state=42)

# Fit the LassoCV model to the scaled feature matrix X_scaled and the target vector y
lasso_cv.fit(X_scaled, y)

# Extract the column names (features) where the coefficients are non-zero
selected_features = X.columns[(lasso_cv.coef_ != 0).ravel()].tolist()

# Print the number of selected features
print(selected_features)

In [25]:
# Create a DataFrame with feature names and their coefficients from Lasso regression
lasso_coefficients = pd.DataFrame({
    'Feature': X.columns,  # Feature names
    'Coefficient': lasso_cv.coef_  # Coefficients from Lasso regression
})

# Filter out features with non-zero coefficients
lasso_filtered = lasso_coefficients[abs(lasso_coefficients['Coefficient']) > 0]

# Sort the filtered DataFrame by coefficient values in descending order and reset index
lasso_filtered_sorted = lasso_filtered.sort_values(by='Coefficient', ascending=False).reset_index(drop=True)

# Display the sorted DataFrame
print(lasso_filtered_sorted)


In [26]:
# Select only the columns (features) identified as important by the Lasso regression model
postlasso_df = X[selected_features]

# Splitting the data into train & test
X_train, X_test, y_train, y_test = train_test_split(postlasso_df, y, test_size=0.3, random_state=42)

In [27]:

# Define models with their corresponding estimators
models = {
    'LGBMRegressor': LGBMRegressor(verbose=-1)
}

# Define hyperparameter grids for each model
params = {
    'LGBMRegressor': {
        'model__learning_rate': [0.01, 0.1, 0.3],
        'model__n_estimators': [50, 100, 200],
        'model__max_depth': [3, 5, 7],
        'model__num_leaves': [15, 31, 63]
    }
}

In [30]:
# Set the display option to show all the contents of DataFrame columns without truncation
pd.set_option('display.max_colwidth', None)

# Set the display option to format float numbers with a precision of 5 decimal places
pd.set_option('display.float_format', '{:.5f}'.format)

# Display the DataFrame results_df
results_df

In [33]:
# Store the counts in a list
features = features_percentage_df['Feature'].tolist()
counts = features_percentage_df['Importance (%)']

# Define labels for the counts
labels = list(map(lambda x: x.replace('_', ' ').title(), features))

# Create a pie chart
plt.pie(counts, labels=labels, autopct='%1.1f%%', startangle=140,
        colors=['#96ceb4', '#ffeead', '#ffcc5c', '#ff6f69', '#96897f'])

# Draw a circle in the center to create a ring
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

# Equal aspect ratio ensures that pie is drawn as a circle
plt.axis('equal')
plt.title('Distribution of Feature Importance')

plt.show()

In [34]:
# Select only the columns (features) identified as important by the Lasso regression model in the testing dataset
X = TEST[selected_features]

# Use the best_model to predict the target variable for the testing dataset
target_pred = grid_search.predict(X)

# Read the sample submission file and set the 'Id' column as the index
submission = pd.read_csv('/kaggle/input/playground-series-s4e5/sample_submission.csv', index_col='id')

# Update the 'SalePrice' column in the submission DataFrame with the predicted target variable values
submission['FloodProbability'] = target_pred

# Save the submission DataFrame to a CSV file named 'submission.csv' in the './data' directory
submission.to_csv('/kaggle/working/submission.csv')

In [35]:
!pip freeze > requirements.txt

In [36]:
!python3 --version

In [37]:
!which python

In [1]:
train = pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv')
orig_features = train.columns[1:-1].to_list()
train.shape, test.shape

In [2]:
train.info()

In [3]:
train.head()

In [4]:
test.head()

In [5]:
# Checking null's
train.isna().sum().sum()

In [6]:
# No duplicates
train.duplicated().sum()

In [7]:
# Descriptive statistics
train.drop('id',axis=1).describe().T

In [8]:
# Lets see the data present in the independent features
for col in train.columns[1:-1]:
    uniques = ', '.join(map(str, sorted(train[col].unique())))
    print(f'Values in {col:<32}: {uniques:<69}| total={train[col].nunique()}')

In [12]:
pca = PCA()
pca.fit(train[orig_features])
pca.explained_variance_ratio_

In [15]:
X = train.iloc[:,1:-1]
y = train['FloodProbability']
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [16]:
# For checking the model performance
def cross_validate_print(alg, X, y, repeat=2, SINGLE_FOLD=False):
    
    start_time = time()
    scores = []
    oof_preds = np.full_like(y, np.nan, dtype=float)
    
    for fold,(train_idx, val_idx) in enumerate(kf.split(X, y.astype(str))):
        x_cv_tr, y_cv_tr = X.iloc[train_idx], y.iloc[train_idx]
        x_cv_vl, y_cv_vl = X.iloc[val_idx], y.iloc[val_idx]
        
        y_pred = np.zeros_like(y_cv_vl, dtype=float)
        for i in range(repeat):
            m = clone(alg)
            if repeat > 1:
                mm = m
                if isinstance(mm, Pipeline):
                    mm = mm[-1]
                mm.set_params(random_state=i)
            m.fit(x_cv_tr, y_cv_tr)
            y_pred += m.predict(x_cv_vl)
        y_pred /= repeat   
        
        score = skm.r2_score(y_cv_vl, y_pred)
        print(f'# Fold {fold}: R2 Score = {score:0.5f}')
        scores.append(score)
        oof_preds[val_idx] = y_pred
        if SINGLE_FOLD: break
        
    total_time = time() - start_time
    if isinstance(m, Pipeline):
        model_name = m[-1].__class__.__name__
    else:
        model_name = m.__class__.__name__
    print(f'{BlueBold}# Mean R2 Score = {np.mean(scores):0.5f} {RedBold}± {np.std(scores):0.5f} {BlueBold}for {model_name}\n'
          f'{"Single Fold" if SINGLE_FOLD else ""}'
          f'Time Elapsed: {np.round((total_time / 60),0)} Min {end}\n')

In [18]:
temp = train['FloodProbability'].groupby(X.sum(axis=1)).mean()
plt.figure(figsize=(10,4))
sb.scatterplot(x=temp.index, y=temp, c=temp.index.isin(np.arange(72,76)), s=30, cmap='RdYlBu_r')
plt.gca().xaxis.set_major_locator(MultipleLocator(5))
plt.gca().xaxis.set_minor_locator(MultipleLocator(1))
plt.gca().yaxis.set_major_locator(MultipleLocator(0.05))
plt.grid(True, which='major', linestyle='--')
plt.grid(True, which='minor', linestyle=':')
plt.xlabel('Feature sums (along column axis)')
plt.tight_layout()

In [19]:
# For getting the unique values in train. Used later in feature engineering
unique = []
for col in train.iloc[:,1:-1].columns:
    unique.extend(train[col].unique())
unique = list(set(unique))

In [23]:
X = train_df
y = train['FloodProbability']

cross_validate_print(xgr_model, X, y)
cross_validate_print(cbr_model, X, y)

In [35]:
def objective(trial):
    params = {
        'num_parallel_tree': trial.suggest_int('num_parallel_tree', 50, 1000),
        'max_depth':         trial.suggest_int('max_depth', 1, 30),
        'colsample_bynode':  trial.suggest_float('colsample_bynode', 0.05, 1),
        'subsample':         trial.suggest_float('subsample', 0.05, 1),
        'min_child_weight':  trial.suggest_int('min_child_weight', 1, 250),
        'lambda':            trial.suggest_float('lambda', 1, 10)
    }
    
    gc.collect()
    xgrf = XGBRegressor(**params, **fixed_params_xgrf)
    r2_score = cross_validate_tuner(xgrf, X, y, XGB=True)
    
    return r2_score

TPESampler = optuna.samplers.TPESampler(multivariate=True, group=True)
optimize_r2_xgrf = optuna.create_study(direction='maximize', sampler=TPESampler, study_name='Optimizing R2 for XGRF')
    

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'playground-series-s4e5:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F73278%2F8121328%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240528%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240528T155103Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0cdf00152e2d9da8616a0fb96ca0eb19f5d0d6fd696ac54a1bb4141a4c47820770d28611dc20da3b5f4f2d2a68a3bef84b8623dbf38002b7aa292109886b76ae8a48b3ff8b3b32434dae233a1dd57fc05ebe753e25ee8aa4860113ed9bf6ac3be3318ea97ca1830bd8257fa20d4bba9bbe218920c5fed9216f21aaf6a7e9769f85d47a4bab0ca62ec05dec8da4d7e62539859b3ce1d13653bf1ae02e97278e164a0534569c232fe490004c434705028e0c9a9e90d09a67282cf3682ec3ee0eddb50400b44a2b351ea11fa204143ba563c7e3bc72ef8fa8b035e051ca8d1a29d42651c8589ec127252966dcccebfc848331cb7d4acd4df5f8f95ff270ecd60e20'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df_train = pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv")

In [3]:
df_train.shape

In [4]:
df_train.head()

In [5]:
df_train.describe()

In [6]:
df_train.info()

In [7]:
####### BOXPLOT FOR EVERY FEATURE ########

import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")

num_cols = len(df_train.columns)
fig, axes = plt.subplots(nrows=1, ncols=num_cols, figsize=(15, 5))

for col in df_train.columns:
    plt.figure(figsize=(8, 5))
    sns.boxplot(x=df_train[col])
    plt.title(col)
    plt.xlabel('')
    plt.ylabel('')
    plt.show()

In [8]:
df_train.shape

In [9]:
def detect_outliers(df, threshold=1.5):
    outlier_columns = []

    for col in df.columns:
        # Calculate the first and third quartiles
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)

        # Interquartile range (IQR)
        iqr = q3 - q1

        # Define the lower and upper bounds for outlier detection
        lower_bound = q1 - threshold * iqr
        upper_bound = q3 + threshold * iqr

        # Find outliers
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]

        if not outliers.empty:
            outlier_columns.append(col)

    return outlier_columns

outlier_cols = detect_outliers(df_train)
print("Columns with outliers: \n", outlier_cols)

In [10]:
numerical_cols = df_train.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df_train.select_dtypes(include=['object']).columns

# Plot numerical columns against 'FloodProbability' using scatter plots
for col in numerical_cols:
    plt.figure(figsize=(8, 6))
    sns.scatterplot(data=df_train, x=col, y='FloodProbability', alpha=0.7)
    plt.title(f'{col} vs. FloodProbability')
    plt.xlabel(col)
    plt.ylabel('FloodProbability')
    plt.grid(True)
    plt.show()

# Plot categorical columns against 'FloodProbability' using bar plots
for col in categorical_cols:
    plt.figure(figsize=(8, 6))
    sns.barplot(data=df_train, x=col, y='FloodProbability', alpha=0.7)
    plt.title(f'{col} vs. FloodProbability')
    plt.xlabel(col)
    plt.ylabel('FloodProbability')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()

In [11]:
# Drop id
df_train.drop("id" , axis = 1  , inplace = True)

In [12]:
# NULL values
df_train.isnull().sum()

In [13]:
## REMOVE OUTLIERS

from scipy import stats
def remove_outliers_zscore(df, threshold=3):
    numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
    # Calculate Z-score for each numerical column
    z_scores = np.abs(stats.zscore(df[numerical_cols]))
    # Identify rows where any Z-score exceeds the threshold
    outlier_rows = np.any(z_scores > threshold, axis=1)
    # Remove outlier rows from DataFrame
    df_cleaned = df[~outlier_rows]

    return df_cleaned

df_train = remove_outliers_zscore(df_train)

In [14]:
df_train.shape

In [15]:
'''No need to perform standardization / normalization as all the features have a small and comparable scale'''

In [16]:
correlation_matrix = df_train.corr(method='pearson')
plt.figure(figsize=(20, 15))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Pearson Correlation Matrix')
plt.show()

In [17]:
flood_prob_corr = correlation_matrix['FloodProbability'].drop('FloodProbability')  # Drop 'FloodProbability' itself
# Sort the correlation coefficients in decreasing order of magnitude
sorted_corr = flood_prob_corr.abs().sort_values(ascending=False)
# Print feature names and their corresponding correlation coefficients
for feature in sorted_corr.index:
    print(f"Feature: {feature},     Correlation: {flood_prob_corr[feature]}")

In [18]:
from scipy.stats import chi2_contingency

categorical_df = df_train

chi2_results = {}
for col in categorical_df.columns:
    contingency_table = pd.crosstab(df_train[col], df_train['FloodProbability'])
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    chi2_results[col] = chi2

# Sort the chi-square test results in decreasing order of magnitude
sorted_chi2_results = {k: v for k, v in sorted(chi2_results.items(), key=lambda item: item[1], reverse=True)}

# Print feature names and their corresponding chi-square statistics
for feature, chi2_statistic in sorted_chi2_results.items():
    print(f"Feature: {feature}, Chi-square statistic: {chi2_statistic}")

In [19]:
'''Not dropping any feature since they all have practically the same correlation coefficient and Chi-square statistic'''

In [21]:
x_train.shape

In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Verify GPU is available
if len(tf.config.experimental.list_physical_devices('GPU')) > 0:
    print("GPU is available and will be used for training.")
else:
    print("GPU is not available, using CPU.")

In [24]:
!pip install keras_tuner

In [25]:
import keras_tuner as kt

In [29]:
tuner.search(x_train , y_train , epochs = 10 , batch_size = 2056,validation_data = (x_test , y_test))

In [30]:
best_hps = tuner.get_best_hyperparameters()[0]
print(best_hps.values)

# best_model = tuner.get_best_models()[0]
# best_model.summary()

In [32]:
best_model.summary()

In [38]:
df_test

In [39]:
df_test_nanid = df_test.drop("id" , axis = 1)

In [40]:
x_target = df_test_nanid

In [41]:
x_target.shape

In [42]:
outputs = best_model.predict(x_target)
output_df = pd.DataFrame(outputs , columns =  ["FloodProbability"])

In [44]:
id = df_test[["id"]]

In [45]:
id

In [46]:
output_df = pd.concat([id, output_df], axis=1)

In [47]:
output_df

In [48]:
output_df = output_df.reset_index(drop=True)

In [49]:
output_df.to_csv("target outputs.csv" , index = False)

In [50]:
import tensorflow.keras.backend as K

def r_squared(y_true, y_pred):
    ss_res = K.sum(K.square(y_true - y_pred))
    ss_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return 1 - ss_res / (ss_tot + K.epsilon())

In [51]:
from tensorflow.keras.models import load_model
model = load_model('/kaggle/input/best_model_flood_prediction/tensorflow2/v2/1/best_flood_prediction_model.h5')

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
sample=pd.read_csv("/kaggle/input/playground-series-s4e5/sample_submission.csv",sep=",")
train=pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv",sep=",")
test=pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv",sep=",")

In [2]:
sample.head()

In [3]:
sample.shape,train.shape,test.shape

In [4]:
train.head()

In [5]:
y=train["FloodProbability"]
df=train.drop(["FloodProbability","id"],axis=1)

In [6]:

df.shape,y.shape,train.shape

In [7]:
df.size,y.size,train.size

In [9]:
test=test.drop(["id"],axis=1)

In [10]:
df.hist(bins=50,figsize=(20,15))
plt.show()

In [11]:
plt.hist(df)
plt.show()

In [12]:
train.info()

In [13]:
test.info()

In [14]:
import matplotlib.pyplot as plt
plt.hist(df)
plt.show()
    

In [15]:
y.value_counts()


In [16]:
df.shape

In [17]:
import matplotlib.pyplot as plt 
import seaborn as sns 

In [18]:
plt.hist(df)
plt.show()

In [20]:
plt.hist(df)
plt.show()

In [21]:
from sklearn.preprocessing import StandardScaler , MinMaxScaler

In [22]:
scaler=MinMaxScaler(clip=True)
df=scaler.fit_transform(df)
test=scaler.transform(test)

In [25]:
# plt.hist(train)
# plt.show()

In [26]:
# from sklearn.decomposition import PCA
# pca=PCA(n_components=10)

In [27]:
# train=pca.fit(train)

In [28]:
# pca.explained_variance_ratio_
# pca.n_components_

In [30]:
# best=SelectKBest(k=15).fit_transform(train,y)

In [31]:
df.shape 

In [32]:
y=pd.Series(y)

In [33]:
feature=df.mean(axis=1)

In [34]:
feature.shape,df.shape

In [35]:
test=pd.DataFrame(test)

In [40]:
from sklearn.model_selection import train_test_split

In [41]:
y.shape

In [43]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression , Ridge ,Lasso ,BayesianRidge
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR

In [45]:
# model3=DecisionTreeRegressor(random_state=0,max_depth=5)

In [46]:
# from sklearn.gaussian_process import GaussianProcessRegressor
# from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

# # Assuming 'X' and 'y' are your features and target variable respectively
# kernel = DotProduct() + WhiteKernel()
# gpr = GaussianProcessRegressor(kernel=kernel)

# gpr.fit(xtrain, ytrain)
# y_pred, y_std = gpr.predict(xtrain, return_std=True)


In [47]:
# model2.fit(xtrain,ytrain)           




In [48]:
# model1.fit(xtrain,ytrain)

In [49]:
# model2 = Pipeline([
#     ('poly', PolynomialFeatures(degree=2,interaction_only=False)),
#     ('Ridge', Ridge(fit_intercept=True,random_state=0,alpha=1.5))
# ])

In [50]:
from sklearn.neural_network import MLPRegressor

In [51]:
# model2=SVR(kernel="rbf",gamma='scale')

In [52]:
# model2=BayesianRidge()

In [53]:
from xgboost import XGBRegressor

In [54]:
# new_df.hist(bins=50,figsize=(20,15))
# plt.show()

In [57]:
# from sklearn.gaussian_process import GaussianProcessRegressor
# model2=GaussianProcessRegressor()

In [63]:
prediction ,testy

In [68]:
cross_val_score(model2,xtrain,ytrain,cv=5)

In [69]:
r2_score(yval,testy)

In [71]:

submission=pd.DataFrame(columns=["id","FloodProbability"])
submission["id"]=[i for i in range(1117957,len(testf)+1117957)]
submission["FloodProbability"]=predict
submission.to_csv("submission.csv",index=None)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
df_train = pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv")

In [2]:
df_train.shape

In [3]:
df_train.head()

In [4]:
df_train.describe()

In [5]:
df_train.info()

In [6]:
####### BOXPLOT FOR EVERY FEATURE ########

import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")

num_cols = len(df_train.columns)
fig, axes = plt.subplots(nrows=1, ncols=num_cols, figsize=(15, 5))

for col in df_train.columns:
    plt.figure(figsize=(8, 5))  
    sns.boxplot(x=df_train[col])
    plt.title(col)
    plt.xlabel('')
    plt.ylabel('')
    plt.show()

In [7]:
df_train.shape

In [10]:
# Drop id
df_train.drop("id" , axis = 1  , inplace = True)

In [11]:
df_train.isnull().sum()

In [12]:
from scipy import stats
def remove_outliers_zscore(df, threshold=3):
    numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
    # Calculate Z-score for each numerical column
    z_scores = np.abs(stats.zscore(df[numerical_cols]))
    # Identify rows where any Z-score exceeds the threshold
    outlier_rows = np.any(z_scores > threshold, axis=1)
    # Remove outlier rows from DataFrame
    df_cleaned = df[~outlier_rows]
    
    return df_cleaned

df_train = remove_outliers_zscore(df_train)

In [13]:
df_train.shape

In [16]:
flood_prob_corr = correlation_matrix['FloodProbability'].drop('FloodProbability')  # Drop 'FloodProbability' itself
# Sort the correlation coefficients in decreasing order of magnitude
sorted_corr = flood_prob_corr.abs().sort_values(ascending=False)
# Print feature names and their corresponding correlation coefficients
for feature in sorted_corr.index:
    print(f"Feature: {feature},     Correlation: {flood_prob_corr[feature]}")

In [17]:
from scipy.stats import chi2_contingency

categorical_df = df_train

chi2_results = {}
for col in categorical_df.columns:
    contingency_table = pd.crosstab(df_train[col], df_train['FloodProbability'])
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    chi2_results[col] = chi2

# Sort the chi-square test results in decreasing order of magnitude
sorted_chi2_results = {k: v for k, v in sorted(chi2_results.items(), key=lambda item: item[1], reverse=True)}

# Print feature names and their corresponding chi-square statistics
for feature, chi2_statistic in sorted_chi2_results.items():
    print(f"Feature: {feature}, Chi-square statistic: {chi2_statistic}")

In [18]:
'''Not dropping any feature since they all have practically the same correlation coefficient and Chi-square statistic'''

In [21]:
# from sklearn.neighbors import KNeighborsRegressor

# for k in range(3 , 7):
#     knn_model = KNeighborsRegressor(n_neighbors = k)
#     outputs = cross_val_score(knn_model, x, y,cv=5)
#     print("K:" , k)
#     print("Outputs: ",outputs)

In [22]:
# from sklearn.svm import SVR
# from sklearn.model_selection import cross_val_score

# svr_model = SVR()
# outputs = cross_val_score(svr_model, x_train, np.ravel(y_train), cv=10)
# print("Outputs:", outputs)

In [23]:
y_train = np.ravel(y_train)

In [25]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Initialize the XGBoost model
xgb_model = xgb.XGBRegressor()

# Train the XGBoost model on the training data
xgb_model.fit(x_train, y_train)

# Make predictions on the training set
y_train_pred = xgb_model.predict(x_train)

# Make predictions on the test set
y_test_pred = xgb_model.predict(x_test)

# Calculate RMSE, MSE, and MAE for training set
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)

# Calculate RMSE, MSE, and MAE for test set
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Print the results
print("Training set:")
print("RMSE:", rmse_train)
print("MSE:", mse_train)
print("MAE:", mae_train)

print("\nTest set:")
print("RMSE:", rmse_test)
print("MSE:", mse_test)
print("MAE:", mae_test)

In [26]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Initialize the Gradient Boosting Regressor model
gbm_model = GradientBoostingRegressor()

# Train the Gradient Boosting Regressor model on the training data
gbm_model.fit(x_train, y_train)

# Make predictions on the training set
y_train_pred = gbm_model.predict(x_train)

# Make predictions on the test set
y_test_pred = gbm_model.predict(x_test)

# Calculate RMSE, MSE, and MAE for training set
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)

# Calculate RMSE, MSE, and MAE for test set
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Print the results
print("Training set:")
print("RMSE:", rmse_train)
print("MSE:", mse_train)
print("MAE:", mae_train)

print("\nTest set:")
print("RMSE:", rmse_test)
print("MSE:", mse_test)
print("MAE:", mae_test)


In [27]:
# from sklearn.ensemble import AdaBoostRegressor
# from sklearn.model_selection import cross_val_score

# adaboost_model = AdaBoostRegressor()
# outputs = cross_val_score(adaboost_model, x, np.ravel(y), cv=10, scoring='neg_mean_squared_error')

# # Convert outputs to positive values
# outputs = -outputs

# print("Outputs:", outputs)

In [29]:
df_test = pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv")

In [30]:
df_test

In [31]:
df_test_nanid = df_test.drop("id" , axis = 1)

In [32]:
x_target = df_test_nanid

In [33]:
x_target.shape

In [35]:
output_df

In [36]:
id = df_test[["id"]]

In [40]:
output_df = output_df.reset_index(drop=True)

In [42]:
output_df.to_csv("submission.csv")

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.metrics import R2Score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from tensorflow.keras.layers import LeakyReLU

from tensorflow.keras.callbacks import Callback, EarlyStopping,ModelCheckpoint, ReduceLROnPlateau

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir, pred_and_plot


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Reading input files
df = pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv")
df_check = pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv")

In [2]:
mean_less_than_30 = df.loc[df['FloodProbability'] < 0.3].mean()
mean_more_than_70 = df.loc[df['FloodProbability'] > 0.7].mean()
mean_between_30_and_70 = df.loc[(df['FloodProbability'] >= 0.3) & (df['FloodProbability'] <= 0.7)].mean()
print("Mean for probability < 30:", mean_less_than_30)
print("Mean for probability > 70:", mean_more_than_70)
print("Mean for probability between 30 and 70:", mean_between_30_and_70)

In [3]:
# Dividing on data and target
X = df.drop(["FloodProbability"], axis=1)
Y = df["FloodProbability"]

In [12]:
# Plot the training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [14]:
# Plotting
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_predicted, color='blue', alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Predictions vs. Actual Values')
plt.grid(True)
plt.show()

In [15]:
ids = df_check["id"]
df_check = df_check.drop("id", axis=1)
df_check = scaler.fit_transform(df_check)

In [17]:
df_submission = pd.DataFrame({'id': ids, 'FloodProbability': predictions.flatten()})
df_submission.to_csv('submission.csv', index=False)

In [20]:
'''from sklearn.metrics import r2_score

# Evaluate the model (optional)
r2 = r2_score(y_test, y_pred)
print("R² Score:", r2)'''

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import xgboost
import lightgbm as lgb
import catboost
import optuna
from matplotlib import pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
sns.set(style="whitegrid")
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [1]:
all_data=pd.read_csv('/kaggle/input/flood-prediction-features/data_all.csv',index_col='id')
# df_test=pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv',index_col='id')
all_data

In [2]:
df_train = all_data[all_data['type']==0]
df_test = all_data[all_data['type']==1]

In [3]:
X = df_train.drop(['FloodProbability', 'type'], axis=1)
y = df_train['FloodProbability']
df_test = df_test.drop(['FloodProbability', 'type'], axis=1)

In [4]:
import optuna
from sklearn.model_selection import cross_validate
def objective(trial):
    params = {
            'num_leaves': trial.suggest_int('num_leaves', 100, 500),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0, log=True),
            'n_estimators': trial.suggest_int('n_estimators', 300, 2000),
            'subsample_for_bin': trial.suggest_int('subsample_for_bin', 20000, 300000),
            'min_child_samples': trial.suggest_int('min_child_samples', 20, 500),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 10.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 10.0, log=True),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
            'subsample': trial.suggest_float('subsample', 0.25, 1.0),
            'max_depth': trial.suggest_int('max_depth', 1, 15)
            }


    
    clf = lgb.LGBMRegressor(**params,boosting_type= 'gbdt',random_state=0, device='gpu',verbosity=-1)
    #交叉验证
    cv_results = cross_validate(clf, X, y, cv=5, scoring='r2')
    
    validation_score = np.mean(cv_results['test_score'])
    
    return validation_score
# def objective(trial):
#     #定义参数范围
#     max_depth = trial.suggest_int('max_depth', 3, 10)
#     n_estimators = trial.suggest_int('n_estimators', 100, 2000)
#     learning_rate = trial.suggest_float('learning_rate', 0.01, 1)
    
#     print('Training the model with', X.shape[1], 'features')

#     params = {'n_estimators': n_estimators,
#               'learning_rate': learning_rate,
#               'max_depth': max_depth}
        
#     clf = XGBRegressor(**params,tree_method='gpu_hist',random_state=0)
#     #交叉验证
#     cv_results = cross_validate(clf, X, y, cv=7, scoring='r2')
    
#     validation_score = np.mean(cv_results['test_score'])
    
#     return validation_score

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Ignore all warnings
import warnings
warnings.filterwarnings('ignore')

In [1]:
train_df = pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv')
sub_df = pd.read_csv('/kaggle/input/playground-series-s4e5/sample_submission.csv')

In [2]:
train_df.sample(5)

In [3]:
test_df.sample(5)

In [4]:
sub_df.head()

In [5]:
train_df.info()

In [8]:
X = train_df.iloc[:, 1:-1]
y = train_df.iloc[:, -1]

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Mean Squared Error
lr_mse = mean_squared_error(y_val, y_pred)
print(f'Mean Squared Error: {np.round(lr_mse, 5)}')

In [18]:
# Mean Absolute Error
sgd_mae = mean_absolute_error(y_val, y_pred)
print(f'Mean Absolute Error: {np.round(sgd_mae, 5)}')

In [19]:
# Mean Squared Error
sgd_mse = mean_squared_error(y_val, y_pred)
print(f'Mean Squared Error: {np.round(sgd_mse, 5)}')

In [20]:
# R-square score
sgd_r2 = r2_score(y_val, y_pred)
print(f'R-Squared Score: {np.round(sgd_r2 * 100, 3)}')

In [25]:
# R-square score
rfr_r2 = r2_score(y_val, y_pred)
print(f'R-Squared Score: {np.round(rfr_r2 * 100, 3)}')

In [29]:
# Mean Squared Error
bgr_mse = mean_squared_error(y_val, y_pred)
print(f'Mean Squared Error: {np.round(bgr_mse, 5)}')

In [33]:
# Mean Absolute Error
gbr_mae = mean_absolute_error(y_val, y_pred)
print(f'Mean Absolute Error: {np.round(gbr_mae, 5)}')

In [34]:
# Mean Squared Error
gbr_mse = mean_squared_error(y_val, y_pred)
print(f'Mean Squared Error: {np.round(gbr_mse, 5)}')

In [40]:
# R-square score
svr_r2 = r2_score(y_val, y_pred)
print(f'R-Squared Score: {np.round(svr_r2 * 100, 3)}')

In [45]:
# R-square score
lgbm_r2 = r2_score(y_val, y_pred)
print(f'R-Squared Score: {np.round(lgbm_r2 * 100, 3)}')

In [47]:
test_df.head()

In [48]:
# Define X_test (remove id column)
X_test = test_df.iloc[:, 1:]

In [50]:
# Get y_pred
y_pred = lr_model.predict(X_test)

In [51]:
# Add prediction column to submission df
target_col = "FloodProbability"
sub_df[target_col] = y_pred

In [52]:
sub_df.head()

In [53]:
# Save the submission dataframe to a CSV file
sub_df.to_csv('submission.csv', index=False)

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv', index_col ='id')
test = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv', index_col ='id')

In [3]:
train.info()

In [4]:
test.info()

In [5]:
cols = train.drop('FloodProbability', axis = 1).columns.tolist()

In [7]:
round(train.agg(['min','mean','median','max','var','std','skew']),2).T

In [14]:
train.isna().sum()/train.shape[0]

In [15]:
print('Shape before :',train.shape)
train.dropna(how='any', inplace=True)
print("Shape after :",train.shape)

In [16]:
y = train['FloodProbability']
X = train.drop('FloodProbability', axis=1)

In [22]:
# Print the best estimators
for name, estimator in best_estimators.items():
    print(f"Best estimator for {name}: {estimator}")

In [24]:
submission = pd.DataFrame({'id':test.index, 'FloodProbability':y_preds})
submission

In [25]:
submission.to_csv('/kaggle/working/submission.csv',index=False)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [1]:
!pip install featuring-data

In [2]:
from featuringdata.featuresEDA import FeaturesEDA
from featuringdata.featureSelector import FeatureSelector

In [3]:
train_data = pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')

In [4]:
# train_data.info()

In [5]:
# train_data.head()

In [6]:
eda = FeaturesEDA(report_prefix='Flood_Prediction', target_col="FloodProbability",
                  cols_to_drop=["id"])

In [8]:
eda.master_columns_df.sort_values(by=["Random Forest"], ascending=False)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score

import lightgbm as lgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import StackingRegressor

from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop

In [2]:
data = pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')
data.head()

In [3]:
data.info()

In [4]:
original_data = pd.read_csv('/kaggle/input/flood-prediction-factors/flood.csv')
original_data.head()

In [5]:
original_data.info()

In [6]:
data = data.drop(['id'], axis=1)

# Check if the order and name of columns of both datasets are the same or not
data.columns == original_data.columns

In [7]:
# concat datasets
data = pd.concat([data, original_data], )
data.head()

In [8]:
# check data information
data.info()

In [9]:
# check if there is duplicated data point
data.duplicated().sum()

In [10]:
# Checking number of each column type

num_cols = [col for col in data.columns if data[col].dtype=='int64']
print('number of numerical columns: ', len(num_cols))

cat_cols = [col for col in data.columns if data[col].dtype=='object']
print('number of categorical columns: ', len(cat_cols))

In [11]:
# Checking unique values of each column

for col in num_cols:
    print(f'{col} contains: {set(data[col])} values')

In [15]:
plt.figure(figsize=(4 * 5, 5 * 4))

for i, col in enumerate(num_cols, 1):
    plt.subplot(4, 5, i)
    sns.boxplot(data=data, x=col);

plt.tight_layout()
plt.show()

In [16]:
# Check the number of outliers in each features
data_out = data.copy()
outlier_counts = {}

for col in num_cols:
    q1=data_out[col].quantile(0.25)
    q3=data_out[col].quantile(0.75)
    iqr = q3 - q1
    outlier_counts[col]=[len(data_out[(data_out[col]<q1-1.5*iqr) | (data_out[col]>q3+1.5*iqr)])]

outlier_df = pd.DataFrame(outlier_counts).T
outlier_df.columns = ['Number of Outliers']
outlier_df

In [19]:
# Then We should drop duplicates
check_ouliers = check_ouliers.drop_duplicates()
# Check if still there are duplicated data
check_ouliers.duplicated().sum()

In [21]:
# Replace with edge values
# data_replaced = data.copy()
# for col in num_cols:
#   q1=data_replaced[col].quantile(0.25)
#   q3=data_replaced[col].quantile(0.75)
#   iqr = q3 - q1
#   data_replaced.loc[(data_replaced[col] < q1 - 1.5*iqr), col] = q1 - 1.5*iqr
#   data_replaced.loc[(data_replaced[col] > q3 + 1.5*iqr), col] = q3 + 1.5*iqr

In [22]:
label = data['FloodProbability']
data_replaced = data.drop('FloodProbability', axis=1)

In [30]:
testset = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv')
ids = testset['id']
testset = testset.drop('id', axis=1)

In [31]:
# Replace with edge values
testset_replaced = testset.copy()
# for col in num_cols:
#   q1=testset_replaced[col].quantile(0.25)
#   q3=testset_replaced[col].quantile(0.75)
#   iqr = q3 - q1
#   testset_replaced.loc[(testset_replaced[col] < q1 - 1.5*iqr), col] = q1 - 1.5*iqr
#   testset_replaced.loc[(testset_replaced[col] > q3 + 1.5*iqr), col] = q3 + 1.5*iqr

# New Statistical features
testset_new = transform(testset_replaced)

# scaling
testset_scaled = sc.transform(testset_new)

In [32]:
## You can predict with any of those models, but we prefer the merging one
xgb_pred = xgb_model.predict(testset_scaled)
cat_pred = cat_model.predict(testset_scaled)
lgb_pred = lgb_model.predict(testset_scaled)
preds = 0.33*xgb_pred + 0.33*cat_pred + 0.34*lgb_pred

preds = pd.Series(preds, name='FloodProbability')
submit = pd.concat([ids, preds], axis=1)
submit.to_csv("submission.csv", index=False)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor

In [3]:
train_data=pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv")
test_data=pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv")
sample=pd.read_csv("/kaggle/input/playground-series-s4e5/sample_submission.csv")

In [4]:
train_data.head()

In [5]:
train_data.columns

In [6]:
train_data.info()

In [7]:
train_data.isin([np.nan,"",'?']).sum()

In [8]:
co_mat=train_data.corr()

In [10]:
train_data.skew()

In [11]:
y=train_data['FloodProbability'].copy()
X=train_data.drop(columns=['id','FloodProbability'])

In [12]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,shuffle=True)

In [13]:
ss=StandardScaler()
lg=LinearRegression(fit_intercept=True,n_jobs=-1)
ri=Ridge(random_state=42)
la=Lasso()
pf=PolynomialFeatures(degree=2,interaction_only=True)
pca=PCA()

In [18]:
df=pd.DataFrame(gv.cv_results_)
df

In [19]:
from sklearn.metrics import r2_score

In [20]:
test_data_new=test_data.iloc[:,1:]
head_id=test_data.iloc[:,0]

In [21]:
head_id=pd.DataFrame(head_id,columns=['id'])

In [23]:
test_final=pd.DataFrame(test_final,columns=['FloodProbability'])

In [24]:
output=pd.concat([head_id,test_final],axis=1)

In [25]:
output.to_csv('submission.csv',index=False)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np 

import seaborn as sns 
import matplotlib.pylab as plt
%matplotlib inline

from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
train_file_path='/kaggle/input/playground-series-s4e5/train.csv'
test_file_path='/kaggle/input/playground-series-s4e5/test.csv'
train_df=pd.read_csv(train_file_path)
test_df=pd.read_csv(test_file_path)

In [3]:
train_df

In [4]:
train_df.isnull().sum()

In [5]:
train_df.drop(['id'],axis=1,inplace=True)

In [6]:
train_describe=train_df.describe()
fields=['mean','min','25%','50%','75%','max']
train_describe=train_describe.loc[fields]
train_describe.T

In [7]:
train_df.info()

In [8]:
df_corr=train_df.corr()['FloodProbability'][:-1]
df_corr.sort_values()

In [9]:
skew_limit=0.75
df_skew=train_df.skew()
df_skew

In [11]:
feature_cols = [x for x in train_df.columns if x != "FloodProbability"]

# Create a new column 'fsum' which is the sum of all feature columns for each row
train_df['fsum'] = train_df[feature_cols].sum(axis=1) 

In [12]:
X = train_df.loc[:, train_df.columns != "FloodProbability"]
y = train_df['FloodProbability']

In [14]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=100)

In [17]:
# Define pipeline with Polynomial Regression degree = 2
polynomial = Pipeline([
    ('polynomial', PolynomialFeatures(include_bias=False, degree=2)),
    ('model', LinearRegression())
])

# Fit the instance on the data and then predict the expected value
polynomial.fit(X_train, y_train)
poly2_pred = polynomial.predict(X_test)

# Calculate R^2 score for the polynomial regression model with degree 2
r2_poly2 = r2_score(y_test, poly2_pred)

# Display R^2 score
print(r2_poly2)

In [19]:
predictions_test = polynomial.predict(test_df)

# result dataframe
result = pd.DataFrame({'id' : test_ids, 'FloodProbability' : predictions_test.flatten()}, 
                                columns=['id', 'FloodProbability'])

# print result
result

In [20]:
result.to_csv('/kaggle/working/submission.csv', index=False)

In [2]:
train_set = pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv")
test_set = pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv")

In [3]:
train_set.info()

In [4]:
test_set.info()

In [5]:
uniques = pd.DataFrame(train_set.nunique())
uniques["test"] = pd.DataFrame(test_set.nunique())
uniques = uniques.rename(columns={0:"train_count",
                       "test":"test_count"})
uniques["error"] =  uniques["train_count"]-uniques["test_count"]
uniques

In [6]:
train_set

In [7]:
test_set

In [8]:
len(train_set)

In [9]:
test_set_id = test_set[["id"]]
train_set = train_set.set_index("id")
test_set = test_set.set_index("id")

In [10]:
cat_columns=['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement', 
          'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality',
          'Siltation', 'AgriculturalPractices', 'Encroachments',
          'IneffectiveDisasterPreparedness', 'DrainageSystems',
          'CoastalVulnerability', 'Landslides', 'Watersheds', 
          'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss',
           'InadequatePlanning', 'PoliticalFactors']

In [13]:
train_set.duplicated().value_counts()

In [14]:
train_corr_df = pd.DataFrame(train_set.corr()['FloodProbability'].drop('FloodProbability'))

In [15]:
train_corr_df

In [19]:
full_df.shape

In [20]:
columns = list(full_df.columns)
columns

In [34]:
sorted_corr = corr_df_X.sort_values(by='FloodProbability', ascending=False)

In [41]:
corr_df_X = pd.DataFrame(for_corr.corr()['FloodProbability'].drop('FloodProbability'))
sorted_corr = corr_df_X.sort_values(by='FloodProbability', ascending=False)
corr_more_02 = list(sorted_corr[(sorted_corr > 0.2) | (sorted_corr < -0.2)].dropna().index)

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [49]:
# param_grid = {
#     'iterations': [500, 1000],
#     'depth': [4, 6, 10],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'l2_leaf_reg': [1, 3, 5, 7],
# }


# grid_search = GridSearchCV(estimator=cat_model, param_grid=param_grid, cv=5)
# grid_search.fit(X_train, y_train)

# # Print the best parameters
# print("Best parameters for CatBoostRegressor:")
# print(grid_search.best_params_)

In [50]:
# svc_model = SVR()
# svc_model.fit(X_train, y_train)

# y_pred = svc_model.predict(X_test)

# print(f"Training score {svc_model.score(X_train,y_train)}")
# print(f"R2: {r2_score(y_test,y_pred)}")
# SVR_score = r2_score(y_test,y_pred)
# SVR_Tscore = svc_model.score(X_train,y_train)

# # Training score 0.6892091193403993
# # R2: 0.6879615201044295

In [51]:
# knn_model = KNeighborsRegressor()
# knn_model.fit(X_train, y_train)

# y_pred = knn_model.predict(X_test)

# print(f"Training score {knn_model.score(X_train,y_train)}")
# print(f"R2: {r2_score(y_test,y_pred)}")
# KNN_score = r2_score(y_test,y_pred)
# KNN_Tscore = knn_model.score(X_train,y_train)

# # Training score 0.7389672547153135
# # R2: 0.6103305408520886

In [52]:
# dt_model = DecisionTreeRegressor()
# dt_model.fit(X_train, y_train)

# y_pred = dt_model.predict(X_test)

# print(f"Training score {dt_model.score(X_train,y_train)}")
# print(f"R2: {r2_score(y_test,y_pred)}")
# DT_score = r2_score(y_test,y_pred)
# DT_score = dt_model.score(X_train,y_train)

# # Training score 1.0
# # R2: 0.011373210481789964

In [53]:
# rf_model = RandomForestRegressor()
# rf_model.fit(X_train, y_train)

# y_pred = rf_model.predict(X_test)
# print(f"Training score {rf_model.score(X_train,y_train)}")
# print(f"R2: {r2_score(y_test,y_pred)}")
# RF_score = r2_score(y_test,y_pred)
# RF_Tscore = rf_model.score(X_train,y_train)

# # Training score 0.9473678172793106
# # R2: 0.6280500145463217

In [55]:
# ET_model = ExtraTreesRegressor()
# ET_model.fit(X_train, y_train)

# y_pred = ET_model.predict(X_test)
# print(f"Training score {ET_model.score(X_train,y_train)}")
# print(f"R2: {r2_score(y_test,y_pred)}")
# ET_score = r2_score(y_test,y_pred)
# ET_Tscore = ET_model.score(X_train,y_train)

In [66]:
# SC_model = StackingRegressor(estimators)
# SC_model.fit(X, y)

# y_pred_SC = SC_model.predict(test_set)
# print(f"Training score {SC_model.score(X,y)}")
# # Training score 0.8509888812778332

# # final
# # Training score 0.868946761417187

# # Training score 0.869303107297456

In [68]:
# y_pred_xg = pd.DataFrame(y_pred_xg)
# output_xg = test_set_id
# output_xg["FloodProbability"] = pd.DataFrame(y_pred_xg)
# output_xg = output_xg.set_index("id")
# output_xg.to_csv("FloodProbability_OHE_project_FEng_xg.csv")

In [71]:
# rf_model = RandomForestRegressor()
# SC_model = StackingRegressor([("rf_model",rf_model)])
# SC_model.fit(X, y)

# y_pred_SC = SC_model.predict(test_set)
# print(f"Training score {SC_model.score(X,y)}")

In [72]:
# y_pred_SC = pd.DataFrame(y_pred_SC)
# output_SC = test_set_id
# output_SC["FloodProbability"] = pd.DataFrame(y_pred_SC)
# output_SC = output_SC.set_index("id")
# output_SC.to_csv("FloodProbability_OHE_project_FEng_SC.csv")

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

In [1]:
flood_train =pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv")
flood_test =pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv")

In [2]:
pd.set_option("float_format",'{:.4f}'.format)
print("shape", flood_train.shape)
print("shape", flood_test.shape)
print("Describe_train_data", flood_train.describe())
print("Describe_test_data", flood_test.describe())

In [3]:
print("missing_count_train", flood_train.isna().sum())
print("missing_count_test", flood_test.isna().sum())

In [5]:
for col in flood_train.columns:
    iqr = flood_train[col].quantile(0.75) - flood_train[col].quantile(0.25)
    upper_bound = flood_train[col].quantile(0.75) + 1.5*iqr
    lower_bound = flood_train[col].quantile(0.25) - 1.5*iqr
    flood_train[col] = flood_train[col].clip(lower=lower_bound, upper=upper_bound)

for col in flood_test.columns:
    iqr = flood_test[col].quantile(0.75) - flood_test[col].quantile(0.25)
    upper_bound = flood_test[col].quantile(0.75) + 1.25*iqr
    lower_bound = flood_test[col].quantile(0.25) - 1.5*iqr
    flood_test[col] = flood_test[col].clip(lower= lower_bound, upper = upper_bound)

In [6]:
x = flood_train.drop(["id", "FloodProbability"], axis =1)
y = flood_train["FloodProbability"]

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x)
x_test_scaled = scaler.fit_transform(flood_test.drop(["id"], axis = 1))

x_train, x_test, y_train, y_test = train_test_split(x_train_scaled, y, test_size = 0.3, random_state = 42)

lm = LinearRegression()
lm.fit(x_train, y_train)

y_pred = lm.predict(x_test)
print(f"r2 = {r2_score(y_test, y_pred) : .4f}")

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv")
df.head()

In [3]:
df.info()

In [4]:
X = df.drop(columns=['FloodProbability'])
y = df.FloodProbability

In [5]:
X.shape

In [6]:
y.shape

In [9]:
df_test = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv')

In [10]:
X_test = df_test
X_test

In [15]:
predictions1 = model_1.predict(x_test)

In [18]:
plt.scatter(x_test.PoliticalFactors, y_test, marker='.')
plt.scatter(x_test.PoliticalFactors, predictions1, marker='x' )
plt.show()

In [19]:
submission = pd.DataFrame({'id' : X_test.id,
                           'FloodProbability' : predictions
                          })

In [20]:
submission.head()

In [21]:
submission.to_csv('submission.csv', index=False)

In [1]:
data = pd.read_csv(r'/kaggle/input/playground-series-s4e5/train.csv')
test = pd.read_csv(r'/kaggle/input/playground-series-s4e5/test.csv')
train = data.copy()

In [2]:
train.head()

In [5]:
train.info()

In [6]:
test.info()

In [7]:
X = train.iloc[:, :-1]
y = train.iloc[:, -1]
X.head()

In [8]:
X_train,X_val,y_train,y_val = train_test_split(X,y, test_size=0.1, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

In [14]:
model2 = Sequential([
    Dense(32, activation='relu', input_dim=X_train.shape[1]),
    Dropout(0.1),
    
    Dense(16, activation='relu'),
    Dense(1, activation='linear'),

])

model2.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['r2_score'])
model2.summary()

In [18]:
max(history2.history['val_r2_score'])

In [1]:
train_df=pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')
test_df=pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv')

In [2]:
train_df.head()

In [3]:
train_df.shape

In [4]:
train_df.info()

In [5]:
train_df.describe()

In [6]:
Q1=train_df.quantile(0.25)
Q3=train_df.quantile(0.75)
IQR=Q3-Q1
lower_bound=Q1-1.5*IQR
upper_bound=Q3+1.5*IQR
outliers = pd.DataFrame(np.logical_or(train_df < lower_bound, train_df > upper_bound), columns=train_df.columns)
outliers.sum()

In [7]:
plt.figure(figsize=(25,20))
sns.heatmap(train_df.iloc[:,1:].corr(),cmap="crest",annot=True)

In [8]:
X_train=train_df.iloc[:,1:21]
y_train=train_df.iloc[:,21:]
X_test=test_df.iloc[:,1:21]

In [9]:
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
pipe=make_pipeline(RobustScaler(),RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1]))
pipe.fit(X_train,y_train)
y_pred=pipe.predict(X_test)
y_pred

In [11]:
submission=pd.DataFrame({
    'id':test_df['id'],
    'FloodProbability':y_pred
})

In [ ]:
import pandas as pd
Data = pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv", index_col = None)
Data = Data.reset_index(drop = True) 

In [5]:
#checking for missing values

Data.isna().sum()

In [6]:
# divde the data into target and factors and run feature scaling 


target = Data['FloodProbability']

X = Data.drop(['FloodProbability'], axis = 1)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [7]:
# divide the data into test train

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=42)


In [8]:
# use gradient boosting

import sklearn
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Evaluate the model
r2_score = sklearn.metrics.r2_score(y_test, y_pred)

print("R^2 score:", r2_score)

In [9]:
# linear regression

from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2_score = sklearn.metrics.r2_score(y_test, y_pred)
print("R2 error:", r2_score)


In [10]:
# Decision Tree

from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor()

# Fit the model on the training data
model.fit(X_train, y_train)

# Evaluate the model on the testing data
y_pred = model.predict(X_test)

# Calculate the R^2 score
r2_score = sklearn.metrics.r2_score(y_test, y_pred)

print("R^2 score:", r2_score)


In [11]:
# Polynomial regression

from sklearn.preprocessing import PolynomialFeatures
import sklearn

# Create a polynomial features object with degree 2
poly_features = PolynomialFeatures(degree=2)

# Transform the training and testing features
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.transform(X_test)

# Create a linear regression model
model = LinearRegression()

# Fit the model on the transformed training data
er = model.fit(X_train_poly, y_train)

# Evaluate the model on the transformed testing data
y_pred = model.predict(X_test_poly)

# Calculate the R^2 score
r2_score = sklearn.metrics.r2_score(y_test, y_pred)

print("R^2 score:", r2_score)


In [12]:
#load the test dataset

import pandas as pd
test = pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv")
testing = test.drop(['id'], axis = 1)
testing.isna().sum()


In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
testing = scaler.fit_transform(testing)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Import the basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the training and test sets
train = pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv', index_col = 'id')
test = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv', index_col = 'id')


# Display the shape of the data frames
print('Shape of the:')
print('Training set: ', train.shape)
print('Test set: ', test.shape)

In [3]:
# Create a function to compute the duplicates 
def duplicates(df, name_df):
    print(f'The {name_df} set has %d duplicates.' %df.duplicated().sum())

In [4]:
# Determine the duplicates in the training set
duplicates(train, 'training')

In [5]:
# Determine the duplicates in the test set
duplicates(test, 'test')

In [6]:
print(train.isna().sum().any())
print(test.isna().sum().any())

In [7]:
train.info()

In [8]:
train.describe()

In [9]:
X = train.drop('FloodProbability', axis = 1)
y = train['FloodProbability']

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_test = test.copy()

print('For the training set:')
print('Shape of the features: ', X_train.shape)
print('Shape of the response variable: ', y_train.shape)
print('*'*60)
print('*'*60)
print('For the validation set:')
print('Shape of the features: ', X_val.shape)
print('Shape of the response variable: ', y_val.shape)
print('*'*60)
print('*'*60)
print('For the test set:')
print('Shape of the features: ', X_test.shape)

In [11]:
train_df = pd.concat([X_train, y_train], axis = 1)
train_df.isna().sum().any()

In [12]:
# Display the min and max values of the numerical variables
min_val = [X_train[col].min() for col in X_train.columns]
max_val = [X_train[col].max() for col in X_train.columns]

minmax_df = pd.DataFrame({'feature': X_train.columns,
                         'min_value': min_val,
                         'max_value': max_val}).set_index('feature')
minmax_df

In [13]:
# Format the plots
plt.style.use('seaborn-whitegrid')
plt.rc('figure', autolayout = True)
plt.rc('axes',
      labelweight = 'bold',
      labelsize = 'large',
      titleweight = 'bold',
      titlesize = 14,
      titlepad = 10)

palette = sns.color_palette('Blues')

In [17]:
for col_name in X_train.columns:
    plt.figure(figsize = (12, 5))
    cat_plot(train_df, col_name, re)

plt.tight_layout()
plt.show()

In [19]:
from sklearn.feature_selection import mutual_info_regression

def compute_mi_scores(X, y):
    mi_scores = mutual_info_regression(X, y)
    mi_scores_df = pd.DataFrame({'feature': X.columns,
                                 'mi_scores': mi_scores}).set_index('feature').sort_values(by = 'mi_scores', 
                                                                                           ascending = False)
    return mi_scores_df

In [20]:
compute_mi_scores(X_train, y_train)

In [21]:
X_train.columns

In [22]:
weather_features = ['MonsoonIntensity', 'ClimateChange']
infrastructure_features = ['TopographyDrainage', 'RiverManagement', 'DamsQuality', 
                  'DrainageSystems', 'DeterioratingInfrastructure'] 

environmental_features = ['Deforestation',  'Siltation', 'CoastalVulnerability', 'WetlandLoss', 'Landslides', 'Watersheds']
human_features = ['Urbanization', 'PopulationScore', 'InadequatePlanning', 'PoliticalFactors', 
        'IneffectiveDisasterPreparedness', 'AgriculturalPractices', 'Encroachments']
 

In [25]:
# Costruct a pairplot of the newly engineered features
sns.pairplot(X_train_1[features])

In [29]:
! pip install lightgbm

In [31]:
# Make a list of the algorithms that require scaling with their names
models_need_scaling = [lin_reg, lasso, ridge, svr, kn_reg]
name_models_need_scaling = ['Linear Regression', 'Lasso', 'Ridge', 
                            'Support Vector Regressor', 'K-Neighbors Regressor'] 

train_score, test_score, r2, mae, rmse = [], [], [], [], []

# Apply the function defined in the previous code snippet
for model, model_name in zip(models_need_scaling, name_models_need_scaling):
    building_model(model, model_name, X_train_scaled, X_val_scaled)

# Create a data frame that displays the metrics corresponding to each model 
scores_df1 = pd.DataFrame({'model': name_models_need_scaling,
                          'train_score': train_score,
                          'test_score': test_score,
                          'r2_score': r2,
                          'mean_absolute_error': mae,
                          'root_mean_squared_error': rmse}).sort_values(by = 'r2_score',
                                                                        ascending = False).set_index('model')
scores_df1

In [32]:
# Make a list of the algorithms that don't require scaling with their names
models_without_scaling = [xgb_reg, tree, forest, grad_boost, lgbm] 
name_models_without_scaling = ['XGB Regression', 'Decision Tree Regression', 
                               'Random Forest Regression', 'Gradient Boosting Regressor', 
                               'Light GBM Regressor'] 

train_score, test_score, r2, mae, rmse = [], [], [], [], []

# Apply the function defined in the previous code snippet
for model, model_name in zip(models_without_scaling, name_models_without_scaling):
    building_model(model, model_name, X_train_1[features], X_val_1[features])

# Create a data frame that displays the metrics corresponding to each model 
scores_df2 = pd.DataFrame({'model': name_models_without_scaling,
                          'train_score': train_score,
                          'test_score': test_score,
                          'r2_score': r2,
                          'mean_absolute_error': mae,
                          'root_mean_squared_error': rmse}).sort_values(by = 'r2_score',
                                                                        ascending = False).set_index('model')

scores_df2

In [36]:
params_xgb = {'learning_rate': [0.01, 0.1],
              'max_depth': [5, 7],
              'min_child_weight': [3, 5],
              'n_estimators' : [100, 500]}
xgb_reg = XGBRegressor(objective = 'reg:squarederror')

hyperparameter_tuning(xgb_reg, params_xgb, X_train_1[features])

In [37]:
params_tree = {'max_depth': [10, None],
               'min_samples_split': [2, 5],
               'min_samples_leaf': [1, 2]}
tree = DecisionTreeRegressor()

hyperparameter_tuning(tree, params_tree, X_train_1[features])

In [40]:
models_without_scaling = [tree, xgb]
name_models_without_scaling = ['Decision Tree Regressor', 'XGB Regressor']

train_score, test_score, r2, mae, rmse = [], [], [], [], []

for model, model_name in zip(models_without_scaling, name_models_without_scaling):
    building_model(model, model_name, X_train_1[features], X_val_1[features])

scores_df4 = pd.DataFrame({'model': name_models_without_scaling,
                          'train_score': train_score,
                          'test_score': test_score,
                          'r2_score': r2,
                          'mean_absolute_error': mae,
                          'root_mean_squared_error': rmse}).sort_values(by = 'r2_score',
                                                                        ascending = False).set_index('model')

scores_df4

In [41]:
scores_df = pd.concat([scores_df3, scores_df4], axis = 0).sort_values(by = 'r2_score',
                                                                      ascending = False)
scores_df

In [42]:
y_test = pd.read_csv('/kaggle/input/playground-series-s4e5/sample_submission.csv', index_col = 'id')

In [43]:
xgb = XGBRegressor(learning_rate = 0.1, max_depth = 7,
                   min_child_weight = 3, n_estimators = 500)

xgb.fit(X_train_1, y_train)
y_pred = xgb.predict(X_test_1)

print('Training score: %.4f' %xgb.score(X_train_1, y_train))
print('Mean Absolute Error: %.4f' %mean_absolute_error(y_test, y_pred))
print('Root Mean Squared Error: %.4f' %np.sqrt(mean_squared_error(y_test, y_pred)))

In [44]:
pd.DataFrame(y_pred).describe()

In [1]:
train_df=pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')

In [2]:
train_df.head(10)

In [3]:
train_df.shape

In [4]:
train_df.info()

In [5]:
train_df.drop('id',axis=1,inplace=True)

In [6]:
train_df.describe()

In [7]:
plt.figure(figsize=(20, 15))
for i, column in enumerate(train_df.columns, 1):
    plt.subplot(5, 5, i) 
    sns.boxplot(y=train_df[column])
    plt.title(column)
plt.tight_layout()
plt.show()

In [8]:
Q1=train_df.quantile(0.25)
Q3=train_df.quantile(0.75)
IQR=Q3-Q1
lower_bound=Q1-1.5*IQR
upper_bound=Q3+1.5*IQR
outliers = pd.DataFrame(np.logical_or(train_df < lower_bound, train_df > upper_bound), columns=train_df.columns)
outliers.sum()

In [9]:
sns.histplot(x='FloodProbability',data=train_df,kde=True)

In [10]:
test_df=pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv')

In [11]:
X_train=train_df.iloc[:,:20]
y_train=train_df['FloodProbability']
X_test=test_df.iloc[:,1:]

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [13]:
pipe=make_pipeline(StandardScaler(),LinearRegression())

In [16]:
result=pd.DataFrame({
    'id': test_df['id'],
    'FloodProbability': y_predict
})

In [17]:
result.head(5)

In [18]:
result.to_csv('submission.csv', index=False)
print('Submission file saved!')

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import warnings
# Suppress all warnings
warnings.filterwarnings('ignore')

In [2]:
train= pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv')

In [3]:
train

In [4]:
train.info()

In [5]:
train.isna().sum()

In [6]:
test

In [7]:
test.isna().sum()

In [8]:
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
train.columns

In [11]:
for i in features:
    print(f' {i} values :',train[i].unique())

In [12]:
plt.figure(figsize = (15,30))
# train.drop(columns = ['id'], inplace = True)

for i,col in enumerate(features,1):
    plt.subplot(7,3,i)
    sns.histplot(data  = train, x = train[col],kde = True, color = 'darkorchid', label = 'Train Data')

In [13]:
plt.figure(figsize = (15,30))
features = ['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement',
       'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality',
       'Siltation', 'AgriculturalPractices', 'Encroachments',
       'IneffectiveDisasterPreparedness', 'DrainageSystems',
       'CoastalVulnerability', 'Landslides', 'Watersheds',
       'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss',
       'InadequatePlanning', 'PoliticalFactors']
for i,col in enumerate(features,1):
    plt.subplot(7,3,i)
    sns.countplot(data  = train, x = train[col], color = 'darkorchid',label = 'Train Data')
    sns.countplot(data  = test, x = test[col], color = 'grey',label = 'Test Data')
    plt.legend()

In [14]:
plt.figure(figsize = (15,15))

# Creating variable to store col names of independent features
features = ['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement',
       'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality',
       'Siltation', 'AgriculturalPractices', 'Encroachments',
       'IneffectiveDisasterPreparedness', 'DrainageSystems',
       'CoastalVulnerability', 'Landslides', 'Watersheds',
       'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss',
       'InadequatePlanning', 'PoliticalFactors']

# Plot the boxplot for Outlier Detection 
for i,col in enumerate(features,1):
    plt.subplot(4,5,i)
    sns.boxplot(data  = train, x = train[col], color = 'darkorchid')
plt.tight_layout()

In [15]:
# What is the distribution of flood probability
plt.figure(figsize = (10,5))
sns.histplot(data = train, x = 'FloodProbability',kde = True, color = 'green',fill = True)
plt.title('Flood Probability Distribution Curve')

In [16]:
plt.figure(figsize = (12,6))
sns.heatmap(train[features].corr(),cmap = 'rocket',annot = True, fmt = '0.1f',)
plt.show()

In [20]:
y

In [21]:
y.shape

In [22]:
X_test.shape

In [23]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
import hyperopt
from sklearn.metrics import mean_squared_error, r2_score

In [27]:
# cv.best_score_

In [32]:
cat_params = {
    'iterations' : [300,500,700,800,1000],
    'learning_rate' : [0.03,0.3,0.1,0.01,0.5,0.05],
    'max_depth' : [3,5,7,9,10],
    'l2_leaf_reg' : [1,3,5,7,9,12],
    'bootstrap_type' : ['Bayessian','Bernoulli'],
    'subsample': [0.5,0.6,0.7,0.8,0.9,1],
    
}

In [33]:
cat_cv = RandomizedSearchCV(estimator = CatBoostRegressor(), param_distributions = cat_params, cv = 3, scoring = 'r2',verbose = 3,n_iter = 20)

In [34]:
# cat_cv.fit(X,y)

In [35]:
# cat_cv.best_score_

In [39]:
# y_cat = catBooster.predict(X_test)

In [40]:
from lightgbm import LGBMRegressor

In [44]:
# light_cv.best_params_

In [46]:
LGBMBooster = LGBMRegressor(n_estimators = 1200, learning_rate = 0.1, lambda_l2 = 1, lambda_l1 = 1, feature_extraction = 0.7, bagging_fraction = 1)

In [49]:
gb_params = {
    'n_estimators' : [200,400, 600,800,1000,1200],
    'max_depth' : [1,3,5,7,9],
    'subsample' : [0.3, 0.5, 0.7, 0.9, 1],
    'max_features' :['Auto','log2','sqrt'],
    'learning_rate' : [0.03,0.3,0.1,0.01,0.5,0.05],
    'min_samples_leaf' : [2,4,6,8,10],
    'min_samples_split':[2,5,8,10]
}

In [50]:
gb_cv = RandomizedSearchCV(estimator = GradientBoostingRegressor(), param_distributions = gb_params, cv = 2, n_iter = 10, scoring = 'r2',verbose = 3)

In [56]:
checkFeatureImportance(gb)

In [58]:
checkFeatureImportance(LGBMBooster)

In [59]:
checkFeatureImportance(XGBooster)

In [60]:
## The Separate Models are not giving much better results.
## So Now i will use a voting regressor in order to increase the model accuracy

In [61]:
from sklearn.ensemble import VotingRegressor

In [62]:
estimators = [
    ('cat',catBooster),
    ('xg', XGBooster),
    ('lgbm',LGBMBooster),
    ('gb',gb)
]

In [63]:
voter = VotingRegressor(estimators = estimators, n_jobs = -1, verbose = 1)

In [65]:
y_vote = voter.predict(X_test)

In [67]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression

In [68]:
final_estimator = LinearRegression()

In [71]:
y_stack = stack.predict(X_test)

In [73]:
voter_result = pd.DataFrame(data = y_vote, index = test['id'],columns = ['FloodProbability'])

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import warnings
warnings.filterwarnings('ignore')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
import lightgbm as ltb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
train = pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv")
origin_col = test.columns[1:]

In [2]:
train.shape

In [3]:
train.info()

In [4]:
train.describe().T

In [5]:
print((train.describe().T["std"]).iloc[1:].mean())

In [6]:
test.describe().T

In [7]:
print((test.describe().T["std"]).iloc[1:].mean())

In [8]:
print("Train Histogramme")
sns.set(rc={'figure.figsize': (20, 16)})
train.hist(color='blue');


In [14]:
plt.subplots(figsize=(20,20))
sns.heatmap(train.corr(), annot=True)
plt.show()

In [15]:
# Necessary Packages
from sklearn.feature_selection import SelectKBest, RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [18]:
feats = list(best_features_kbest)

In [19]:
X_test

In [22]:
def generate_new_data(df, num_samples):
    new_data = []
    for _ in range(num_samples):
        new_row = {}
        for col in df.columns:  # Exclure la colonne cible
            mean = df[col].mean()
            std = df[col].std()
            new_row[col] = np.random.normal(mean, std)
        new_data.append(new_row)
    return pd.DataFrame(new_data)

new_data = generate_new_data(train[feats+['FloodProbability']], 1000)

new_data

In [23]:
augmented_train_df = pd.concat([train, new_data])
X_augmented = augmented_train_df[feats]
y_augmented = augmented_train_df["FloodProbability"]
X_train_aug,  X_test_aug, y_train_aug, y_test_aug = train_test_split(X_augmented, y_augmented, test_size=0.2, random_state=42)

model = ltb.LGBMRegressor()
model.fit(X_train_aug, y_train_aug)

y_pred_aug = model.predict(X_test_aug)

mse = mean_squared_error(y_test_aug, y_pred_aug)
print("Score", r2_score(y_test_aug, y_pred_aug))
print(f'Mean Squared Error: {mse}')

In [1]:
os.listdir('/kaggle/input/playground-series-s4e5')

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as ax

In [3]:
df_train=pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')

In [4]:
df_train.info()

In [5]:
df_train.head()

In [6]:
df_train.isna().sum()

In [7]:
df_train.duplicated().sum()

In [8]:
df_train.drop(columns=['id'],inplace=True)
df_train.columns

In [9]:
df_train.describe().T

In [10]:
NumCols=df_train.columns
NumCols

In [11]:
j=0
k=0
fig,ax=plt.subplots(nrows=7,ncols=3,figsize=(18,30))
for i,fea in enumerate(NumCols[:-1]):
    if i==0:
        j=0
        k=0
    elif i%3==0:
        j=j+1
        k=0
    else:
        k=k+1

    ax[j,k].set_xlabel(fea)
    sns.scatterplot(data=df_train,x=df_train[fea],y=df_train['FloodProbability'],ax=ax[j,k])

In [14]:
df_train=pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')
df_train.drop(columns=['id'],inplace=True)


In [15]:
NumCols=df_train.select_dtypes(exclude=['object','boolean']).columns
NumCols

In [16]:
df_out=pd.DataFrame()
for i in NumCols:
    q1=0
    q3=0
    q1=df_train[i].quantile(.25)
    q3=df_train[i].quantile(.75)
    iqr=q3-q1
    lf=q1-1.5*iqr
    uf=q3+1.5*iqr
    df_out=df_train[(df_train[i]>=lf) & (df_train[i]<=uf)]
df_out

In [17]:
corr=df_train.corr()*100
plt.figure(figsize=(30,12))
sns.heatmap(corr,mask=np.triu(corr),fmt='.2f',annot=True,cmap="Greens",linewidths=.5, annot_kws={"size": 15})


In [19]:
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.metrics import *

In [20]:
Xout_train,Xout_test,yout_train,yout_test=train_test_split(Xout,yout,test_size=.3,random_state=42)
print(Xout_train.shape,Xout_test.shape)


In [23]:
Xout1_train,Xout1_test,yout1_train,yout1_test=train_test_split(Xout1,yout1,test_size=.3,random_state=42)
print(Xout1_train.shape,Xout1_test.shape)

In [25]:
df_train=pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')
df_train.drop(columns=['id'],inplace=True)
df_train.columns

In [37]:
df_test.iloc[:,1:].columns

In [42]:
df=pd.read_csv('/kaggle/working/submission.csv')
df

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from xgboost import XGBRegressor

In [2]:
df_train=pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv")
df_test=pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv")
#
print("Train:",len(df_train))
sample_sub=pd.read_csv("/kaggle/input/playground-series-s4e5/sample_submission.csv")
#
df_train.head()

In [10]:
test_df = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv')
test_df=getFeats(test_df)
test_df

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification, make_regression
from sklearn.feature_selection import mutual_info_regression, mutual_info_classif, RFECV, SequentialFeatureSelector
from sklearn.ensemble import HistGradientBoostingClassifier,HistGradientBoostingRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from lightgbm import LGBMClassifier,LGBMRegressor
from sklearn.metrics import r2_score
from scipy.stats import skew, kurtosis



In [12]:
num_features=int(np.ceil(1.5*(num_inf+num_red+num_dup)))
columns=[f'I_{item}' for item in range(num_inf)]+[f'R_{item}' for item in range(num_red)]+[f'D_{item}' for item in range(num_dup)]  
i=0
while len(columns)<num_features:
    columns.append(f'random_{i}')
    i+=1
dt_class,y_class=make_classification(n_samples=n_samples,
                         n_informative=num_inf,
                         n_redundant=num_red,
                         n_repeated=num_dup,
                         n_features=num_features,
                         flip_y=1e-2,
                         shuffle=False,
                         n_classes=n_classes)
dt_class=pd.DataFrame(dt_class,columns=columns)

In [32]:
dt_gen['random_target']=np.random.normal(y_gen.mean(),y_gen.std(),y_gen.shape[0])

In [4]:
train_data = pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')
test_data = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv')

In [5]:
initial_features = list(test_data.drop(['id'], axis = 1).columns)
initial_features

In [6]:
unique_vals = []
for df in [train_data, test_data]:
    for col in initial_features:
        unique_vals += list(df[col].unique())

unique_vals = list(set(unique_vals))
unique_vals

In [ ]:
n_repeats, n_splits, seed = 3, 5, 1984
target = "FloodProbability"

import os
path, oofs_path = ".", "dataset/"    # when I work on my own PC
if os.path.isdir("/kaggle/input/"):  #             on Kaggle
    path, oofs_path = "/kaggle/input", "/kaggle/input/pss4e05-dataset-oofs-preds/"

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 300)

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import r2_score
def score_(y_trn, y_pred):
    return r2_score(y_trn, y_pred)

from sklearn.linear_model import Ridge

In [1]:
end = "\033[0m" # reset

bold       = "\033[1m" ; resetbold       = "\033[21m"
underline  = "\033[4m" ; resetunderline  = "\033[24m"
blink      = "\033[5m" ; resetblink      = "\033[25m"
reverse    = "\033[7m" ; resetreverse    = "\033[27m"

Default      = "\033[39m" ; Black        = "\033[30m" ; White        = "\033[97m"
Red          = "\033[31m" ; LightRed     = "\033[91m"
Green        = "\033[32m" ; LightGreen   = "\033[92m"
Yellow       = "\033[33m" ; LightYellow  = "\033[93m"
Blue         = "\033[34m" ; LightBlue    = "\033[94m"
Magenta      = "\033[35m" ; LightMagenta = "\033[95m"
Cyan         = "\033[36m" ; LightCyan    = "\033[96m"
DarkGray     = "\033[90m" ; LightGray    = "\033[37m"

BackgroundDefault     = "\033[49m"  ; BackgroundBlack        = "\033[40m"
BackgroundRed         = "\033[41m"  ; BackgroundLightRed     = "\033[101m"
BackgroundGreen       = "\033[42m"  ; BackgroundLightGreen   = "\033[102m"
BackgroundYellow      = "\033[43m"  ; BackgroundLightYellow  = "\033[103m"
BackgroundBlue        = "\033[44m"  ; BackgroundLightBlue    = "\033[104m"
BackgroundMagenta     = "\033[45m"  ; BackgroundLightMagenta = "\033[105m"
BackgroundCyan        = "\033[46m"  ; BackgroundLightCyan    = "\033[106m"
BackgroundDarkGray    = "\033[100m" ; BackgroundLightGray    = "\033[47m"

bold_blue = bold + LightBlue

In [2]:
train = pd.read_csv(f'{path}/playground-series-s4e5/train.csv', index_col = "id")
test = pd.read_csv(f'{path}/playground-series-s4e5/test.csv', index_col = "id")

In [3]:
models = [f"xgb{i}" for i in [8, 10, 11, 12, 13]] + [f"lgbm{i}" for i in [5, 9, 16]] + [
    f"cb{i}" for i in [2, 3, 7, 9, 12, 13]] + ["bestlgbmcorr"] + [f"autog{i}" for i in [3, 5]]

oofs, preds = [], []
for r in range(n_repeats):
    
    for i, m in enumerate(models):
        
        if i == 0:
            oofs.append(pd.read_csv(f"{oofs_path}oof_{m}_r{r+1}.csv", index_col = "id").rename(columns = {target:m}))
            preds.append(pd.read_csv(f"{oofs_path}pred_{m}_r{r+1}.csv", index_col = "id").rename(columns = {target:m}))
        else:
            oofs[r] = pd.concat([oofs[r], pd.read_csv(f"{oofs_path}oof_{m}_r{r+1}.csv", index_col = "id").rename(columns = {target:m})], axis = 1)        
            preds[r] = pd.concat([preds[r], pd.read_csv(f"{oofs_path}pred_{m}_r{r+1}.csv", index_col = "id").rename(columns = {target:m})], axis = 1)        

    oofs[r] = oofs[r]
    oofs[r][target] = train[target]
    preds[r] = preds[r]

In [7]:
def do_ensemble(train = train, target = target, features = test.columns, test = test, 
                folds = StratifiedKFold(n_splits = n_splits, random_state = seed, shuffle = True), 
                my_model = None, digit = 6, lib_metric = "R2", do_feat_imp = False):
    
    feat_imp = FeatureImportance(n_permutations = 2, lib_metric = lib_metric)
    val_scores, trn_scores = [], []
    oofs = pd.Series(0.0, name = target, index = train.index)
    preds = pd.Series(0.0, name = target, index = test.index)
    intercept, coefs = [], pd.DataFrame(0.0, index = features, columns = [f"fold{i+1}" for i in range(folds.get_n_splits())])
    
    for fold, (trn_idx, val_idx) in enumerate(folds.split(train, (train[target]*400).astype(np.int16).astype("category"))):
        
        X_trn, y_trn = train.loc[trn_idx, features], train.loc[trn_idx, target]
        X_val, y_val = train.loc[val_idx, features], train.loc[val_idx, target]
            
        if (fold) % n_splits == 0:
            oof = pd.Series(0.0, name = target, index = train.index)
            pred = pd.Series(0.0, name = target, index = test.index)
        
        # Training
        my_model.fit(X_trn, y_trn)
        intercept.append(my_model.intercept_) ; coefs[f"fold{fold+1}"] += my_model.coef_

        # Score & prediction
        trn_scores.append(score_(y_trn, my_model.predict(X_trn)))
        oof.iloc[val_idx] += my_model.predict(X_val)
        val_scores.append(score_(y_val, oof.loc[val_idx]))
        oofs.iloc[val_idx] += oof.loc[val_idx]
        pred += my_model.predict(test[features]) / folds.get_n_splits()
            
        scores = []
        for f in features:
            scores.append(score_(y_val, X_val[f]))
        
        my_color = Green if np.max(scores) < val_scores[-1:][0] else LightRed
        print(f"    Fold {fold + 1:2} : {lib_metric} {val_scores[-1:][0]:.{digit}f} (max : {my_color}{np.max(scores):.{digit}f}{end} - ", end='')
        my_color = Green if score_(y_val, X_val[features].mean(axis = 1)) < val_scores[-1:][0] else LightRed
        print(f"mean : {my_color}{np.mean(scores):.{digit}f}{end})", end='')        
        print(f" | in train {trn_scores[-1:][0]:.{digit}f}", end='')
        print(f' | Overfitting {trn_scores[-1:][0] - val_scores[-1:][0]:.{digit}f}')
        if (fold + 1) % n_splits == 0:
            print(f'  {bold}Repeat {(fold + 1 ) // n_splits:2} : OOF {lib_metric} {bold_blue}{score_(train[target], oof):.{digit}f}{end} | ', end='')
            print(f'Mean {lib_metric} {np.mean(val_scores[-n_splits:]):.{digit}f}')
            preds += pred
        
        # Feature importance
        if do_feat_imp:
            for feat in features:
                for i in range(feat_imp.n_permutations):
                    df = X_val.copy()
                    df[feat] = pd.Series(np.random.permutation(df[feat]), index=df.index).astype(X_val[feat].dtypes)
                    feat_imp.append(feat, score_(y_val, my_model.predict(df)), val_scores[-1:][0])
    # end of loop
    
    scores = []
    for f in features:
        scores.append(score_(train[target], train[f]))
    print(f'OOF {lib_metric} {bold_blue}{score_(train[target], oofs):.{digit}f}{end}', end='')
    my_color = Green if np.max(scores) < score_(train[target], oofs) else LightRed
    print(f' (max was {my_color}{np.max(scores):.{digit}f}{end} - ', end='')
    my_color = Green if np.mean(scores) < score_(train[target], oofs) else LightRed
    print(f'mean {my_color}{np.mean(scores):.{digit}f}{end}) | ', end='')
    print(f'Mean {lib_metric} : {bold_blue}{np.mean(val_scores):.{digit}f}{end}({bold+Red}±{np.std(val_scores):.{digit}f}){end}) | ', end='')
    print(f'Overfitting : {np.mean(trn_scores)-np.mean(val_scores):.{digit}f}')
    
    feat_imp.plot()
    
    return {"intercept":intercept, "coefs":coefs.transpose(), "oofs_score":score_(train[target], oofs), "oofs":oofs, "preds":preds}

In [1]:
# データの読込
train = pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv", index_col="id")
print(train.shape)
test = pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv", index_col="id")
print(test.shape)

In [2]:
# trainデータの確認
train.head()

In [3]:
# testデータの確認
test.head()

In [4]:
# 欠損値の確認：trainデータ
train.info()

In [5]:
# 欠損値の確認：testデータ
test.info()

In [6]:
# 重複値の確認

print(train.drop(columns="FloodProbability").duplicated().sum())
print(test.duplicated().sum())

In [7]:
# train
train.describe().T

In [8]:
# test
test.describe().T

In [9]:
min_max_tr = train.describe().T[['min', 'max']].add_suffix('_train')
min_max_te = test.describe().T[['min', 'max']].add_suffix('_test')
nunique_tr = train.nunique().rename('nunique_train')
nunique_te = test.nunique().rename('nunique_test')
counts_tr = pd.Series({ft: [train[ft].value_counts().round(3).iloc[:5].to_dict()] for ft in train.columns}, name='top_5_counts_train')
counts_te = pd.Series({ft: [test[ft].value_counts().round(3).iloc[:5].to_dict()] for ft in test.columns}, name='top_5_counts_test')

min_max_count = pd.concat(
    [
        min_max_tr,
        min_max_te,
        nunique_tr,
        nunique_te,
        counts_tr,
        counts_te,
    ],
    axis=1)
min_max_count

In [10]:
# 各特徴量の分布

n_bins = 50
histplot_params = {
    'kde': True,
    'alpha': 0.4,
    'stat': 'percent',
    'bins': n_bins
}

columns = test.columns[test.dtypes!=object]
n_cols = 3
n_rows = math.ceil(len(columns)/n_cols)
fig, ax = plt.subplots(n_rows, n_cols, figsize=(20, n_rows*4))
ax = ax.flatten()

for i, column in enumerate(columns):
    plot_axes = [ax[i]]
    sns.kdeplot(train[column], label='Train', ax=ax[i])
    sns.kdeplot(test[column], label='Test', ax=ax[i])
    
    ax[i].set_title(f'{column} Distribution')
    ax[i].set_xlabel(None)
    
    plot_axes = [ax[i]]
    handles = []
    labels = []
    for plot_ax in plot_axes:
        handles += plot_ax.get_legend_handles_labels()[0]
        labels += plot_ax.get_legend_handles_labels()[1]
        plot_ax.legend().remove()

for i in range(i+1, len(ax)):
    ax[i].axis('off')
    
fig.suptitle('Dataset Feature Distributions\n\n\n', ha='center', fontweight='bold', fontsize=25)
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.95), fontsize=25, ncol=3)
plt.tight_layout()

In [14]:
# 実際に実行

from sklearn.preprocessing import MinMaxScaler

# 特徴量のリスト
features = ["MonsoonIntensity", "TopographyDrainage", "RiverManagement", "Deforestation", 
            "Urbanization", "ClimateChange", "DamsQuality", "Siltation", 
            "AgriculturalPractices", "Encroachments", "IneffectiveDisasterPreparedness", 
            "DrainageSystems", "CoastalVulnerability", "Landslides", "Watersheds", 
            "DeterioratingInfrastructure", "PopulationScore", "WetlandLoss", 
            "InadequatePlanning", "PoliticalFactors"]

# MinMaxScalerのインスタンス化
scaler = MinMaxScaler()

# trainデータの特徴量のスケーリング
train[features] = scaler.fit_transform(train[features])

# testデータの特徴量のスケーリング
test[features] = scaler.transform(test[features])


In [19]:
# sample_submission.csvの読込
sub = pd.read_csv("/kaggle/input/playground-series-s4e5/sample_submission.csv")
sub.head()

In [1]:
# Save a file_path to data
train_data_path = "/kaggle/input/playground-series-s4e5/train.csv"
test_data_path = "/kaggle/input/playground-series-s4e5/train.csv"

In [2]:
# Read & Store data
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

In [3]:
# Display basic info about the datasets
print("Train Data Shape:", train_data.shape)
print("Test Data Shape:", test_data.shape)

In [4]:
# Display first 5 rows of train data
print("Train Data Preview:")
print(train_data.head())

In [5]:
# Display summary statistics
print("Train Data Summary Statistics:")
print(train_data.describe())

In [6]:
# Check for missing values
print("Train Data Missing Values:")
print(train_data.isnull().sum())

In [7]:
plt.figure(figsize=(8, 6))
sns.histplot(train_data['FloodProbability'], bins=20, kde=True)
plt.title('Distribution of FloodProbability')
plt.xlabel('FloodProbability')
plt.ylabel('Count')
plt.show()

In [9]:
# Encoding categorical features and target variable
label_encoders = {}
for column in train_data.select_dtypes(include=['object']).columns:
    if column != 'FloodProbability':
        le = LabelEncoder()
        combined_data = pd.concat([train_data[column], test_data[column]], axis=0).fillna('missing')
        le.fit(combined_data)
        train_data[column] = le.transform(train_data[column].fillna('missing'))
        test_data[column] = le.transform(test_data[column].fillna('missing'))
        label_encoders[column] = le

In [10]:
# Encode the target variable
le_target = LabelEncoder()
train_data['FloodProbability'] = le_target.fit_transform(train_data['FloodProbability'])
label_encoders['FloodProbability'] = le_target

In [11]:
# Drop 'id' column from both training and test data
X_train = train_data.drop(columns=['FloodProbability', 'id'])
y_train = train_data['FloodProbability']
test_features = test_data.drop(columns=['id'])

In [15]:
# Calculate R2 score
r2 = r2_score(y_train, y_pred)
print(f"Validation R2 Score: {r2:.4f}")

In [18]:
# Create a submission DataFrame
submission = pd.DataFrame({
    'id': test_data['id'],
    'FloodProbability': test_preds
})

In [19]:
# Print first five rows of submission file
print("First five rows of submission file: ", submission.head())

In [20]:
# Save the submission file
submission.to_csv('submission.csv', index=False)
print('Submission file saved!')

In [ ]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [1]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [2]:
flood_dataset = pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv")

In [3]:
flood_dataset.describe().T

In [4]:
def add_features(df):
    addfeatures = df.columns.tolist()
    df['std_features'] = df[addfeatures].std(axis=1)
    df['mean_features'] = 0.1*df[addfeatures].mean(axis=1)
    df['median_features'] = 0.1*df[addfeatures].median(axis=1)
    df['max_features'] = df[addfeatures].max(axis=1)
    df['min_features'] = df[addfeatures].min(axis=1)    
    return df

In [5]:
flood_dataset_df=flood_dataset.copy()
flood_dataset_df.drop(['id'], axis = 1, inplace = True)

In [6]:
df = flood_dataset_df
df1 = df.drop(["FloodProbability"], axis=1)

In [7]:
flood_dataset_df = add_features(df1)
y = df["FloodProbability"]

In [8]:
def add_more_features(flood_dataset_df):
    flood_dataset_df['weather'] = - flood_dataset_df['MonsoonIntensity'] - flood_dataset_df['ClimateChange']
    flood_dataset_df['structure'] = flood_dataset_df['TopographyDrainage'] + flood_dataset_df['RiverManagement'] - flood_dataset_df['Deforestation'] + flood_dataset_df['DamsQuality'] + flood_dataset_df['Encroachments'] + flood_dataset_df['DrainageSystems'] - flood_dataset_df['Watersheds'] - flood_dataset_df['DeterioratingInfrastructure'] + flood_dataset_df['WetlandLoss'] 
    flood_dataset_df['population'] = - flood_dataset_df['Urbanization'] * flood_dataset_df['PopulationScore']
    flood_dataset_df['soil_infrastructure'] = - flood_dataset_df['Siltation'] + flood_dataset_df['AgriculturalPractices'] - flood_dataset_df['Landslides']
    flood_dataset_df['preparedness'] = - flood_dataset_df['IneffectiveDisasterPreparedness'] + flood_dataset_df['CoastalVulnerability'] - flood_dataset_df['InadequatePlanning'] - flood_dataset_df['PoliticalFactors'] 
    return flood_dataset_df

In [11]:
flood_dataset_df.info()

In [12]:
flood_dataset_df.head(5)

In [13]:
flood_dataset_df.to_csv('train_without_preprocessing.csv', index=False)

In [14]:
del flood_dataset_df

In [15]:
flood_dataset = pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv")

flood_dataset_df=flood_dataset.copy()
flood_dataset_df.drop(['id'], axis = 1, inplace = True)

df = flood_dataset_df
flood_dataset_df = add_features(df)

flood_dataset_df = add_more_features(flood_dataset_df)

flood_dataset_df.info()

In [16]:
flood_dataset_df.to_csv('test_data.csv', index=False)

In [17]:
flood_dataset_df = pd.read_csv("/kaggle/working/train_without_preprocessing.csv")

In [18]:
flood_dataset_df.columns

In [23]:
flood_dataset_df = pd.read_csv("/kaggle/working/test_data.csv")
flood_dataset_df = add_manipulated_features(flood_dataset_df)
flood_dataset_df.to_csv('test_data_featured.csv', index=False)

In [27]:
flood_dataset_test = pd.read_csv("/kaggle/working/test_data_featured.csv")

In [28]:
flood_dataset_test.shape

In [32]:
catb = CatBoostRegressor()
catb_model = catb.fit(X_train, Y_train,
                     verbose = 0)

y_pred_catb = catb_model.predict(X_test)

catb_rmse_calculator = np.sqrt(mean_squared_error(np.log(Y_test), np.log(y_pred_catb)))
catboost_r2_metric = r2_score(y_test, y_pred_catb)

print(f"Category Boosting RMSE Metric: {catb_rmse_calculator}")
print(f"Category Boosting R-squared Metric: {catboost_r2_metric}")

In [33]:
testddata = pd.read_csv("/kaggle/working/test_data_featured.csv")

input_test = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv')

id_no = input_test['id']
test_pred_lgbm = lgbm_model.predict(testddata)

submission = pd.DataFrame({
    'id': id_no,
    'FloodProbability': test_pred_lgbm
})

print(submission.head())
submission.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Options
pd.set_option('display.max_columns',50)

In [1]:
sample = pd.read_csv('/kaggle/input/playground-series-s4e5/sample_submission.csv')
sample.head()

In [2]:
df_train = pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv', index_col='id')
df_test = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv', index_col='id')

In [3]:
df_train.head()

In [4]:
df_train.info()

In [5]:
df_test.info()

In [6]:
df_train.describe().T

In [7]:
df_test.describe().T

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [9]:
for col in df_test.columns:
  sns.catplot(data=df_train, x=col, kind='count')

In [10]:
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

In [11]:
X = df_train.drop('FloodProbability', axis=1)
y = df_train['FloodProbability']

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [16]:
submission.to_csv('/kaggle/working/submission.csv', index=False)

In [2]:
# Create a copy of the DataFrame
cleaned_train_df = df.drop(['id', 'FloodProbability'], axis=1).copy()

# Define the multiplier for identifying outliers based on standard deviation
multiplier = 3

# Calculate the upper outlier threshold for each column
upper_thresholds = cleaned_train_df.mean() + multiplier * cleaned_train_df.std()

# Identify rows containing outliers in each column
outlier_rows = (cleaned_train_df > upper_thresholds).any(axis=1)

# Drop outlier rows from the DataFrame
cleaned_train_df = cleaned_train_df[~outlier_rows]
cleaned_target_df = df.loc[~outlier_rows,'FloodProbability']

# Print the shape of the cleaned DataFrame
print("Original DataFrame shape:", df.shape)
print("Cleaned DataFrame shape:", cleaned_train_df.shape)


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import keras_tuner as kt

import tensorflow as tf
from tensorflow.keras import backend as K

def r2_score_metric(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return 1 - SS_res/(SS_tot + K.epsilon())


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(cleaned_train_df, cleaned_target_df, test_size=0.3, random_state=11)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)))

    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(tf.keras.layers.Dense(
            units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),
            activation='relu',
            kernel_regularizer=tf.keras.regularizers.l1_l2(
                l1=hp.Float('l1_' + str(i), 1e-5, 1e-2, sampling='LOG'),
                l2=hp.Float('l2_' + str(i), 1e-5, 1e-2, sampling='LOG')
            )
        ))
        model.add(tf.keras.layers.Dropout(hp.Float('dropout_' + str(i), 0.0, 0.5, step=0.1)))

    model.add(tf.keras.layers.Dense(1))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Float('learning_rate', 1e-4, 1e-2, sampling='LOG')
        ),
        loss='mean_squared_error',
        metrics=[r2_score_metric]
    )
    return model

# Create a tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Search for the best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=11, validation_split=0.2)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test_scaled)
r2 = r2_score(y_test, y_pred)
print("R2 Score of the best model:", r2)


In [2]:
df = df.drop("id", axis=1)

In [9]:
plt.matshow(df[train_columns + ["FloodProbability"]].corr())
plt.show()

In [18]:
plt.barh(train_columns, model_catboost_base.feature_importances_)

In [14]:
# Define base features by dropping the target variable column
BASE_FEATURES = df.drop(columns='flood_probability').columns


In [20]:
# Calculate weights for each feature group
weights = [1/len(value) for key, value in feature_list.items()]

# Initialize columns with zeros for each feature group
for key in feature_list.keys():
    df[key] = 0

# Apply weighted sum for each feature group
for weight, (name, features) in zip(weights, feature_list.items()):
    for feature in features:
        df[name] += (df[feature] * weight)


In [21]:
df[feature_list.keys()].head()
        

In [22]:
# Create a figure with custom size
plt.figure(figsize=(11, 10))

# Generate a heatmap of correlations for numerical columns, excluding base features
sns.heatmap(
    df.drop(columns=BASE_FEATURES).select_dtypes(include=np.number).corr(),  # DataFrame correlation matrix
    annot=True,  # Show correlation values in cells
    cmap=['#638889', '#678788', '#6c8788', '#718788', '#768788', '#7b8788', '#808788', '#858788', '#8a8787', '#8f8787', 
          '#948687', '#998687', '#9e8687', '#a38687', '#a88687', '#ac8686', '#b18686', '#b68686', '#bb8686', '#c08686',
          '#c58586', '#ca8586', '#cf8585', '#d48585', '#d98585', '#de8585', '#e38585', '#e88585', '#ed8585', '#f28585'],
    annot_kws={"fontsize": 7}  # Annotation font size
)

# Display the heatmap
plt.show()

In [28]:
# Define a function to calculate the root mean squared error (RMSE)
def rmse(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred))

# Create a custom scorer 'rmse_scorer' using make_scorer
# 'greater_is_better=False' indicates that lower values of the scoring function are better
rmse_scorer = make_scorer(rmse, greater_is_better=False)

In [32]:
# Access the best estimator from the grid search
best_estimator = grid_search.best_estimator_

# Access the regressor from the best estimator
regressor = best_estimator.named_steps['model']

# Check if the regressor has feature_importances_ attribute
if hasattr(regressor, 'feature_importances_'):
    # Get feature importances
    feature_importances = regressor.feature_importances_
    
    # Access the 'features' pipeline from the best estimator
    features_pipeline = best_estimator.named_steps['features']
    
    # Manually specify the feature names after transformation
    numeric_feature_names = X_train.columns.tolist()  # Assuming all columns are numeric
    
    # Calculate the total importance sum of all features
    total_importance = feature_importances.sum()
    
    # Calculate the percentage contribution of each feature
    feature_importance_percentage = (feature_importances / total_importance) * 100
    
    # Create a DataFrame with the percentage contribution of each feature
    features_percentage_df = pd.DataFrame({'Feature': numeric_feature_names,
                                           'Importance (%)': feature_importance_percentage})
    
    # Sort the DataFrame by importance in descending order
    features_percentage_df = features_percentage_df.sort_values(by='Importance (%)', ascending=False)
    
    # Display the top features with their percentage contributions
    print(features_percentage_df.head())





In [ ]:
import cupy as cp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import optuna
import gc

from time import time
from joblib import load, dump
from matplotlib.ticker import MaxNLocator,MultipleLocator
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import Ridge
from sklearn.ensemble import VotingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.base import clone
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from tqdm import tqdm
import sklearn.metrics as skm

import warnings
warnings.filterwarnings("ignore")

In [14]:
# For output text color
end = "\033[0m" ; bold = "\033[1m" ; LightRed = "\033[91m" ; LightBlue = "\033[94m"
BlueBold = LightBlue + bold ; RedBold = LightRed + bold

In [20]:
def make_features(data, train_data=True, scaler=None):
    '''
    For feature engineering of both train and test data. Returns the data with new features added in standardized form.
    The standardization parameters are learned only from the train data so as to avoid data leakage.
    '''
    df = data.copy()
    with tqdm(total=12 ,desc='processing...') as pbar:
        sort_cols = [f'col{i+1}' for i in range(len(orig_features))]
        
        df['sum'] = df.sum(axis=1)         
        pbar.update(1)
        
        df['linear_ft'] = df['sum'].isin(np.arange(72, 76)).astype(int)
        pbar.update(1)
        
        df['median'] = df[orig_features].median(axis=1)         
        pbar.update(1)
        
        df['std'] = df[orig_features].std(axis=1)         
        pbar.update(1)
        
        df[sort_cols] = np.sort(df[orig_features].values, axis=1)         
        pbar.update(1)
        
        for u in unique:
            df[f'count_{u}'] = (df[orig_features] == u).sum(axis=1)
        pbar.update(1)
        
        df['mean'] = df[orig_features].mean(axis=1)
        pbar.update(1)
        
        df['max'] = df[orig_features].max(axis=1) 
        pbar.update(1)               
        
        df['min'] = df[orig_features].min(axis=1)
        pbar.update(1)
        
        df['skew'] = df[orig_features].skew(axis=1)
        pbar.update(1)
        
        df['kurt'] = df[orig_features].kurt(axis=1)
        pbar.update(1)
        
        if train_data:
            scaler = StandardScaler()
            scaler.fit(df)
            df[df.columns.to_list()] = scaler.transform(df.values)
            pbar.update(1)
        else:
            df[df.columns.to_list()] = scaler.transform(df.values)
            pbar.update(1)
        
    return df, scaler

In [21]:
train_df, scaler = make_features(X)
dump(train_df,'train_df.joblib')
train_df.head()

In [22]:
test_df, _ = make_features(test.iloc[:,1:], train_data=False, scaler=scaler)
dump(test_df,'test_df.joblib')
test_df.head()

In [34]:
fixed_params_xgrf = {
    'device'       : 'cuda',
    'booster'      : 'gbtree',
    'learning_rate': 1,
    'tree_method'  : 'hist',
    'n_estimators' : 1,
    'objective'    : 'reg:squarederror',
    'n_jobs'       : -1,
}

In [42]:
# Creating the predictions dataset for final prediction
ensemble_train = np.c_[xgr_tr_pred1, cbr_tr_pred2, lgr_tr_pred3, xgrf_tr_pred4, ridge_tr_pred5]
ensemble_test = np.c_[xgr_ts_pred1, cbr_ts_pred2, lgr_ts_pred3, xgrf_ts_pred4, ridge_ts_pred5]

np.save('ensemble_train.npy', ensemble_train)
np.save('ensemble_test.npy', ensemble_test)

In [20]:
from sklearn.model_selection import train_test_split

x = df_train.drop("FloodProbability" , axis = 1)
y = df_train[['FloodProbability']]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=42)

In [22]:
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow-gpu
!pip install --upgrade keras

In [36]:
best_model.save("best_flood_prediction_model.h5")

In [23]:
df=pd.DataFrame(df)

In [24]:
# df.hist(bins=50,figsize=(20,15))
# plt.show()

In [29]:
from sklearn.feature_selection import SelectKBest

In [36]:
new_df=pd.concat([df,feature+0.3,feature+0.3,feature+0.3,df.median(axis=1)+0.2,feature*df.median(axis=1)],axis=1,ignore_index=True)

In [37]:
testf=pd.concat([test,test.mean(axis=1)+0.3,test.mean(axis=1)+0.3,test.mean(axis=1)+0.3,test.median(axis=1)+0.2,test.mean(axis=1)*test.median(axis=1)],axis=1,ignore_index=True)

In [38]:
new_df.hist(bins=50,figsize=(20,15))
plt.show()

In [39]:
# new_y=pd.concat([y,y.mean()],axis=0,ignore_index=True)

In [42]:
xtrain,xval,ytrain,yval=train_test_split(new_df,y,random_state=0,test_size=0.15,stratify=y,shuffle=True)

In [44]:
# model2=LinearRegression(fit_intercept=True)
# model2=RandomForestRegressor(random_state=0,oob_score=True,warm_start=True,ccp_alpha=0.3)
# model=KMeans()
# model2=GradientBoostingRegressor()
# model5=PolynomialFeatures(degree=2, interaction_only=True)

In [8]:
def detect_outliers(df, threshold=1.5):
    outlier_columns = []
    
    for col in df.columns:
        # Calculate the first and third quartiles
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        
        # Interquartile range (IQR)
        iqr = q3 - q1
        
        # Define the lower and upper bounds for outlier detection
        lower_bound = q1 - threshold * iqr
        upper_bound = q3 + threshold * iqr
        
        # Find outliers
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        
        if not outliers.empty:
            outlier_columns.append(col)
    
    return outlier_columns

outlier_cols = detect_outliers(df_train)
print("Columns with outliers: \n", outlier_cols)

In [9]:
numerical_cols = df_train.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df_train.select_dtypes(include=['object']).columns

# Plot numerical columns against 'FloodProbability' using scatter plots
for col in numerical_cols:
    plt.figure(figsize=(8, 6))
    sns.scatterplot(data=df_train, x=col, y='FloodProbability', alpha=0.7)
    plt.title(f'{col} vs. FloodProbability')
    plt.xlabel(col)
    plt.ylabel('FloodProbability')
    plt.grid(True)
    plt.show()

# Plot categorical columns against 'FloodProbability' using bar plots
for col in categorical_cols:
    plt.figure(figsize=(8, 6))
    sns.barplot(data=df_train, x=col, y='FloodProbability', alpha=0.7)
    plt.title(f'{col} vs. FloodProbability')
    plt.xlabel(col)
    plt.ylabel('FloodProbability')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()

In [14]:
'''No need to perform standardization / normalization as all the features have a small and comparable scale'''

In [15]:
correlation_matrix = df_train.corr(method='pearson')
plt.figure(figsize=(20, 15))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Pearson Correlation Matrix')
plt.show()

In [19]:
from sklearn.model_selection import train_test_split

x = df_train.drop("FloodProbability" , axis = 1)
y = df_train[['FloodProbability']]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=42)

In [24]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

rf_model = RandomForestRegressor()

rf_model.fit(x_train, y_train)

# Make predictions on the training set
y_train_pred = rf_model.predict(x_train)

# Make predictions on the test set
y_test_pred = rf_model.predict(x_test)

# Calculate RMSE, MSE, and MAE for training set
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)

# Calculate RMSE, MSE, and MAE for test set
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Print the results
print("Training set:")
print("RMSE:", rmse_train)
print("MSE:", mse_train)
print("MAE:", mae_train)

print("\nTest set:")
print("RMSE:", rmse_test)
print("MSE:", mse_test)
print("MAE:", mae_test)


In [37]:
id

In [38]:
output_df = pd.concat([id, output_df], axis=1)
# output_df.name = 'FloodProbability'

In [39]:
output_df

In [41]:
output_df

In [4]:
X = X.drop(["id"], axis=1)

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [6]:
X

In [7]:
# Splitting on test and train data
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=42)

In [16]:
# Predicting test data for submission
predictions = model.predict(df_check)

In [9]:
scaler = MinMaxScaler()

X = scaler.fit_transform(X)

In [13]:
# Mean Absolute Error
lr_mae = mean_absolute_error(y_val, y_pred)
print(f'Mean Absolute Error: {np.round(lr_mae, 5)}')

In [15]:
# R-square score
lr_r2 = r2_score(y_val, y_pred)
print(f'R-Squared Score: {np.round(lr_r2 * 100, 3)}')

In [35]:
# R-square score
gbr_r2 = r2_score(y_val, y_pred)
print(f'R-Squared Score: {np.round(gbr_r2 * 100, 3)}')

In [49]:
# Transform scaler
X_test = scaler.transform(X_test)

In [13]:
for col in cols:
    col_iqr = iqr(train[col])
    Q1, Q3 = np.quantile(train[col], [0.25, 0.75])
    
    # Convert outliers to np.nan
    train.loc[train[col] < (Q1 - 1.5*col_iqr), col] = np.nan
    train.loc[train[col] > (Q3 + 1.5*col_iqr), col] = np.nan    

In [17]:
scaler = StandardScaler()
X[X.columns] = scaler.fit_transform(X)
test[test.columns] = scaler.transform(test)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state =  987)

In [12]:
# Checking Skewness of each feature

skew = {}
for col in num_cols:
    skewness = data[col].skew()
    skew[col] = [skewness]
    print(f'Skewness of {col} feature is {skewness}')

In [17]:
# Check how many data points are outliers
data_check = data.copy()
check_ouliers = pd.DataFrame()

for col in num_cols:
    q1=data_check[col].quantile(0.25)
    q3=data_check[col].quantile(0.75)
    iqr = q3 - q1
    if_outliers = data_check[(data_check[col] < q1 - 1.5*iqr) | (data_check[col] > q3 + 1.5*iqr)]
    check_ouliers = pd.concat([check_ouliers, if_outliers])

In [18]:
# Check if there are duplicated data point
check_ouliers.duplicated().sum()

In [20]:
# Determine the number of outliers in the dataset
print(len(check_ouliers))

In [23]:
def transform(df):
    features = df.columns.tolist()
    sorted_new =  np.sort(df.values, axis=1)
    df['mean_features'] = 0.1 * df[features].mean(axis=1)
    df['std_features'] = df[features].std(axis=1)
    df['max_features'] = df[features].max(axis=1)
    df['min_features'] = df[features].min(axis=1)
    df['median_features'] = 0.1 * df[features].median(axis=1)
    df['sum_features'] = 0.1 * df[features].sum(axis=1)
    df1 = pd.concat([df, pd.DataFrame(sorted_new, index=df.index)], axis=1)
    df1 = df1.drop(features, axis=1)
    df1.columns = df1.columns.astype('str')
    return df

data_new = transform(data_replaced)

In [24]:
X = data_new
y = label

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)

In [14]:
pipe=Pipeline([('ss',ss)])
X_train_t=pipe.fit_transform(X_train)
X_test_t=pipe.transform(X_test)

In [13]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math

from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import Ridge, SGDRegressor, LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor,AdaBoostRegressor, VotingRegressor, StackingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, f_regression, chi2, f_classif, RFE

from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score

In [17]:
X = pd.DataFrame(train_set.drop(columns=["FloodProbability"]))
y = pd.DataFrame(train_set["FloodProbability"])

In [18]:
full_df = pd.concat([X, test_set])

In [21]:
full_df["Mon-Climate"] = (full_df["MonsoonIntensity"] + full_df["ClimateChange"])/2
full_df["Topo-Drainage"] = (full_df["TopographyDrainage"] + full_df["DrainageSystems"])/2
full_df["Deforest-LandS"] = (full_df["Deforestation"] + full_df["Landslides"])/2
full_df["Urban-Planning"] = (full_df["Urbanization"] + full_df["InadequatePlanning"])/2
full_df["Dams-Siltation"] = (full_df["DamsQuality"] + full_df["Siltation"])/2
full_df["Wetland-CoastVul"] = (full_df["WetlandLoss"] + full_df["CoastalVulnerability"])/2
full_df["RivMan- WaterS"] = (full_df["RiverManagement"] + full_df["Watersheds"])/2
full_df["Agri-Encro"] = (full_df["AgriculturalPractices"] + full_df["Encroachments"])/2
full_df["InefDisPrep-DetInfra"] = (full_df["IneffectiveDisasterPreparedness"] + full_df["DeterioratingInfrastructure"])/2
full_df["PopScore-Urban"] = (full_df["PopulationScore"] + full_df["Urbanization"])/2
full_df["PFactors-Planning"] = (full_df["PoliticalFactors"] + full_df["InadequatePlanning"])/2

In [22]:
full_df['mean'] = 0.1 * full_df[columns].mean(axis=1)
full_df['std'] = 0.1 * full_df[columns].std(axis=1)
full_df['var'] = 0.1 * full_df[columns].var(axis=1)
full_df['skew'] = full_df[columns].skew(axis=1)
full_df['kurtosis_features'] = full_df[columns].kurtosis(axis=1)
full_df['cv'] = full_df['std'] / full_df['mean']

In [23]:
full_df['geom_mean'] = 0.1 * np.exp(np.log(full_df[columns]+1).mean(axis=1))
full_df['harm_mean'] = 0.1 * len(columns) / (1 / (full_df[columns]+1)).sum(axis=1)

In [24]:
 full_df['double'] = 0.1 * np.sqrt(np.sum(np.tril(full_df[columns].to_numpy()[:, :, None] * full_df[columns].to_numpy()[:, None, :], k=-1), axis=(1,2)) /380)

In [25]:
# full_df_dum = pd.get_dummies(full_df[['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement', 
#                               'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality',
#                               'Siltation', 'AgriculturalPractices', 'Encroachments',
#                               'IneffectiveDisasterPreparedness', 'DrainageSystems',
#                               'CoastalVulnerability', 'Landslides', 'Watersheds', 
#                               'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss',
#                               'InadequatePlanning', 'PoliticalFactors']], columns = columns)
# full_df = full_df.join(full_df_dum)

In [26]:
# full_df_X_scale  = full_df.loc[:, list(cat_columns)]

In [27]:
scaler = StandardScaler()
full_df[['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement', 
                              'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality',
                              'Siltation', 'AgriculturalPractices', 'Encroachments',
                              'IneffectiveDisasterPreparedness', 'DrainageSystems',
                              'CoastalVulnerability', 'Landslides', 'Watersheds', 
                              'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss',
                              'InadequatePlanning', 'PoliticalFactors']] = scaler.fit_transform(full_df[['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement', 
                                                                                                         'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality',
                                                                                                         'Siltation', 'AgriculturalPractices', 'Encroachments',
                                                                                                         'IneffectiveDisasterPreparedness', 'DrainageSystems',
                                                                                                         'CoastalVulnerability', 'Landslides', 'Watersheds', 
                                                                                                         'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss',
                                                                                                         'InadequatePlanning', 'PoliticalFactors']])

In [28]:
full_df

In [29]:
scaler = StandardScaler()
full_df[["Mon-Climate", "Topo-Drainage", "Deforest-LandS", "Urban-Planning", 
           "Dams-Siltation", "Wetland-CoastVul", "RivMan- WaterS", "Agri-Encro", 
            "InefDisPrep-DetInfra", "PopScore-Urban", "PFactors-Planning"]] = scaler.fit_transform(full_df[["Mon-Climate", "Topo-Drainage", "Deforest-LandS", "Urban-Planning", 
                                                                                                            "Dams-Siltation", "Wetland-CoastVul", "RivMan- WaterS", "Agri-Encro",
                                                                                                            "InefDisPrep-DetInfra", "PopScore-Urban", "PFactors-Planning"]])

In [30]:
X = full_df[0:1117957]
test_set = full_df[1117957:]

In [31]:
for_corr = X.join(y)

In [32]:
corr_df_X = pd.DataFrame(for_corr.corr()['FloodProbability'].drop('FloodProbability'))

In [33]:
corr_df_X

In [35]:
sorted_corr

In [36]:
corr_more_02 = list(sorted_corr[(sorted_corr > 0.2) | (sorted_corr < -0.2) ].dropna().index)

In [37]:
corr_more_02

In [38]:
full_df['mean_highest'] = 0.1 * full_df[corr_more_02].mean(axis=1)

In [39]:
X = full_df[0:1117957]
test_set = full_df[1117957:]

In [40]:
for_corr = X.join(y)

In [42]:
sorted_corr.loc[["mean_highest"]]

In [44]:
X.describe()

In [54]:
# bc_model = BaggingRegressor(estimator=rf_model, n_estimators=100)
# bc_model.fit(X_train, y_train)

# y_pred = bc_model.predict(X_test)
# print(f"Training score {bc_model.score(X_train,y_train)}")
# print(f"R2: {r2_score(y_test,y_pred)}")
# BagReg_score = r2_score(y_test,y_pred)
# BagReg_Tscore = bc_model.score(X_train,y_train)

In [64]:
# result_df

In [70]:
# y_pred_cat = pd.DataFrame(y_pred_cat)
# output_cat = test_set_id
# output_cat["FloodProbability"] = pd.DataFrame(y_pred_cat)
# output_cat = output_cat.set_index("id")
# output_cat.to_csv("FloodProbability_OHE_project_FEng_cat_fulldf.csv")

In [7]:
y_pred_sc = lm.predict(x_test_scaled)
submission = pd.DataFrame({'id':flood_test.id, "FloodProbability" : y_pred_sc})
submission.to_csv( "submission.csv")
submission

In [13]:
X_train, x_test, Y_train, y_test = train_test_split(X,y, test_size=0.3)

In [3]:
train.set_index(train['id'], inplace=True)
train.drop(columns=['id'], inplace=True)
train.head()

In [4]:
# do the same for the test
test.set_index(test['id'], inplace=True)
test.drop(columns=['id'], inplace=True)
test.head()

In [17]:
# Define needed variables
tr_loss = history2.history['loss']
val_loss = history2.history['val_loss']
index_loss = np.argmin(val_loss)
val_lowest = val_loss[index_loss]

Epochs = [i+1 for i in range(len(tr_loss))]
loss_label = f'best epoch= {str(index_loss + 1)}'

# Plot training history
plt.figure(figsize= (20, 8))
plt.style.use('fivethirtyeight')

plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
plt.scatter(index_loss + 1, val_lowest, s= 150, c= 'blue', label= loss_label)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout
plt.show()


In [12]:
submission.to_csv('submission.csv', index=False)
print('Submission file saved!')

In [1]:
# Drop id column from dataset

Data.drop(columns = ['id'], inplace = True)
Data.head(2)


In [3]:
# normalizing the data

import numpy as np

for col in Data.columns:
  Data[col] = np.log(Data[col] + 1)  # Add 1 to avoid log(0)

In [4]:

# Create a heatmap/correlation matrix
plt.figure(figsize=(20, 9))
sns.heatmap(Data.corr(), annot=True)
plt.title("Heatmap/Correlation Matrix of Data")
plt.show()

In [15]:
Floodprobability = pd.DataFrame(FloodProbability)

Submission = pd.concat([test['id'], Floodprobability], axis = 1)

Submission.to_csv('Submission.csv', index = False)

In [16]:
Submission

In [23]:
X_train_1 = X_train.copy()
X_val_1 = X_val.copy()
X_test_1 = X_test.copy()

X_train_1['weather_average'] = round(X_train_1[weather_features].mean(axis = 1), 2)
X_val_1['weather_average'] = round(X_val_1[weather_features].mean(axis = 1), 2)
X_test_1['weather_average'] = round(X_test_1[weather_features].mean(axis = 1), 2)

X_train_1['infrastructure_average'] = round(X_train_1[infrastructure_features].mean(axis = 1), 2)
X_val_1['infrastructure_average'] = round(X_val_1[infrastructure_features].mean(axis = 1), 2)
X_test_1['infrastructure_average'] = round(X_test_1[infrastructure_features].mean(axis = 1), 2)

X_train_1['environmental_average'] = round(X_train_1[environmental_features].mean(axis = 1), 2)
X_val_1['environmental_average'] = round(X_val_1[environmental_features].mean(axis = 1), 2)
X_test_1['environmental_average'] = round(X_test_1[environmental_features].mean(axis = 1), 2)

X_train_1['human_average'] = round(X_train_1[human_features].mean(axis = 1), 2)
X_val_1['human_average'] = round(X_val_1[human_features].mean(axis = 1), 2)
X_test_1['human_average'] = round(X_test_1[human_features].mean(axis = 1), 2)

X_train_1.head()

In [24]:
features = ['human_average', 'environmental_average', 'infrastructure_average', 'weather_average']

In [28]:
X_train_scaled = X_train_1[features].copy()
X_val_scaled = X_val_1[features].copy()
X_test_scaled = X_test_1[features].copy()

# Implement min-max scaling to make the range of all variables between 0 and 1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train_scaled)
X_val_scaled = scaler.transform(X_val_scaled)
X_test_scaled = scaler.transform(X_test_scaled)

In [35]:
params_kn = {'n_neighbors' : [5, 7, 9],
             'weights' : ['uniform', 'distance'],
             'metric' : ['minkowski', 'euclidean', 'manhattan']}
kn_reg = KNeighborsRegressor()
hyperparameter_tuning(kn_reg, params_kn, X_train_scaled)

In [10]:
features = ['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement',
       'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality',
       'Siltation', 'AgriculturalPractices', 'Encroachments',
       'IneffectiveDisasterPreparedness', 'DrainageSystems',
       'CoastalVulnerability', 'Landslides', 'Watersheds',
       'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss',
       'InadequatePlanning', 'PoliticalFactors']

In [17]:
from sklearn.preprocessing import StandardScaler

In [18]:
from sklearn.model_selection import train_test_split,GridSearchCV, RandomizedSearchCV
scaler = StandardScaler()
X = train[features]
X = scaler.fit_transform(X)
test_new = test.drop(columns = ['id'])
X_test = scaler.fit_transform(test_new)
y = train[['FloodProbability']]

In [19]:
X.shape

In [36]:
# cat_cv.best_params_

In [41]:
light_params = {
    'n_estimators' :[200,400,600,800,1000,1200],
    'learning_rate' : [0.03,0.3,0.1,0.01,0.5,0.05],
    'bagging_fraction' : [0.5,0.7,1],
    'feature_fraction': [0.4,0.5,0.7,0.9,1],
    'lambda_l1' : [1,3,5,7,9,12],
    'lambda_l2' : [1,3,5,7,9,12]  
}

In [42]:
light_cv = RandomizedSearchCV(estimator = LGBMRegressor(), param_distributions = light_params, cv = 3, scoring = 'r2',verbose = 1,n_iter = 10)

In [45]:
# light_cv.best_score_

In [54]:
from sklearn.inspection import permutation_importance

In [55]:
# Create a function in order to show bar plots 

def checkFeatureImportance(model, top_n = 20, random_state = 42):
    
    if hasattr(model, 'coef_'):
        importances = np.round(model.coef_,2)
        method = 'models\'s coefficients'
    elif hasattr(model, 'feature_importances_'):
        importances = np.round(model.feature_importances_,2)
        method = 'Feature importances'
    
    else:
        result = permutation_importance(model, X, y, scoring = 'r2', n_jobs = -1)
        importances = result.importances_mean
        method = 'Permutation importance'
    
    cols = [i for i in features]
    
    importance_df = pd.DataFrame({'Feature': cols, 'Importance' : importances })
    importance_df.head(top_n).plot(kind='bar', x='Feature', y='Importance', legend=False,color = 'darkorchid')
    plt.title(f'Top {top_n} Feature Importance ({method})')
    plt.show()
        
        

In [57]:
checkFeatureImportance(catBooster)

In [66]:
y_vote

In [72]:
y_stack

In [74]:
stack_result = pd.DataFrame(data = y_stack, index = test['id'],columns = ['FloodProbability'])

In [12]:
## 1. Creating new features

def mode_or_mean(row):
    mode = row.mode()
    if len(mode) == 0:  # No mode found
        return row.mean()
    else:
        return mode.iloc[0]


train["sum"] = train[origin_col].mean(axis=1)
test["sum"] = test[origin_col].mean(axis=1)
train["max*min"] = train[origin_col].max(axis=1)*train[origin_col].min(axis=1)
test["max*min"] = test[origin_col].max(axis=1) * test[origin_col].min(axis=1)
train["min"] = train[origin_col].min(axis=1)
test["min"] = test[origin_col].min(axis=1)
train["max-min"] = train[origin_col].max(axis=1)-train[origin_col].min(axis=1)
test["max-min"] = test[origin_col].max(axis=1) -  test[origin_col].min(axis=1)
train["max"] = train[origin_col].max(axis=1)
test["max"] = test[origin_col].max(axis=1)
train["var"] = train[origin_col].var(axis=1)
test["var"] = test[origin_col].var(axis=1)
train["median"] = train[origin_col].median(axis=1)
test["median"] = test[origin_col].median(axis=1)
train["mean*median"] = train[origin_col].mean(axis=1)*train[origin_col].median(axis=1)
test["mean*median"] = test[origin_col].mean(axis=1) * test[origin_col].median(axis=1)
train["sum-div-max"] = train[origin_col].sum(axis=1)/train[origin_col].max(axis=1)
test["sum-div-max"] = test[origin_col].sum(axis=1)/test[origin_col].max(axis=1)
train["std-max"] = train[origin_col].std(axis=1)-train[origin_col].max(axis=1)
test["std-max"] = test[origin_col].std(axis=1)-test[origin_col].max(axis=1)
train["sum-std"] = train[origin_col].sum(axis=1)/train[origin_col].std(axis=1)
test["sum-std"] = test[origin_col].sum(axis=1)/test[origin_col].std(axis=1)
train["sum-var"] = train[origin_col].sum(axis=1) / train[origin_col].var(axis=1)
train["sum-median"] = train[origin_col].sum(axis=1) / train[origin_col].median(axis=1)
train["sum-median-std"] = train[origin_col].sum(axis=1) / (train[origin_col].median(axis=1) - train[origin_col].std(axis=1))
#train["mode"] = train[origin_col].apply(lambda row: mode_or_mean(row), axis=1)
test["sum-var"] = test[origin_col].sum(axis=1) / test[origin_col].var(axis=1)
test["sum-median"] = test[origin_col].sum(axis=1) / test[origin_col].median(axis=1)
test["sum-median-std"] = test[origin_col].sum(axis=1) / (test[origin_col].median(axis=1) - test[origin_col].std(axis=1))
#test["mode"] = test[origin_col].apply(lambda row: mode_or_mean(row), axis=1)

train

In [13]:
train.replace([np.inf, -np.inf], 0, inplace=True)
train.fillna(0, inplace=True)
test.replace([np.inf, -np.inf], 0, inplace=True)
test.fillna(0, inplace=True)

In [17]:
# Selecting features
## Methode 1: SelectKBest with f_regression
selector = SelectKBest(k=10) # nombre of features
X_train_kbest = selector.fit_transform(X_train, y_train)
best_features_kbest = X.columns[selector.get_support()]

# Methode 2: RFE (Recursive Feature Elimination) with Linear Regression
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=10) 
selector = selector.fit(X_train, y_train)
best_features_rfe = X.columns[selector.support_]

# Methode 3: SequentialFeatureSelector (SFS) with Lasso
lasso = Lasso(alpha=0.01) 
sfs = SFS(lasso, k_features=10, forward=True, scoring='neg_mean_squared_error', cv=5)
sfs = sfs.fit(X_train.values, y_train.values)
best_features_sfs = X.columns[list(sfs.k_feature_idx_)]

# Showing results
print("Meilleures caractéristiques avec SelectKBest:", best_features_kbest)
print("Meilleures caractéristiques avec RFE:", best_features_rfe)
print("Meilleures caractéristiques avec SequentialFeatureSelector:", best_features_sfs)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os

In [18]:
Xout=df_out.iloc[:,:-1]
yout=df_out['FloodProbability']
Xout

In [21]:
RFEModel = LinearRegression()
rfe=RFE(RFEModel)
rfe.fit(Xout,yout)
selected_indices = [i for i,data in enumerate(rfe.support_) if data == True]
selected_column_name = [NumCols[i] for i in selected_indices]
selected_column_name

In [22]:
Xout1=df_out[selected_column_name]
yout1=df_out['FloodProbability']
print(Xout1.shape,yout1.shape)

In [26]:
Xall=df_train.iloc[:,:-1]
yall=df_train.iloc[:,-1]
print(Xall.shape,yall.shape)

In [28]:
StdSca = StandardScaler()
Xall_train_std=StdSca.fit_transform(Xall_train)
Xall_test_std=StdSca.transform(Xall_test)
Xall_train_std

In [35]:
RFReg = RandomForestRegressor()
RFReg.fit(Xall_train_std, yall_train)
RFReg_pred =RFReg.predict(Xall_test_std)
mseRF=mean_squared_error(yall_test,RFReg_pred)
rmseRF=np.sqrt(mseRF)
maeRF=mean_absolute_error(yall_test,RFReg_pred)
r2scoreRF = r2_score(yall_test,RFReg_pred)
print('Mean Squared Error : ', mseRF)
print('Root Mean Squared Error : ', rmseRF)
print('Mean Absolute Error : ', maeRF)
print('R2 Score : ' ,r2scoreRF)



In [36]:
df_test=pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv')
df_test

In [39]:
test_predictions = LR.predict(test_std)
test_predictions

In [3]:
num_cols=['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement',
       'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality',
       'Siltation', 'AgriculturalPractices', 'Encroachments',
       'IneffectiveDisasterPreparedness', 'DrainageSystems',
       'CoastalVulnerability', 'Landslides', 'Watersheds',
       'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss',
       'InadequatePlanning', 'PoliticalFactors']

unique_vals = []
for df in [df_train, df_test]:
    for col in num_cols:
        unique_vals += list(df[col].unique())

unique_vals = list(set(unique_vals))
#
def getFeats(df):
    
    scaler = StandardScaler()
    
    df['ClimateAnthropogenicInteraction'] = (df['MonsoonIntensity'] + df['ClimateChange']) * (df['Deforestation'] + df['Urbanization'] + df['AgriculturalPractices'] + df['Encroachments'])
    df['InfrastructurePreventionInteraction'] = (df['DamsQuality'] + df['DrainageSystems'] + df['DeterioratingInfrastructure']) * (df['RiverManagement'] + df['IneffectiveDisasterPreparedness'] + df['InadequatePlanning'])
    
    df['sum'] = df[num_cols].sum(axis=1)
    df['std']  = df[num_cols].std(axis=1)
    df['mean'] = df[num_cols].mean(axis=1)
    df['max']  = df[num_cols].max(axis=1)
    df['min']  = df[num_cols].min(axis=1)
    df['mode'] = df[num_cols].mode(axis=1)[0]
    df['median'] = df[num_cols].median(axis=1)
    df['q_25th'] = df[num_cols].quantile(0.25, axis=1)
    df['q_75th'] = df[num_cols].quantile(0.75, axis=1)
    df['skew'] = df[num_cols].skew(axis=1)
    df['kurt'] = df[num_cols].kurt(axis=1)
    df['sum_72_76'] = df['sum'].isin(np.arange(72, 76))
    for i in range(10,100,10):
        df[f'{i}th'] = df[num_cols].quantile(i/100, axis=1)
    df['harmonic'] = len(num_cols) / df[num_cols].apply(lambda x: (1/x).mean(), axis=1)
    df['geometric'] = df[num_cols].apply(lambda x: x.prod()**(1/len(x)), axis=1)
    df['zscore'] = df[num_cols].apply(lambda x: (x - x.mean()) / x.std(), axis=1).mean(axis=1)
    df['cv'] = df['std'] / df['mean']
    df['Skewness_75'] = (df[num_cols].quantile(0.75, axis=1) - df[num_cols].mean(axis=1)) / df[num_cols].std(axis=1)
    df['Skewness_25'] = (df[num_cols].quantile(0.25, axis=1) - df[num_cols].mean(axis=1)) / df[num_cols].std(axis=1)
    df['2ndMoment'] = df[num_cols].apply(lambda x: (x**2).mean(), axis=1)
    df['3rdMoment'] = df[num_cols].apply(lambda x: (x**3).mean(), axis=1)
    df['entropy'] = df[num_cols].apply(lambda x: -1*(x*np.log(x)).sum(), axis=1)
    df['rng'] = df['max'] - df['min']
    
    for v in unique_vals:
        if v<16:
            df['cnt_{}'.format(v)] = (df[num_cols] == v).sum(axis=1)
    
    df[num_cols] = scaler.fit_transform(df[num_cols])
    
    return df

In [4]:
df_train['typ']=0
df_test['typ']=1
#
df_all=pd.concat([df_train,df_test],axis=0)
df_all=getFeats(df_all)
df_all.head()

In [5]:
df_train=df_all[df_all['typ']==0]
df_test=df_all[df_all['typ']==1]
#
cleaned_train_df=df_train.drop(['id','FloodProbability','typ'],axis=1)
cleaned_target_df=df_train['FloodProbability']
#
feats=list(cleaned_trained_df.columns)

In [1]:
transpose=lambda x: list(zip(*x))

In [9]:
def classify_columns(X,cat_limit=10,print_output=False):
    '''
    Naive features classificator
    '''
    
    total=X.shape[0]

    categorical_cols=[(item,X[item].nunique()) for item in X.columns  if str(X[item].dtype) in ['object','category'] ]
   
    catcols= [i[0] for i in categorical_cols]
    discrete_cols=[(item,X[item].nunique()) for item in X.columns  if not item in catcols and X[item].nunique()<cat_limit and 'int' in str(type(X[item][0])) ] 
    discols=[i[0] for i in discrete_cols]
    numerical_cols=[(item,X[item].nunique()) for item in X.columns if not (item in catcols or item in discols)]
    numcols=[i[0] for i in numerical_cols]
    X[numcols]=X[numcols].astype(float)
    nan_cols=[(item,count) for item in X.columns if (count:=sum(X[item].isnull()))]
    duplicates=X.duplicated().sum()
    if print_output:
        print(f'{total} rows')
        if nan_cols:
            print('Cols. with NaN:')
            for item in nan_cols:
                print(f'\t{item[0]:>20}({"Cat." if item[0] in catcols else "Num."}): {item[1]} ({item[1]/total:.2%})')
        if categorical_cols:
            print('\nCategorical cols.:')
            for item in categorical_cols:
                print(f'\t{item[0]:>20}: {X[item[0]].unique()}')
        if discrete_cols:
            print('\nDiscrete cols.:')
            for item in discrete_cols:
                print(f'\t{item[0]:>20}: {X[item[0]].unique()}')
        if numerical_cols:
            print('\nNumerical cols.:')

            for item in numerical_cols:
                print(f'\t{item[0]:>20}({X[item[0]].dtype}): {item[1]} unique values')
        if duplicates:
            print(f'Duplicated rows: {duplicates} ({duplicates/total:.2%})')
        else:
            print('No duplicates')
            
    
    return {'nan_cols':[item[0] for item in  nan_cols],
            'cat_cols':[item[0] for item in categorical_cols],
            'disc_cols':[item[0] for item in discrete_cols],
            'num_cols':[item[0] for item in numerical_cols]}

In [11]:
#sklearn make_classification stuff
n_samples=10000
n_classes=3 # #classes in the categorical output
num_inf=3 # # informative features
num_red=3  # # redundant features
num_dup=3  # # duplicated features



In [13]:
def sample_distribution(p,n=1):
    '''
    Simple function for sample an empirical probability distribution function.
    Input:
        p: list of float or int. Empirical distribution function. Each number correspond to frequency of a class. The classes are taken as 0, 1, ...
        n: number of samples
    Output:
        int list. n random numbers accordingly p
    '''
    p=np.array(p)
    p=p/p.sum()
    p=p.cumsum()
    #print(p)
    r=[]
    for i in range(n):
        throw=np.random.uniform()
        r.append(len([item for item in p if item<throw]))
    return r


In [14]:
from collections import Counter

empirical=dict(Counter(y_class))
empirical={key:empirical[key] for key in range(n_classes)}

dt_class['random_target']=sample_distribution(list(empirical.values()),n=y_class.shape[0])

In [15]:
model=LGBMClassifier(verbose=-1,random_state=0)#HistGradientBoostinggenifier(random_state=0)#
n_jobs=1

In [23]:
dt_reg['random_target']=np.random.normal(y_reg.mean(),y_reg.std(),y_reg.shape[0])

In [30]:
n_features=9
n_cols=int(np.ceil(1.5*n_features))

n_samples=10000


dt_gen=pd.DataFrame(np.random.poisson(5,(n_samples,n_features)),columns=[f'feat_{i}' for i in range(n_features)]) 

std=dt_gen.mean(axis=1).std()

rcols=[f'random_{i}' for i in range(n_cols-n_features)]

dt_gen[rcols]=np.random.poisson(5,(n_samples,n_cols-n_features))#np.random.normal(0,std,(n_samples,n_cols-n_features))

y_gen=0.075*(np.mean(dt_gen.iloc[:,:n_features],axis=1)+np.random.normal(0,std*0.5,(n_samples,)))+0.3


In [39]:
for data in [dt_gen]:
    orig=[item for item in data.columns if 'feat' in item]
    sort_cols=[f'rank_{i}' for i in range(len(orig))]
    data[sort_cols]=np.sort(data[orig],axis=1)
    #data['Sum']=np.sum(data[orig],axis=1)
    #data['average']=np.mean(data[orig],axis=1)
    data['st_dev']=np.std(data[orig],axis=1)
    #data['skew']=skew(data[orig],axis=1)
    #data['kurt']=kurtosis(data[orig],axis=1)
    data['Q25']=np.quantile(data[orig],0.25,axis=1)
    data['Q50']=np.quantile(data[orig],0.5,axis=1)
    data['Q75']=np.quantile(data[orig],0.75,axis=1)
    data['IQR']=data['Q75']-data['Q25']
    #data['LogProd']=np.log((1+data[orig]).prod(axis=1))
    data[["nb_inf4", "nb_inf3", "nb_inf2", "nb_sup6", "nb_sup7", "nb_sup8"]] = 0
    for f in orig:
        data.loc[data[f] <=4 , "nb_inf4"] += 1
        data.loc[data[f] >=6 , "nb_sup6"] += 1
        data.loc[data[f] <=3 , "nb_inf3"] += 1
        data.loc[data[f] >=7 , "nb_sup7"] += 1
        data.loc[data[f] <=2 , "nb_inf2"] += 1
        data.loc[data[f] >=8 , "nb_sup8"] += 1
    data=data.fillna(0)

In [ ]:
pip install --upgrade scikit-learn

In [13]:
# 実際に実行

# trainデータに対する処理
train["Urbanization_Drainage_Ratio"] = train.apply(lambda row: row["DrainageSystems"] / row["Urbanization"] if row["Urbanization"] != 0 else 0, axis=1)

# testデータに対する処理
test["Urbanization_Drainage_Ratio"] = test.apply(lambda row: row["DrainageSystems"] / row["Urbanization"] if row["Urbanization"] != 0 else 0, axis=1)

In [15]:
# 対数変換によるスケーリング

# 対数変換対象の特徴量リスト
features = [
    "MonsoonIntensity",
    "TopographyDrainage",
    "RiverManagement",
    "Deforestation",
    "Urbanization",
    "ClimateChange",
    "DamsQuality",
    "Siltation",
    "AgriculturalPractices",
    "Encroachments",
    "IneffectiveDisasterPreparedness",
    "DrainageSystems",
    "CoastalVulnerability",
    "Landslides",
    "Watersheds",
    "DeterioratingInfrastructure",
    "PopulationScore",
    "WetlandLoss",
    "InadequatePlanning",
    "PoliticalFactors"
]

# trainデータに対する対数変換
for feature in features:
    train[feature] = np.log1p(train[feature])

# testデータに対する対数変換
for feature in features:
    test[feature] = np.log1p(test[feature])


In [16]:
# モデルの構築
target = "FloodProbability"
X = train.drop(columns=target)
y = train[target]

model = XGBRegressor(
    learning_rate=0.3,
    max_depth=5,
    subsample=0.6,
    colsample_bytree=0.9)
regressor = model

In [12]:
# Initialize RandomForestRegressor
rf_model = RandomForestRegressor(random_state=42)

In [16]:
# Prepare the submission file
test_ids = test_data['id']
test_features = test_data.drop(columns=['id', 'FloodProbability'])  # Exclude 'FloodProbability'

In [17]:
# Make predictions on the test data
test_preds = rf_model.predict(test_features)

In [9]:
flood_dataset_df = add_more_features(flood_dataset_df)

In [10]:
flood_dataset_df["FloodProbability"]=y

In [19]:
def add_manipulated_features(df):    
    df['all_row_MonsoonIntensity'] = df['MonsoonIntensity'] / df['MonsoonIntensity'].mean()   
    df['all_row_TopographyDrainage'] = df['TopographyDrainage'] / df['TopographyDrainage'].mean() 
    df['all_row_RiverManagement'] = df['RiverManagement'] / df['RiverManagement'].mean() 
    df['all_row_Deforestation'] = df['Deforestation'] / df['Deforestation'].mean() 
    df['all_row_Urbanization'] = df['Urbanization'] / df['Urbanization'].mean() 
    df['all_row_ClimateChange'] = df['ClimateChange'] / df['ClimateChange'].mean() 
    df['all_row_DamsQuality'] = df['DamsQuality'] / df['DamsQuality'].mean() 
    df['all_row_Siltation'] = df['Siltation'] / df['Siltation'].mean() 
    df['all_row_AgriculturalPractices'] = df['AgriculturalPractices'] / df['AgriculturalPractices'].mean() 
    df['all_row_Encroachments'] = df['Encroachments'] / df['Encroachments'].mean() 
    df['all_row_IneffectiveDisasterPreparedness'] = df['IneffectiveDisasterPreparedness'] / df['IneffectiveDisasterPreparedness'].mean()
    df['all_row_DrainageSystems'] = df['DrainageSystems'] / df['DrainageSystems'].mean() 
    df['all_row_CoastalVulnerability'] = df['CoastalVulnerability'] / df['CoastalVulnerability'].mean() 
    df['all_row_Landslides'] = df['Landslides'] / df['Landslides'].mean() 
    df['all_row_Watersheds'] = df['Watersheds'] / df['Watersheds'].mean() 
    df['all_row_DeterioratingInfrastructure'] = df['DeterioratingInfrastructure'] / df['DeterioratingInfrastructure'].mean() 
    df['all_row_PopulationScore'] = df['PopulationScore'] / df['PopulationScore'].mean() 
    df['all_row_WetlandLoss'] = df['WetlandLoss'] / df['WetlandLoss'].mean()
    df['all_row_InadequatePlanning'] = df['InadequatePlanning'] / df['InadequatePlanning'].mean() 
    df['all_row_PoliticalFactors'] = df['PoliticalFactors'] / df['PoliticalFactors'].mean() 
    df['all_row_weather'] = df['weather'] / df['weather'].mean() 
    df['all_row_structure'] = df['structure'] / df['structure'].mean()
    df['all_row_population'] = df['population'] / df['population'].mean() 
    df['all_row_soil_infrastructure'] = df['soil_infrastructure'] / df['soil_infrastructure'].mean() 
    df['all_row_preparedness'] = df['preparedness'] / df['preparedness'].mean() 
    
    df['ClimateImpact'] = df['MonsoonIntensity'] + df['ClimateChange']
    df['AnthropogenicPressure'] = df['Deforestation'] + df['Urbanization'] + df['AgriculturalPractices'] + df['Encroachments']
    df['InfrastructureQuality'] = df['DamsQuality'] + df['DrainageSystems'] + df['DeterioratingInfrastructure']
    df['CoastalVulnerabilityTotal'] = df['CoastalVulnerability'] + df['Landslides']
    df['PreventiveMeasuresEfficiency'] = df['RiverManagement'] + df['IneffectiveDisasterPreparedness'] + df['InadequatePlanning']
    df['EcosystemImpact'] = df['WetlandLoss'] + df['Watersheds']
    df['SocioPoliticalContext'] = df['PopulationScore'] * df['PoliticalFactors']


    df['FloodVulnerabilityIndex'] = (df['AnthropogenicPressure'] + df['InfrastructureQuality'] +
                                     df['CoastalVulnerabilityTotal'] + df['PreventiveMeasuresEfficiency']) / 4
    
    df['PopulationDensityImpact'] = df['PopulationScore'] * (df['Urbanization'] + df['Encroachments'])
    
    df['DeforestationUrbanizationRatio'] = df['Deforestation'] / df['Urbanization']
    
    df['AgriculturalEncroachmentImpact'] = df['AgriculturalPractices'] * df['Encroachments']
    
    df['DamDrainageInteraction'] = df['DamsQuality'] * df['DrainageSystems']
    
    df['LandslideSiltationInteraction'] = df['Landslides'] * df['Siltation']
    
    df['WatershedWetlandRatio'] = df['Watersheds'] / df['WetlandLoss']
    
    df['PoliticalPreparednessInteraction'] = df['PoliticalFactors'] * df['IneffectiveDisasterPreparedness']
    
    
    df['TopographyDrainageSiltation'] = df['TopographyDrainage'] + df['Siltation']
    
    df['ClimateAnthropogenicInteraction'] = df['ClimateImpact'] * df['AnthropogenicPressure']
    
    df['InfrastructurePreventionInteraction'] = df['InfrastructureQuality'] * df['PreventiveMeasuresEfficiency']
    
    df['CoastalEcosystemInteraction'] = df['CoastalVulnerabilityTotal'] * df['EcosystemImpact']

    
    return df

In [20]:
flood_dataset_df = add_manipulated_features(flood_dataset_df)

In [21]:
flood_dataset_df.head(5)

In [22]:
flood_dataset_df.to_csv('train_preprocessing.csv', index=False)
del flood_dataset_df

In [24]:
del flood_dataset_df

In [25]:
flood_dataset_df = pd.read_csv("/kaggle/working/train_preprocessing.csv")

In [26]:
flood_dataset_df.shape

In [29]:
dataX = flood_dataset_df.drop(['FloodProbability'], axis = 1)
datay = flood_dataset_df['FloodProbability']

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(dataX, datay, test_size = 0.3, random_state = 42)

In [4]:
best_model.summary()



In [5]:
# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters:")
for key, value in best_hps.values.items():
    print(f"{key}: {value}")

In [6]:
def build_fixed_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)))
    model.add(tf.keras.layers.Dense(242, activation='relu'))
    model.add(tf.keras.layers.Dense(12, activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='mean_squared_error',
        metrics=[r2_score_metric]
    )
    return model

# Build and compile the model
fixed_model = build_fixed_model()

# Print the summary of the model
fixed_model.summary()

# Train the model
fixed_model.fit(X_train_scaled, y_train, epochs=5, validation_split=0.2)



In [7]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
# Evaluate the model on the test set
y_pred = fixed_model.predict(X_test_scaled)

# Calculate and print detailed metrics
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("R² Score of the fixed model:", r2)
print("Mean Squared Error of the fixed model:", mse)
print("Mean Absolute Error of the fixed model:", mae)

In [8]:

test_df = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv')

test1_df = test_df.drop('id',axis=1)

# 3. Make Predictions
X_test = scaler.transform(test1_df)  # Preprocess the test data
predictions = fixed_model.predict(X_test)  # Make predictions

# 4. Prepare Submission File
submission_df = pd.DataFrame({
    'id': test_df['id'],  # Assuming 'id' is the identifier column
    'FloodProbability': predictions.flatten()  # Assuming 'FloodProbability' is the target column
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission.csv', index=False)



In [24]:
def cross_validate_tuner(model, X, y, XGB=False):
    
    r2_scores = []
    for trn_idx,val_idx in kf.split(X, y.astype(int)):
        x_trn, y_trn = X.iloc[trn_idx], y.iloc[trn_idx]
        x_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
        
        if XGB: x_trn, x_val = cp.array(x_trn), cp.array(x_val)
        
        model.fit(x_trn, y_trn)
        y_prd = model.predict(x_val)
        fold_r2 = skm.r2_score(y_val, y_prd)
        r2_scores.append(fold_r2)
        
    return np.mean(r2_scores)

In [25]:
fixed_params_xgr = {
    'device'      : 'cuda',
    'n_estimators': 500,
    'tree_method' : 'hist',
    'objective'   : 'reg:squarederror',
    'n_jobs'      : -1
}

In [26]:
def objective(trial):
    params = {
        'learning_rate':     trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
        'max_depth':         trial.suggest_int('max_depth', 1, 12),
        'colsample_bytree':  trial.suggest_float('colsample_bytree', 0.05, 1),
        'subsample':         trial.suggest_float('subsample', 0.05, 1),
        'min_child_weight':  trial.suggest_float('min_child_weight', 1, 20)
    }
    
    gc.collect()
    xgr = XGBRegressor(**params, **fixed_params_xgr)
    r2_score = cross_validate_tuner(xgr, X, y, XGB=True)
    
    return r2_score

TPESampler = optuna.samplers.TPESampler(multivariate=True, group=True)
optimize_r2_xgr = optuna.create_study(direction='maximize', sampler=TPESampler, study_name='Optimizing R2 for XGBRegressor')

In [27]:
optimize_r2_xgr.optimize(objective, timeout=3*60*60, show_progress_bar=True)
dump(optimize_r2_xgr,'Optimizing R2 for XGBRegressor.pkl')

In [28]:
fixed_params_ctr = {
    'verbose'       : False,
    'task_type'     : 'GPU',
    'thread_count'  : -1,
    'iterations'    : 500,
    'max_bin'       : 2540,
    'bootstrap_type': 'Bernoulli'
}

In [29]:
def objective(trial):
    params = {
        'learning_rate':     trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
        'depth':             trial.suggest_int('depth', 1, 10),
        'subsample':         trial.suggest_float('subsample', 0.05, 1),
        'min_data_in_leaf':  trial.suggest_float('min_data_in_leaf', 1, 100)
    }
    
    gc.collect()
    cbr = CatBoostRegressor(**params, **fixed_params_ctr)
    r2_score = cross_validate_tuner(cbr, X, y)
    
    return r2_score

TPESampler = optuna.samplers.TPESampler(multivariate=True, group=True)
optimize_r2_cbr = optuna.create_study(direction='maximize', sampler=TPESampler, study_name='Optimizing R2 for CatBoostRegressor')

In [30]:
optimize_r2_cbr.optimize(objective, timeout=3*60*60, show_progress_bar=True)
dump(optimize_r2_cbr,'Optimizing R2 for CatBoostRegressor.pkl')

In [31]:
fixed_params_lbr = {
    'verbosity'   : -1,
    'device'      : 'gpu',
    'n_jobs'      : -1,
    'n_estimators': 500,
    'objective'   : 'regression',
    'metric'      : 'rmse',
    'bagging_freq': 1
}

In [32]:
def objective(trial):
    params = {
        'learning_rate':    trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
        'num_leaves':       trial.suggest_int('num_leaves', 2, 2**8),
        'subsample':        trial.suggest_float('subsample', 0.05, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.05, 1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
    }
    
    gc.collect()
    lbr = LGBMRegressor(**params, **fixed_params_lbr)
    r2_score = cross_validate_tuner(lbr, X, y)
    
    return r2_score

TPESampler = optuna.samplers.TPESampler(multivariate=True, group=True)
optimize_r2_lbr = optuna.create_study(direction='maximize', sampler=TPESampler, study_name='Optimizing R2 for LGBMRegressor')

In [33]:
optimize_r2_lbr.optimize(objective, timeout=3*60*60, show_progress_bar=True)
dump(optimize_r2_lbr,'Optimizing R2 for LGBMRegressor.pkl')

In [36]:
optimize_r2_xgrf.optimize(objective, timeout=3*60*60, show_progress_bar=True)
dump(optimize_r2_xgrf,'Optimizing R2 for XGRF.pkl')

In [37]:
def objective(trial):
    
    params = {
        'alpha':    trial.suggest_float('alpha', 1e-6, 1e2, log=True)
    }
    
    ridge = Ridge(**params)
    r2_score = cross_validate_tuner(ridge, X, y)
    
    return r2_score

TPESampler = optuna.samplers.TPESampler()
optimize_r2_ridge = optuna.create_study(direction='maximize', sampler=TPESampler, study_name='Optimizing R2 for Ridge')

In [38]:
optimize_r2_ridge.optimize(objective, n_trials=50, show_progress_bar=True)
dump(optimize_r2_ridge,'Optimizing R2 for Ridge.pkl')

In [39]:
kf = StratifiedKFold(n_splits=7, shuffle=True, random_state=77)
FOLDS = 7

def cross_validate_predict(model, X, y, test_data, repeat=2):
    '''
    Function to make the train preds and test preds using cross validation.
    The resultant ensemble of preds will be further used to make the final test prediction.
    '''
    start_time = time()
    scores = []
    oof_preds_full = np.full_like(y, np.nan, dtype=float)
    test_pred_final = np.zeros(len(test_data), dtype=float)
    
    for fold,(train_idx, val_idx) in enumerate(kf.split(X, y.astype(str))):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        x_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
        
        y_val_pred = np.zeros_like(y_val, dtype=float)
        y_pred = np.zeros(len(test_data), dtype=float)
        
        for i in range(repeat):
            m = clone(model)
            if repeat > 1:
                mm = m
                if isinstance(mm, Pipeline):
                    mm = mm[-1]
                mm.set_params(random_state=i)
            m.fit(X_train, y_train)
            y_val_pred += m.predict(x_val)
            y_pred += m.predict(test_data)
        y_val_pred /= repeat
        y_pred /= repeat
        
        score = skm.r2_score(y_val, y_val_pred)
        print(f'# Fold {fold}: R2 Score = {score:0.5f}')
        scores.append(score)
        oof_preds_full[val_idx] = y_val_pred
        test_pred_final += y_pred
        
    test_pred_final /= FOLDS
    
    total_time = time() - start_time
    if isinstance(m, Pipeline):
        model_name = m[-1].__class__.__name__
    else:
        model_name = m.__class__.__name__
        
    print(f'{BlueBold}# Mean R2 Score = {np.mean(scores):0.5f} {RedBold}± {np.std(scores):0.5f} {BlueBold}for {model_name}\n'
          f'Time Elapsed: {np.round((total_time / 60),0)} Min {end}\n')

    return oof_preds_full, test_pred_final

In [40]:
# Getting the best parameters obtained from Hyperparameter Tuning
xgr_params = optimize_r2_xgr.best_trial.params
cbr_params = optimize_r2_cbr.best_trial.params
lbr_params = optimize_r2_lbr.best_trial.params
xgrf_params = optimize_r2_xgrf.best_trial.params
ridge_params = optimize_r2_ridge.best_trial.params

In [41]:
xgr_tr_pred1, xgr_ts_pred1 = cross_validate_predict(XGBRegressor(**fixed_params_xgr, **xgr_params),
                                                    X, y, test_df)

cbr_tr_pred2, cbr_ts_pred2 = cross_validate_predict(CatBoostRegressor(**fixed_params_ctr, **cbr_params),
                                                    X, y, test_df)

lgr_tr_pred3, lgr_ts_pred3 = cross_validate_predict(LGBMRegressor(**fixed_params_lbr, **lbr_params),
                                                    X, y, test_df)

xgrf_tr_pred4, xgrf_ts_pred4 = cross_validate_predict(XGBRegressor(**fixed_params_xgrf, **xgrf_params),
                                                      X, y, test_df)

ridge_tr_pred5, ridge_ts_pred5 = cross_validate_predict(Ridge(**ridge_params), X, y, test_df)

In [44]:
ensemble_test[:5,:]

In [45]:
def objective(trial):
    
    weights = [trial.suggest_float(f'w{i}', 0, 1) for i in range(1,6)]   # weight suggestions
    weights = np.array(weights) / np.sum(weights)                        # normalizing
    
    ensemble_pred = ensemble_train.dot(weights)
    r2_score = skm.r2_score(y, ensemble_pred)
    return r2_score

TPESampler = optuna.samplers.TPESampler(multivariate=True, group=True)
optimize_weights = optuna.create_study(direction='maximize', sampler=TPESampler, study_name='Ensemble Weights')

In [46]:
optimize_weights.optimize(objective, n_trials=1000, show_progress_bar=True)
dump(optimize_weights,'Ensemble Weights.pkl')

In [47]:
weights = optimize_weights.best_trial.params
weights

In [48]:
# Normalizing the weights
weights = np.array(list(weights.values()))
weights /= np.sum(weights)
weights

In [49]:
submission = pd.DataFrame()
submission['id'] = test['id']
submission['FloodProbability'] = ensemble_test.dot(weights)
submission.to_csv('submission.csv',index=False)

In [50]:
submission

In [26]:
from keras import backend as K

def r_squared(y_true, y_pred):
    ss_res = K.sum(K.square(y_true - y_pred))
    ss_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return 1 - ss_res / (ss_tot + K.epsilon())

def build_model(hp):
    model = Sequential()
    model.add(Dense(8))

    for i in range(hp.Int("num_hidden_layers", min_value=8, max_value=80)):
        number_of_nodes = hp.Int("number_of_nodes_" + str(i), min_value=8, max_value=1024, step=16)
        activation_function = hp.Choice("activation_" + str(i), values=["relu", "tanh", "sigmoid"])
        model.add(Dense(number_of_nodes, activation=activation_function))

    model.add(Dense(1, activation="sigmoid"))

    optimizer = hp.Choice("optimizer", values=["rmsprop", "adam", "nadam", "adadelta", "sgd"])
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=[r_squared])

    return model

In [27]:
tuner = kt.RandomSearch(
    build_model,
    objective=kt.Objective("val_r_squared", direction="max"),
    max_trials=50,
    directory="mydir",
    project_name="testings"
)

In [28]:
tuner.search_space_summary()

In [31]:
best_model = tuner.get_best_models(num_models = 1)[0]

In [33]:
best_model.fit(x_train , y_train , epochs = 150 , batch_size = 2056 , initial_epoch = 6 , validation_data = (x_test , y_test))

In [34]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [54]:
model.compile(optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.01),
              loss="binary_crossentropy",
              metrics=[r_squared])

In [5]:
study = optuna.create_study(direction='maximize') #创建study对像，寻找r2的最大值
study.optimize(objective, n_trials=100)  #调优开始，进行100次试验，找出最优参数

In [6]:
best_params = study.best_params
print("Best Hyperparameters:", best_params)

In [7]:
# cat_params = {
#     'n_estimators': 2000,                  # Number of boosting iterations
#     'random_state': 0,                     
#     'learning_rate': 0.011277016304363601, # 学习率
#     'depth': 8,                            # 树深度
#     'subsample': 0.8675506657380021,       # Subsample ratio of the training instances
#     # 'colsample_bylevel': 0.7183884158632279, # Subsample ratio of columns for each level (commented out)
#     'min_data_in_leaf': 98,                # 最小叶子树
#     'task_type': 'GPU',                    # 使用GPU
#     'bootstrap_type': 'Bernoulli'          # 随机抽样类型
# }


In [21]:
# Define the models
models = {
    'bayesian_ridge': BayesianRidge(),
    'elastic_net': ElasticNetCV(cv=5),
    'linear_reg': LinearRegression(),
    'lasso': LassoCV(cv=5),
    'ridge': RidgeCV(cv=5)
}

# Define the parameter grids for each model
param_grids = {
    'bayesian_ridge': {
        'alpha_1': [1e-6, 1e-5],
        'alpha_2': [1e-6, 1e-4],
        'lambda_1': [1e-6, 1e-5],
        'lambda_2': [1e-6,1e-5]
    },
    'elastic_net': {
    },
    'linear_reg': {
        'fit_intercept': [True, False]
    },
    'lasso': {
    },
    'ridge': {
        'alphas': [(0.1, 1.0, 10.0)]
    }
}


# Initialize the GridSearchCV for each model
grid_searches = {name: GridSearchCV(model, param_grid, cv=5, n_jobs=-1, verbose=1)
                 for name, (model, param_grid) in zip(models.keys(), zip(models.values(), param_grids.values()))}


# Assume you have a dataset X (features) and y (target)
best_estimators = {}

# Fit the GridSearchCV for each model
for name, gs in grid_searches.items():
    print(f"Running GridSearchCV for {name}")
    gs.fit(X, y)
    print(f"Best parameters for {name}: {gs.best_params_}")
    print(f"Best score for {name}: {gs.best_score_}")
    best_estimators[name] = gs.best_estimator_


In [17]:
param=[{'alpha':[0.05,0.1,0.5,0.8]},{'solver':['auto','saga']}]
gv=GridSearchCV(ri,param_grid=param,cv=5,return_train_score=True)
gv.fit(X_train_t,y_train)


In [57]:
estimators = [("catBoost",cat_model),("linearReg",linear),("xgBoost",xg_model)]

In [62]:
# result_df

In [65]:
# xg_model = XGBRegressor()
# xg_model.fit(X, y)

# y_pred_xg = xg_model.predict(test_set)
# print(f"Training score {xg_model.score(X,y)}")
# # Training score 0.8690384222081213

In [74]:
y_pred_LG = pd.DataFrame(y_pred_LG)
output_LG = test_set_id
output_LG["FloodProbability"] = pd.DataFrame(y_pred_LG)
output_LG = output_LG.set_index("id")
output_LG.to_csv("FloodProbability_SS_project_FEng_LGBM.csv")

In [19]:
model3 = Sequential([
    Dense(16, activation='relu', input_dim=X_train.shape[1]),    
    Dense(8, activation='relu'),
    Dense(1, activation='linear'),

])

model3.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['r2_score'])
model3.summary()

In [22]:
# Define needed variables
tr_loss = history3.history['loss']
val_loss = history3.history['val_loss']
index_loss = np.argmin(val_loss)
val_lowest = val_loss[index_loss]

Epochs = [i+1 for i in range(len(tr_loss))]
loss_label = f'best epoch= {str(index_loss + 1)}'

# Plot training history
plt.figure(figsize= (20, 8))
plt.style.use('fivethirtyeight')

plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
plt.scatter(index_loss + 1, val_lowest, s= 150, c= 'blue', label= loss_label)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout
plt.show()


In [23]:
max(history3.history['val_r2_score'])

In [25]:
call_back4 = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True , verbose=0)

In [26]:
history4 = model4.fit(X_train, y_train, epochs=100, batch_size=32, callbacks= call_back4 , validation_data=(X_val, y_val), verbose=1)

In [27]:
# Define needed variables
tr_loss = history4.history['loss']
val_loss = history4.history['val_loss']
index_loss = np.argmin(val_loss)
val_lowest = val_loss[index_loss]

Epochs = [i+1 for i in range(len(tr_loss))]
loss_label = f'best epoch= {str(index_loss + 1)}'

# Plot training history
plt.figure(figsize= (20, 8))
plt.style.use('fivethirtyeight')

plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
plt.scatter(index_loss + 1, val_lowest, s= 150, c= 'blue', label= loss_label)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout
plt.show()


In [33]:
from sklearn.model_selection import GridSearchCV

# define a function that fits and the train the model on the training data for all parameters while using 
# cross-validation
def hyperparameter_tuning(model, params, X_train):
    clf = GridSearchCV(estimator = model, 
                       param_grid = params, 
                       scoring = 'neg_mean_squared_error', 
                       n_jobs = -1,
                       return_train_score = True,
                       refit = True,
                       cv = 5)
    clf.fit(X_train, y_train)
    
    print('Best estimator: ', clf.best_estimator_)
    print('Best score: ', clf.best_score_)

In [34]:
ridge = Ridge()
params_ridge = {'alpha': [0.1, 0, 1, 1.5]}
hyperparameter_tuning(ridge, params_ridge, X_train_scaled)

In [38]:
lin_reg = LinearRegression()
ridge = Ridge(alpha = 0)
kn_reg = KNeighborsRegressor(n_neighbors = 9)
tree = DecisionTreeRegressor(min_samples_leaf = 2, min_samples_split = 5)
xgb = XGBRegressor(learning_rate = 0.1, max_depth = 7,
                   min_child_weight = 3, n_estimators = 500)

In [39]:
models_need_scaling = [lin_reg, ridge, kn_reg]
models_names_need_scaling = ['Linear Regression', 'Ridge', 'k-Neighbors Regression']

train_score, test_score, r2, mae, rmse = [], [], [], [], []

for model, model_name in zip(models_need_scaling, models_names_need_scaling):
    building_model(model, model_name, X_train_scaled, X_val_scaled)

scores_df3 = pd.DataFrame({'model': models_names_need_scaling,
                          'train_score': train_score,
                          'test_score': test_score,
                          'r2_score': r2,
                          'mean_absolute_error': mae,
                          'root_mean_squared_error': rmse}).sort_values(by = 'r2_score',
                                                                        ascending = False).set_index('model')
scores_df3

In [46]:
median = results['predicted_proba'].median()
mean = results['predicted_proba'].mean()

y_pred.plot(kind = 'hist', color = '#2CBD8F')
plt.axvline(x = median, linestyle = '-', color = '#B136EA', label = 'median')
plt.axvline(x = mean, linestyle = '--', color = '#EA3690', label = 'mean')

plt.xlabel('predicted proba')
plt.title('Distribution of the Flood Predicted Probability')
plt.legend()
plt.show()

In [24]:
params = {
    'n_estimators' : [300,400,500,600,800,900,1100,1000],
    'eta' : [0.1,0.01,1,0.5, 0.05,5],
    'alpha' : [1,5,10,15,20],
    'reg_lambda': [1,5,10,15,20],
    'max_depth' : [1,3,5,7,9,12],
    'colsample_bytree' : [0.3,0.5,0.7,1],
    'subsample' : [0.3,0.5,0.7,1],
    'min_child_weight': [1,3,5,7,9,10]
}

In [25]:
xg_cv = RandomizedSearchCV(estimator = XGBRegressor(), param_distributions = params, cv = 3, scoring = 'r2',verbose = 1,n_iter = 20)

In [26]:
# cv.fit(X, y)

In [28]:
# cv.best_params_

In [29]:
XGBooster = XGBRegressor(subsample = 1, eta =  0.5, alpha = 10, max_depth = 1, n_estimators = 400, reg_lambda = 10, min_child_weight = 3,colsample_bytree = 1)

In [31]:
from catboost import CatBoostRegressor

In [37]:
catBooster = CatBoostRegressor(subsample = 1, max_depth = 10, learning_rate = 0.1, l2_leaf_reg = 5, iterations = 800, bootstrap_type = 'Bernoulli')

In [38]:
catBooster.fit(X,y)

In [43]:
# light_cv.fit(X,np.ravel(y))

In [48]:
from sklearn.ensemble import GradientBoostingRegressor

In [51]:
# gb_cv.fit(X,np.ravel(y))

In [52]:
gb = GradientBoostingRegressor(learning_rate=0.5, max_depth=3, max_features='log2', min_samples_leaf=6, min_samples_split=8, n_estimators=700, subsample=1)

In [53]:
gb.fit(X,np.ravel(y))

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import keras_tuner as kt

import tensorflow as tf
from tensorflow.keras import backend as K

def r2_score_metric(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return 1 - SS_res/(SS_tot + K.epsilon())


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(cleaned_train_df, cleaned_target_df, test_size=0.3, random_state=11)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)))

    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(tf.keras.layers.Dense(
            units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),
            activation='relu',
            kernel_regularizer=tf.keras.regularizers.l1_l2(
                l1=hp.Float('l1_' + str(i), 1e-5, 1e-2, sampling='LOG'),
                l2=hp.Float('l2_' + str(i), 1e-5, 1e-2, sampling='LOG')
            )
        ))
        model.add(tf.keras.layers.Dropout(hp.Float('dropout_' + str(i), 0.0, 0.5, step=0.1)))

    model.add(tf.keras.layers.Dense(1))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Float('learning_rate', 1e-4, 1e-2, sampling='LOG')
        ),
        loss='mean_squared_error',
        metrics=[r2_score_metric]
    )
    return model

# Create a tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Search for the best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=11, validation_split=0.2)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test_scaled)
r2 = r2_score(y_test, y_pred)
print("R2 Score of the best model:", r2)

In [7]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters:")
for key, value in best_hps.values.items():
    print(f"{key}: {value}")
'''
    Best hyperparameters:
num_layers: 1
units_0: 256
l1_0: 2.3699222722321995e-05
l2_0: 0.0005694007711882117
dropout_0: 0.1
learning_rate: 0.0016212150201753844
units_1: 288
l1_1: 1.3965518320658789e-05
l2_1: 3.8408860301375645e-05
dropout_1: 0.1
units_2: 416
l1_2: 6.66931421001826e-05
l2_2: 0.0005722594832996224
dropout_2: 0.30000000000000004'''


In [8]:
def build_fixed_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)))
    model.add(tf.keras.layers.Dense(242, activation='relu'))
    model.add(tf.keras.layers.Dense(12, activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='mean_squared_error',
        metrics=[r2_score_metric]
    )
    return model

# Build and compile the model
fixed_model = build_fixed_model()

# Print the summary of the model
fixed_model.summary()

# Train the model
fixed_model.fit(X_train_scaled, y_train, epochs=5, validation_split=0.2)


In [9]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
# Evaluate the model on the test set
y_pred = fixed_model.predict(X_test_scaled)

# Calculate and print detailed metrics
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("R² Score of the fixed model:", r2)
print("Mean Squared Error of the fixed model:", mse)
print("Mean Absolute Error of the fixed model:", mae)

In [11]:
est1_df = test_df.drop('id',axis=1)


X_test = scaler.transform(test1_df)  # Preprocess the test data
predictions = fixed_model.predict(X_test)  # Make predictions

# 4. Prepare Submission File
submission_df = pd.DataFrame({
    'id': test_df['id'],  # Assuming 'id' is the identifier column
    'FloodProbability': predictions.flatten()  # Assuming 'FloodProbability' is the target column
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('FE_NN.csv', index=False)

In [2]:
def cvr_sequences(model,x_train,target,folds=10,n_rounds=10,n_jobs=1,scorer='',output=False,graphs=False):
    '''
    Function to get the sequence of features up to the first random feature or the max CVS in the sequence.
    Is a wrapper for "rank_feats" and "get_rank" functions.
    Output:
        a dictionary with the sequence of features as key and its CVS as value. 
        The sequences are those explored in "rank_feats" plus the mean rank sequence computed with "get_ranks".
    '''
    if not scorer:
        print('scorer must be a valid sklearn scorer')
    ranking=rank_feats(model,x_train,target,n_jobs=n_jobs,n_rounds=n_rounds,scorer=scorer,output=output)
    scores=get_ranks(ranking,x_train,target,n_jobs=n_jobs,scorer=scorer,graphs=graphs)
    sqd={}
    for s,numf,*_ in ranking:
        seq=[]
        # At most up to the sequence with the highest CVS
        for f in s[:numf]:
            if  f.startswith('random_'):
                break
            seq.append(f)
        sqd[tuple(sorted(seq))]=cross_val_score(model,x_train[seq],target,scoring=scorer,cv=folds).mean()
    # Computes the mean rank sequence
    seq=[]
    for f in scores.keys():
        if f.startswith('random_'):
            break
        seq.append(f)
    
    sqd[tuple(sorted(seq))]=cross_val_score(model,x_train[seq],target,scoring=scorer,cv=folds).mean()
    
    return dict(sorted(sqd.items(),key=lambda x:x[1]))
    
    


In [3]:
def perf_estimation(model,X_train,y_train,score=None ,scorer='r2',folds=10,output=True,graphs=True,n_jobs=-1):
    '''
    Computes the sequence of CVS corresponding to the sequence of features defined in X_train. That is, given a sequence of features {f0, f1, ..., fn}
    computes the sequence [CVS(f0),CVS(f0, f1),....,CVS(f0, f1, ..., fn)]
    Input
        * model: A model (classificator or regressor) sklearn cross_val_score compatible.
        * X_train: Pandas dataframe with features as columns.
        * y_train: Pandas series with the target
        * score: The order in which features are added. It could be:
            +None: default. The X_train columns order
            +Any string: random order
            +list: a ordered list with the name of features (X_train columns)
        * scorer: Any sklearn cross_val_score valid scorer
        * folds: Integer. Number of folds for cross validation
        * output, graphs: Boolean. Defect True. If False no graphs or no print output
        * n_jobs: Integer. Default -1. sklearn n_jobs for cross_val_score

    Output:
        * A list of tuples, one for each sequence [{f0}, {f0, f1}, ..., {f0, f1, ..., fn}]: 
            [(#features, time spent in computation, list of features used, list with score of each fold), ....]
        * If 'output' is True prints #features in the sequence, last feature added to the sequence, mean +- std cross_val_score, computation time
            for each iteration
        * If 'graphs' is True displays 3 plots: The box-plot of the folds scores for each iteration and the mean and median line plot. 
            
    '''
    from time import time
    result=[]
    time00=time()
    if type(score)==type(None):
        score=pd.Series(list(range(X_train.shape[1])),index=X_train.columns)
    elif isinstance(score,str):
        from random import shuffle
        a=X_train.columns.to_list()
        shuffle(a)
        score=pd.Series(list(range(X_train.shape[1])),index=a)
    elif isinstance(score,list):
        score=pd.Series(list(range(len(score))),index=score)
    
                        
    sequence=[]
    for nf in score.index.to_list():
        
        sequence.append(nf)
        if output:
             print(f'num. feats.: {len(sequence):3.0f} ({nf:>35})',end='    ')
        
        
        time0=time()
        cv_score=cross_val_score(model,X_train[sequence],y_train,scoring=scorer,n_jobs=n_jobs,cv=folds)
        result.append((len(sequence),time()-time0,nf,cv_score)),
        if output:
             print(f'\t cv score: {result[-1][-1].mean():0.4f} ± {result[-1][-1].std():0.4f}\t time: {result[-1][1]:0.2f}s')
    
    if output:
             print(f'Total time: {time()-time00}')
    if graphs:
        nf,tm,feats,cvtrain=transpose(result)


        values=[[numf,val] for numf,values in zip(nf,cvtrain) for val in values]
        values=pd.DataFrame(values,columns=['nf','score'])
        values.boxplot('score',by='nf',figsize=(15,6))
        plt.show()
        
        plt.figure(figsize=(18,6))
        plt.subplot(1,2,1)
        grouped=values.groupby('nf').mean()
        top=grouped.max().values[0]
        bottom=(grouped.min().values[0]+grouped.max().values[0])/2
        delta=(top-bottom)/50
        plt.plot(grouped.index,grouped.score)
        plt.hlines(top,1,X_train.shape[1],color='red',linestyles='dashed')
        lg=f'Max: {top:0.4f}'
        plt.text(0,top+delta,lg)
        plt.vlines(np.argmax(grouped)+1,bottom,top,color='red',linestyles='dashed')
        lg=f'#feats: {np.argmax(grouped)+1}'
        plt.text(np.argmax(grouped)+1,bottom+delta,lg)
        plt.grid()
        plt.ylim(bottom=bottom)
        plt.title('cv Mean')

        plt.subplot(1,2,2)
        grouped=values.groupby('nf').median()
        top=grouped.max().values[0]
        bottom=(grouped.min().values[0]+grouped.max().values[0])/2
        delta=(top-bottom)/50
        plt.plot(grouped.index,grouped.score)
        plt.hlines(top,1,X_train.shape[1],color='red',linestyles='dashed')
        lg=f'Max: {top:0.4f}'
        plt.text(0,top+delta,lg)
        plt.vlines(np.argmax(grouped)+1,bottom,top,color='red',linestyles='dashed')
        lg=f'#feats: {np.argmax(grouped)+1}'
        plt.text(np.argmax(grouped)+1,bottom+delta,lg)
        plt.grid()
        plt.ylim(bottom=bottom)
        plt.title('cv Median')
        plt.show()
        plt.close()

    
    return result

In [4]:
def rank_feats(model,dt,y,scorer='r2',folds=10,n_rounds=5,n_jobs=-1,output=True,initial=[]):
    '''
    Computes the increment in cross_val_score due i-th feature from a permutation of features. 
    Then sorts the features sequence by this increment and repeat the process.
    Input:
        * model: A model (classificator or regressor) sklearn "cross_val_score" compatible.
        * dt: : Pandas dataframe with features as columns.
        * y: Pandas series with the target.
        * scorer: Any sklearn "cross_val_score" valid scorer.
        * folds: Integer. Number of folds for cross validation.
        * n_jobs: Integer. Default -1. sklearn n_jobs for "cross_val_score"
        * n_rounds: Integer. Default 5. How many of times the algorithm is repeated.
    Output:
        A list with a tuple for each repetition: 
        [(sequence of features,  #features max CV mean, max CV mean,  #features max CV median, max CV median), ...]


    '''
    from time import time
    ranking=[]
    time0=time()
    if output:
            print('Initial round', end=' .... ')
    for col in dt:
        ranking.append([col,cross_val_score(model,dt[[col]],y,scoring=scorer,cv=folds,n_jobs=n_jobs).mean()])
        rankf,cv=transpose(sorted(ranking,key=lambda x: x[1],reverse=True))
    if output:
            print(f'Done! ({round(time()-time0,1)} s)')
    
    rounds=[]
    for i in range(n_rounds):
        if output:
            print(f'Round {i+1}',end=': ')
        
        result=perf_estimation(model,dt,y,score=list(rankf),scorer=scorer,output=False,graphs=False,n_jobs=n_jobs,folds=folds)
        max_mean=np.argmax([item[-1].mean() for item in result])
        max_median=np.argmax([np.median(item[-1]) for item in result])
        
        rounds.append((rankf,result[max_mean][0],result[max_mean][-1].mean(),result[max_median][0],np.median(result[max_median][-1])))
        if output:
            print(f'mean: {rounds[-1][1]} ({rounds[-1][2]:0.5f})  median: {rounds[-1][3]} ({rounds[-1][4]:0.5f})')
        ranking=[[result[0][2],result[0][3].mean()]]
        prev=ranking[-1][-1]
        for nf,tm,col,cv in result[1:]:
            ranking.append([col,cv.mean()-prev])
            prev=cv.mean()
        rankf,cv=transpose(sorted(ranking,key=lambda x: x[1],reverse=True))
    return rounds

In [5]:
def get_ranks(ranking,dt=None,y=None,n_jobs=-1,scorer='',graphs=True,model=''):
    '''
    Assigns the ranks to each feature. The rank is the order number of the feature in the sequence. 
    Then computes de mean and standard deviation for the ranks of each feature. Finally sort the features by mean rank.
    Input:
        * ranking: the `rank_feats` output.
        * dt: Optional. Only active if "graphs" is True. Pandas dataframe with features as columns.
        * y: Optional. Only active if "graphs" is True. Pandas series with the target.
        * scorer: Optional. Only active if "graphs" is True. Any sklearn cross_val_score valid scorer.
        * n_jobs: Optional. Only active if "graphs" is True. Integer. Default -1. sklearn n_jobs for cross_val_score
    Output:
        * An ordered  dictionary:
            {feature name: (mean rank, std rank), ....}
        * If "graphs"=True and "dt", "y", "scorer" are defined runs `perf_estimation` for the sequence of features sorted by mean rank.
        * If graphs=True displays a line plot for the mean rank. The line is surrounded by +- std rank red area
   
    '''
    rank={key:[] for key in ranking[0][0]}
    for r,*_ in ranking[1:]:
        for i,f in enumerate(r):
            rank[f].append(i)
    rank={key:(np.mean(val),np.std(val)) for key,val in rank.items()}
    rank=dict(sorted(rank.items(),key=lambda x:x[1]))

    if graphs:
        if model:
            res=perf_estimation(model,dt,y,list(rank.keys()),scorer=scorer,n_jobs=n_jobs)
    
        ticks=list(rank.keys())
        y_val=[]
        y_err=[]
        for avg,sqtd in rank.values():
            
            y_val.append(avg)
            y_err.append(sqtd)

        if len(rank)>30:
            plt.figure(figsize=(18,6))
            
        plt.plot(ticks,y_val)
        plt.xticks(rotation=80)#yerr=y_err)
        plt.fill_between(ticks,np.array(y_val)+np.array(y_err),np.array(y_val)-np.array(y_err),alpha=0.15,color='red')
        plt.xticks(rotation=80)#yerr=y_err)
        plt.ylabel('mean rank')
        plt.grid()
        plt.show()
        plt.close


    return rank

In [6]:
def MI(x,y,func=None,random_seed=0,discrete_features='auto'):
    '''
    Wrapper for sklearn "mutual_info_classif/regression". These are statistical estimators that have some subtleties.
    Input:
        x: Pandas dataframe or numpy array with features as columns
        y: Pandas dataframe or numpy array with the target 
        func: A function for compute the mutual information. sklearn "mutual_info_classif/regression". 
            If it is None, the function will be automatically selected based on the number of unique values of the target.
        random_seed: Integer. A value for the "random_state" parameter 
        discrete_features: sklearn discrete_features parameter
    Output:
        A numpy array with the I(X_i,Y) values.
    '''
    if not func:
        if np.unique(y).shape[0]<=10:
            func=mutual_info_classif
            y=y.astype(int)
        else:
            func=mutual_info_regression
       
    return func(x,y,random_state=random_seed,discrete_features=discrete_features)

In [7]:
def mi_score(orig,crossmi,criterion='relative',weight=1.0): 
    '''
    Computes M_i scores: M_i=I(X_i,Y)-weight*sum(I(X_i,X_j))
    Input:
        orig: first output from "get_mutual_info"
        crossmi: MI between features. Second output from "get_mutual_info"
        criterion: String. If it is "relative", normalizes I(X,Y) as I(X,Y)/H(Y) and the cross I(X_i,X_j) as  I(X_i,X_j)/min(H(X_i),H(X_j)). 
            In other case does nothing.
        weight: Float. The weight for the penalty of the cross mutual info between features.
    Output:
        Pandas Series with features as index and the M_i score as value.
    '''
        
    mi=pd.Series(orig['mi'])
    H=pd.Series(orig['Sh'])
    mi=mi.sort_values()
    penalty=pd.Series(np.zeros_like(mi),index=mi.index)
    score=mi.copy()
    
    
    cols=score.sort_values().index.to_list()
    selected=[]
    while cols:
        #feature selected
        pivot=cols.pop(-1)

        #update penalties for remaining features
        for c in cols:
            if criterion=='relative':
                penalty[c]+=crossmi.loc[c,pivot]/min(H[c],H[pivot])
            else:
                penalty[c]+=crossmi.loc[c,pivot]

        #update scores, cols and selected
        selected.append(pivot)
        if criterion=='relative':
            score[cols]=(mi/H['target']-weight*penalty/len(selected))[cols]
        else:
            score[cols]=(mi-weight*penalty/len(selected))[cols]
        cols=score.sort_values().index.to_list()
        cols=[item for item in cols if not item in selected]
    
    
    return (score).sort_values(ascending=False)

In [10]:
penalty_weight=0.2 #weight of MIR penalty

In [17]:
%%time
seqs=cvr_sequences(model,dt_class,y_class,n_jobs=n_jobs,n_rounds=10,scorer='accuracy',output=True,graphs=True)

seqs=(sorted(seqs.items(),key=lambda x: x[1],reverse=True))
for s,c in seqs:
    print(f'{c:0.5f} {", ".join(s)}')

In [18]:
%%time
selector=RFECV(model,cv=10,scoring='accuracy',n_jobs=n_jobs)
selector.fit(dt_class,y_class)
seq=sorted(dt_class.columns[selector.support_])
cvs=cross_val_score(model,dt_class[seq],y_class,scoring='accuracy',cv=10).mean()
print(f'{cvs:0.5f}  {", ".join(seq)}')



In [19]:
%%time
selector=SequentialFeatureSelector(model, direction='forward', scoring='accuracy', cv=10, n_jobs=n_jobs,n_features_to_select='auto',tol=1e-4)
selector.fit(dt_class,y_class)
seq=sorted(dt_class.columns[selector.support_])
cvs=cross_val_score(model,dt_class[seq],y_class,scoring='accuracy',cv=10).mean()
print(f'{cvs:0.5f}  {", ".join(seq)}')


In [20]:
%%time
selector=SequentialFeatureSelector(model, direction='backward', scoring='accuracy', cv=10, n_jobs=n_jobs,n_features_to_select='auto',tol=-1e-4)
selector.fit(dt_class,y_class)
seq=sorted(dt_class.columns[selector.support_])
cvs=cross_val_score(model,dt_class[seq],y_class,scoring='accuracy',cv=10).mean()
print(f'{cvs:0.5f}  {", ".join(seq)}')

In [21]:
#sklearn make_regression stuff

num_inf=10
num_features=int(np.ceil(1.5*(num_inf)))
columns=[f'I_{item}' for item in range(num_inf)]
i=0
while len(columns)<num_features:
    columns.append(f'random_{i}')
    i+=1
dt_reg,y_reg=make_regression(n_samples=n_samples,
                         n_informative=num_inf,
                         
                         n_features=num_features,
                        
                         shuffle=False)
dt_reg=pd.DataFrame(dt_reg,columns=columns)

In [24]:
model=LGBMRegressor(verbose=-1,random_state=0)#HistGradientBoostingRegressor(random_state=0)#
n_jobs=1

In [26]:
%%time

seqs=dict(cvr_sequences(model,dt_reg,y_reg,n_jobs=n_jobs,n_rounds=10,scorer='r2',output=True,graphs=True))
seqs=(sorted(seqs.items(),key=lambda x: x[1],reverse=True))
for s,c in seqs:
    print(f'{c:0.5f} {", ".join(s)}')

In [27]:
%%time
selector=RFECV(model,cv=10,scoring='r2',n_jobs=n_jobs)
selector.fit(dt_reg,y_reg)
seq=sorted(dt_reg.columns[selector.support_])
cvs=cross_val_score(model,dt_reg[seq],y_reg,scoring='r2',cv=10).mean()
print(f'{cvs:0.5f}  {", ".join(seq)}')

In [28]:
%%time
selector=SequentialFeatureSelector(model, direction='forward', scoring='r2', cv=10, n_jobs=n_jobs,n_features_to_select='auto',tol=1e-4)
selector.fit(dt_reg,y_reg)
seq=sorted(dt_reg.columns[selector.support_])
cvs=cross_val_score(model,dt_reg[seq],y_reg,scoring='r2',cv=10).mean()
print(f'{cvs:0.5f}  {", ".join(seq)}')

In [29]:
%%time
selector=SequentialFeatureSelector(model, direction='backward', scoring='r2', cv=10, n_jobs=n_jobs,n_features_to_select='auto',tol=-1e-4)
selector.fit(dt_reg,y_reg)
seq=sorted(dt_reg.columns[selector.support_])
cvs=cross_val_score(model,dt_reg[seq],y_reg,scoring='r2',cv=10).mean()
print(f'{cvs:0.5f}  {", ".join(seq)}')

In [33]:
model=LGBMRegressor(verbose=-1,random_state=0)#HistGradientBoostingRegressor(random_state=0)#
n_jobs=1

In [35]:
%%time
model=LGBMRegressor(verbose=-1,random_state=0)#HistGradientBoostingRegressor(random_state=0)#
n_jobs=1


seqs=dict(cvr_sequences(model,dt_gen,y_gen,n_jobs=n_jobs,n_rounds=10,scorer='r2',output=True,graphs=True))

seqs=(sorted(seqs.items(),key=lambda x: x[1],reverse=True))
for s,c in seqs:
    print(f'{c:0.5f} {", ".join(s)}')

In [36]:
%%time
selector=RFECV(model,cv=10,scoring='r2',n_jobs=n_jobs)
selector.fit(dt_gen,y_gen)
seq=sorted(dt_gen.columns[selector.support_])
cvs=cross_val_score(model,dt_gen[seq],y_gen,scoring='r2',cv=10).mean()
print(f'{cvs:0.5f}  {", ".join(seq)}')

In [37]:
%%time
selector=SequentialFeatureSelector(model, direction='forward', scoring='r2', cv=10, n_jobs=n_jobs,n_features_to_select='auto',tol=1e-4)
selector.fit(dt_gen,y_gen)
cvs=cross_val_score(model,dt_gen[seq],y_gen,scoring='r2',cv=10).mean()
print(f'{cvs:0.5f}  {", ".join(seq)}')

In [38]:
%%time
selector=SequentialFeatureSelector(model, direction='backward', scoring='r2', cv=10, n_jobs=n_jobs,n_features_to_select='auto',tol=-1e-4)
selector.fit(dt_gen,y_gen)
cvs=cross_val_score(model,dt_gen[seq],y_gen,scoring='r2',cv=10).mean()
print(f'{cvs:0.5f}  {", ".join(seq)}')

In [40]:
%%time
ent,crossmi,mi_sc=get_mutual_info(dt_gen,y_gen,weight=penalty_weight)

seq_mir=[]
for f in mi_sc.index:
    if f.startswith('random_'):
        break
    seq_mir.append(f)
seq_mir=sorted(seq_mir)
cvs_mir=cross_val_score(model,dt_gen[seq_mir],y_gen,scoring='r2',cv=10).mean()
print(f'{cvs_mir:0.5f}  {", ".join(seq_mir)}')

In [41]:
%%time
model=LGBMRegressor(verbose=-1,random_state=0)#HistGradientBoostingRegressor(random_state=0)#
n_jobs=1


seqs=dict(cvr_sequences(model,dt_gen,y_gen,n_jobs=n_jobs,n_rounds=10,scorer='r2',output=True,graphs=True))

seqs=(sorted(seqs.items(),key=lambda x: x[1],reverse=True))
for s,c in seqs:
    print(f'{c:0.5f} {", ".join(s)}')

In [42]:
%%time
selector=RFECV(model,cv=10,scoring='r2',n_jobs=n_jobs)
selector.fit(dt_gen,y_gen)
seq=sorted(dt_gen.columns[selector.support_])
cvs=cross_val_score(model,dt_gen[seq],y_gen,scoring='r2',cv=10).mean()
print(f'{cvs:0.5f}  {", ".join(seq)}')

In [43]:
%%time
selector=SequentialFeatureSelector(model, direction='forward', scoring='r2', cv=10, n_jobs=n_jobs,n_features_to_select='auto',tol=1e-4)
selector.fit(dt_gen,y_gen)
seq=sorted(dt_gen.columns[selector.support_])
cvs=cross_val_score(model,dt_gen[seq],y_gen,scoring='r2',cv=10).mean()
print(f'{cvs:0.5f}  {", ".join(seq)}')

In [44]:
%%time
selector=SequentialFeatureSelector(model, direction='backward', scoring='r2', cv=10, n_jobs=n_jobs,n_features_to_select='auto',tol=-1e-4)
selector.fit(dt_gen,y_gen)
seq=sorted(dt_gen.columns[selector.support_])
cvs=cross_val_score(model,dt_gen[seq],y_gen,scoring='r2',cv=10).mean()
print(f'{cvs:0.5f}  {", ".join(seq)}')

In [ ]:
# ファイル操作
import os

# 数値計算系
import math
import numpy as np

# データ操作・表計算系
import pandas as pd

# グラフ描写系
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# 機械学習モデル
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV

# エラー処理用
import warnings
warnings.filterwarnings('ignore')
import sys
sys.stderr = open(os.devnull, "w")  # silence stderr
# sys.stderr = sys.__stderr__  # unsilence stderr

In [7]:
eda.run_full_eda(train_data, run_collinear=False, generate_plots=True)

In [8]:
def get_mutual_info(X_orig,y_orig,graphics=True,discrete_limit=10,crossmi=True,weight=1.0,criterion='relative'):
    '''
    Function for compute entropy based information about features and target.
    Input:
        X_orig: Pandas dataframe or numpy array with the features as columns.
        y_orig: Pandas Series or numpy array with the target.
        graphics: Boolean. If it is True displays available graphs
        discrete_limit: int. The number of unique values below of which a feature is considered discrete, 
            in the sense of the sklearn "discrete_features" parameter.
        crossmi: Boolean. If it is True computes the cross mutual information between features.
        weight: Float. The weight for the penalty of the cross mutual info between features. Only active if "crossmi" is True.
        criterion: String. The "mi_score" "criterion" parameter.
    Output:
        Three items:
        * First item: a dictionary with the items:
            +'mi': list of mutual info between each feature and the target.
            +'Sh': list with entropies of target and each feature.
            +'noise': list with the noise (H(X_i)-I(X_i,Y)) of each feature
        * Second item: If "crossmi" is set to True a numpy array with I(X_i,X_j), else None
        * Third item: If "crossmi" is set to True the output of "mi_score", else None.
        * If "graphs" is set to True:
            + A bar-plot for the I(X_i,Y) with the features sorted by score.
            If "crossmi" is set to True:
                +A bar-plot with the noise of each feature
                +A bar-plot with the signal/noise ratio
                +A heat map with the I(X_i,X_j)
            
    
    '''
    
    mutual_array=[]
    ccols=classify_columns(X_orig,cat_limit=discrete_limit)
    cat_cols=ccols['cat_cols']
    d_cols=[i for i,c in enumerate(X_orig.columns) if c in  cat_cols+ccols['disc_cols']]
    
    if len(d_cols)<1:
        d_cols='auto'
    S=[MI(np.array(y_orig).reshape(-1,1),y_orig)[0]] #estimate target entropy

    #remove NaNs
    x_train=X_orig.dropna()
    #encode catagoricals
    if cat_cols:
        x_train[cat_cols]=OrdinalEncoder().fit_transform(x_train[cat_cols])

    mutual_array=pd.Series(MI(x_train,y_orig,discrete_features=d_cols),index=x_train.columns)
    interfeat=[]
    if crossmi:
        for i,col in enumerate(X_orig.columns):
            interfeat.append(MI(x_train,x_train[col],discrete_features=d_cols))
            S.append(interfeat[-1][i])
        for i in range(len(interfeat)):
            interfeat[i][i]=0
        interfeat=pd.DataFrame(interfeat,columns=X_orig.columns,index=X_orig.columns)
    else:
        S+=[0]*len(X_orig.columns)
       
    S=np.array(S)
    mi_orig=pd.Series(mutual_array,index=X_orig.columns)
    Sh=pd.Series(S,index=['target']+X_orig.columns.to_list())
    noise=Sh[1:]-mi_orig
    original={'mi':mi_orig.to_dict(),'Sh':Sh.to_dict(),'noise':noise.to_dict()}
    score=None
    if crossmi:
        score=mi_score(original,interfeat,weight=weight,criterion=criterion)

    if graphics:
        sorted_features=list(score.keys())
        
        heigh=10 if X_orig.shape[1]<20 else X_orig.shape[1]//2
        plt.figure(figsize=(20,heigh))
        plt.subplot(2,2,1)
        sns.barplot(y=sorted_features,x=mutual_array[sorted_features])
        plt.ylabel('features sorted by rank')
        plt.xlabel('I(X,Y)')
        plt.title('features-label mutual information')
        plt.grid()
        
        
        if crossmi:
            plt.subplot(2,2,2)
            sns.barplot(y=sorted_features,x=noise[sorted_features]/Sh[sorted_features])
            plt.xlim(left=min(noise[sorted_features]/Sh[sorted_features])*0.9)
            plt.title('% noise')
            plt.ylabel('')
            plt.xlabel('1-I(X,Y)/H(X)')
            plt.grid()
        
            
            plt.subplot(2,2,3)
            sns.barplot(y=sorted_features,x=mutual_array[sorted_features]/noise[sorted_features])
            plt.title('Signal/noise')
            plt.ylabel('')
            plt.xlabel('I(X,Y)/[1-I(X,Y)/H(X)]')
            plt.grid()
            
            plt.subplot(2,2,4)
            to_show=interfeat.loc[sorted_features,sorted_features]
            
            for coli in sorted_features:
                for colj in sorted_features:
                    to_show.loc[coli,colj]=interfeat.loc[coli,colj]/min(original['Sh'][coli],original['Sh'][colj])
                
            sns.heatmap(to_show,cmap='viridis',mask=~np.tri(X_orig.shape[1],dtype=bool))
            
            plt.title('Features mutual information (relative)')
            plt.ylabel('')
            plt.xlabel('I(X_i,X_j)/min(H(X_i),H(X_j))')
        plt.show()
        plt.close()
        
    return original,interfeat,score

In [16]:
%%time
ent,crossmi,mi_sc=get_mutual_info(dt_class,y_class,weight=penalty_weight)

seq_mir=[]
for f in mi_sc.index:
    if f.startswith('random_'):
        break
    seq_mir.append(f)
seq_mir=sorted(seq_mir)
cvs_mir=cross_val_score(model,dt_class[seq_mir],y_class,scoring='accuracy',cv=10).mean()
print(f'{cvs_mir:0.5f}  {", ".join(seq_mir)}')

In [34]:
%%time
ent,crossmi,mi_sc=get_mutual_info(dt_gen,y_gen,weight=penalty_weight)

seq_mir=[]
for f in mi_sc.index:
    if f.startswith('random_'):
        break
    seq_mir.append(f)
seq_mir=sorted(seq_mir)
cvs_mir=cross_val_score(model,dt_gen[seq_mir],y_gen,scoring='r2',cv=10).mean()
print(f'{cvs_mir:0.5f}  {", ".join(seq_mir)}')

In [4]:
all_scores = {}
ascending = True
for r in range(n_repeats):
    for m in [f for f in oofs[r].columns if f not in [target]]:
        if r == 0:
            all_scores[m] = []
        all_scores[m].append(score_(train[target], oofs[r][m]))
        
all_scores = pd.DataFrame(all_scores)
_col_order = list(all_scores.mean(axis=0).sort_values(ascending = ascending).index)
all_scores = all_scores[_col_order]

_t = list(all_scores.mean(axis=0).sort_values(ascending = ascending).values)
_labels = [f"{l:15} ({v:.5})" for l,v in zip(_col_order, _t)]

_, ax = plt.subplots(1, 1, figsize = (10, int(all_scores.shape[1] * 1/2)))
ax.boxplot(all_scores, vert = False, labels = _labels)
ax.set_title("OOFS R2")
ax.set_xlabel("R2 (higher is better)")
ax.xaxis.set_ticks_position("top")
ax.xaxis.set_label_position('top')
ax.spines[["right", "bottom"]].set_visible(False)

In [5]:
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform

def hierarchical_clustering(data, title):
    
    fig, ax = plt.subplots(1, 1, figsize=(15, 10), dpi=120)
    correlations = data.corr()
    converted_corr = 1 - np.abs(correlations)
    Z = linkage(squareform(converted_corr), 'complete')
    
    dn = dendrogram(Z, labels = data.columns, ax = ax, above_threshold_color = '#ff0000', orientation = 'right')
    hierarchy.set_link_color_palette(None)
    plt.grid(axis='x')
    plt.title(f'Hierarchical clustering, Dendrogram - part {title} ', fontsize=18, fontweight='bold')
    plt.show()
    
for i in range(n_repeats):
    hierarchical_clustering(oofs[i][models], title = f"{i}")

In [18]:
from sklearn.linear_model import LinearRegression


In [11]:
from sklearn.linear_model import LinearRegression

model_lr = LinearRegression()

model_lr.fit(X_train, y_train)

In [29]:
class NumberSelector(BaseEstimator, TransformerMixin):
    # Initialize the transformer with a key
    def __init__(self, key):
        # Store the provided key as an instance variable
        self.key = key

    # The fit method is required but doesn't need to do anything for this transformer
    def fit(self, X, y=None):
        # Return self to allow for method chaining
        return self

    # The transform method selects and returns the column specified by the key
    def transform(self, X):
        # Use the key to select the column from the DataFrame X and return it
        return X[[self.key]]



# Initialize an empty list to store the results
results = []

# Initialize variables to track the best model and its R-squared score
best_model = None
best_r2 = float('-inf')

# Iterate over each model in the models dictionary
for name, model in models.items():


    # List to store transformers for each column
    final_transformers = []
    
    # Create a pipeline for each column in the training set
    for col in X_train.columns:
        pipeline = Pipeline([
            ('selector', NumberSelector(key=col)),  # Select the column
            ('scaler', StandardScaler()),
        ])
        final_transformers.append((col, pipeline))
    
    # Combine all column transformers into a FeatureUnion
    feats = FeatureUnion(final_transformers)
    
    # Create a pipeline for feature processing
    feature_processing = Pipeline([('feats', feats)])
    
    # Create the final pipeline including the feature processing and the classifier
    pipeline = Pipeline([
        ('features', feature_processing),
        ('model', model)
    ])

    
    # Check if hyperparameters for the current model are defined in the params dictionary
    if name in params:
        # If hyperparameters are defined, perform grid search with cross-validation
        grid_search = GridSearchCV(pipeline, params[name], cv=5, scoring=rmse_scorer)
    else:
        # If hyperparameters are not defined, perform grid search with default hyperparameters
        grid_search = GridSearchCV(pipeline, {}, cv=5, scoring=rmse_scorer)
      
    # Fit the grid search to the training data
    grid_search.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = grid_search.predict(X_test)
    
    # Calculate R-squared score and RMSE on the test set
    r2 = r2_score(y_test, y_pred)
    rmse_val = rmse(y_test, y_pred)
    
    # Update the best model and its R-squared score if the current model performs better
    if r2 > best_r2:
        best_r2 = r2
        best_model = grid_search
    
    # Append the results to the results list
    results.append({
        'Model': name,
        'R-Squared Score': r2,
        'RMSE Score': rmse_val,
        'Best Parameters': grid_search.best_params_
    })

# Create a DataFrame from the results list
results_df = pd.DataFrame(results)

In [31]:
# Calculate residuals
residuals = y_test - y_pred

# Calculate standard deviation of residuals
sigma = np.std(residuals)

# Create a list for hue parameter
hue = list(map(lambda x: abs(x) < sigma, residuals))

# Create a residual plot
# plt.figure(figsize=(10, 7))
# sns.scatterplot(x=y_pred, y=residuals, hue=hue, palette = ['#E5C287', '#007F79'], s=100)
# plt.axhline(y=0, color='#A79277', linestyle='--', lw=1)
# plt.xlabel('Predicted')
# plt.ylabel('Residuals')
# plt.title('Residual Plot')
# plt.savefig('./images/residuals.png')
# plt.show()


In [17]:
# Getting the model performances
xgr_model = make_pipeline(StandardScaler(),
                          XGBRegressor(device='cuda'))

cbr_model = make_pipeline(StandardScaler(),
                          CatBoostRegressor(verbose=False, task_type='GPU'))

cross_validate_print(xgr_model, X, y)
cross_validate_print(cbr_model, X, y)

In [43]:
ensemble_train[:5,:]

In [35]:
y_train_pred = best_model.predict(x_train)

# Make predictions on the test set
y_test_pred = best_model.predict(x_test)

# Calculate RMSE, MSE, and MAE for training set
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)

# Calculate RMSE, MSE, and MAE for test set
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Print the results
print("Training set:")
print("RMSE:", rmse_train)
print("MSE:", mse_train)
print("MAE:", mae_train)

print("\nTest set:")
print("RMSE:", rmse_test)
print("MSE:", mse_test)
print("MAE:", mae_test)

In [37]:
df_test = pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv")

In [43]:
output_df

In [52]:
model.summary()

In [53]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

y_train_pred = model.predict(x_train)

# Make predictions on the test set
y_test_pred = model.predict(x_test)

# Calculate RMSE, MSE, and MAE for training set
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)

# Calculate RMSE, MSE, and MAE for test set
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Print the results
print("Training set:")
print("RMSE:", rmse_train)
print("MSE:", mse_train)
print("MAE:", mae_train)

print("\nTest set:")
print("RMSE:", rmse_test)
print("MSE:", mse_test)
print("MAE:", mae_test)

In [55]:
model.fit(x_train , y_train , epochs = 500 , batch_size = 1024 , validation_data = (x_test , y_test))

In [56]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

y_train_pred = model.predict(x_train)

# Make predictions on the test set
y_test_pred = model.predict(x_test)

# Calculate RMSE, MSE, and MAE for training set
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)

# Calculate RMSE, MSE, and MAE for test set
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Print the results
print("Training set:")
print("RMSE:", rmse_train)
print("MSE:", mse_train)
print("MAE:", mae_train)

print("\nTest set:")
print("RMSE:", rmse_test)
print("MSE:", mse_test)
print("MAE:", mae_test)

In [55]:
model2=XGBRegressor(random_state=0,learning_rate=.1,max_depth=10,subsample=0.70,booster="gbtree")

In [56]:
# model2=MLPRegressor(random_state=0)

In [58]:
model2.fit(xtrain,ytrain)

In [59]:
# ytrain=np.array(ytrain).reshape(-1,)

In [60]:
# y.ndim

In [61]:
prediction=model2.predict(xtrain)
testy=model2.predict(xval)

In [62]:
# p=model.predict(train)

In [64]:
from sklearn.metrics import r2_score

In [65]:
# neural net-mlp regressor train score= 0.8509


In [66]:

r2_score(ytrain,prediction)

In [67]:
from sklearn.model_selection import cross_val_score

In [70]:
predict=model2.predict(testf)

In [20]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

lr_model = LinearRegression()
lr_model.fit(x_train , y_train)


# Make predictions on the training set
y_train_pred = lr_model.predict(x_train)

# Make predictions on the test set
y_test_pred = lr_model.predict(x_test)

# Calculate RMSE, MSE, and MAE for training set
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)

# Calculate RMSE, MSE, and MAE for test set
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Print the results
print("Training set:")
print("RMSE:", rmse_train)
print("MSE:", mse_train)
print("MAE:", mae_train)

print("\nTest set:")
print("RMSE:", rmse_test)
print("MSE:", mse_test)
print("MAE:", mae_test)


In [28]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Initialize the AdaBoost Regressor model
ada_model = AdaBoostRegressor()

# Train the AdaBoost Regressor model on the training data
ada_model.fit(x_train, y_train)

# Make predictions on the training set
y_train_pred = ada_model.predict(x_train)

# Make predictions on the test set
y_test_pred = ada_model.predict(x_test)

# Calculate RMSE, MSE, and MAE for training set
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)

# Calculate RMSE, MSE, and MAE for test set
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Print the results
print("Training set:")
print("RMSE:", rmse_train)
print("MSE:", mse_train)
print("MAE:", mae_train)

print("\nTest set:")
print("RMSE:", rmse_test)
print("MSE:", mse_test)
print("MAE:", mae_test)

In [34]:
outputs = ada_model.predict(x_target)
output_df = pd.DataFrame(outputs , columns =  ["FloodProbability"])

In [8]:
def create_model(num_layers, num_neurons):
    model = Sequential()
    model.add(Input((20,)))
    print("Model Created")
    model.add(Dense(num_neurons, activation=LeakyReLU(negative_slope=0.01)))
    #model.add(Dropout(0.2, seed=42))
    model.add(Dense(num_neurons, activation=LeakyReLU(negative_slope=0.01)))
    model.add(Dense(num_neurons, activation=LeakyReLU(negative_slope=0.01)))
    model.add(Dense(num_neurons, activation=LeakyReLU(negative_slope=0.01)))
    
    model.add(Dense(1, activation='sigmoid'))  # Output layer

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[R2Score()])
    #model.compile(optimizer='adam', loss='mean_squared_error')
    
    return model

In [9]:
# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
early_stopping = EarlyStopping(monitor = "r2_score", # watch the val loss metric
                               patience = 5,
                               restore_best_weights = True) # if val loss decreases for 3 epochs in a row, stop training

reduce_lr = ReduceLROnPlateau(monitor='r2_score', factor=0.2, patience=3, min_lr=1e-6)

In [10]:

model = create_model(5, 8)
        
# Train the model
history = model.fit(x_train, y_train, epochs=10, callbacks=[
                        early_stopping,
                        reduce_lr
    ],
    validation_data=(x_test, y_test))

In [11]:
model.summary()

In [13]:
# Assuming y_test_actual contains the actual target values
# Assuming y_test_predicted contains the predicted target values
y_predicted = model.predict(x_test)

In [18]:
'''from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=50, random_state=42)

# Train the model
rf_model.fit(x_train, y_train)'''

In [19]:
'''#Predict probabilities on the test set
y_pred = rf_model.predict(x_test)'''

In [11]:
# Model intialization
lr_model = LinearRegression()

lr_model.fit(X_train, y_train)

In [12]:
# Prediction
y_pred = lr_model.predict(X_val)

In [16]:
# define model
sgd_model = SGDRegressor()

sgd_model.fit(X_train, y_train)

In [17]:
# Prediction
y_pred = sgd_model.predict(X_val)

In [21]:
rfr = RandomForestRegressor()

rfr.fit(X_train, y_train)

In [22]:
y_pred = rfr.predict(X_val)

In [23]:
# Mean Absolute Error
rfr_mae = mean_absolute_error(y_val, y_pred)
print(f'Mean Absolute Error: {np.round(rfr_mae, 5)}')

In [24]:
# Mean Squared Error
rfr_mse = mean_squared_error(y_val, y_pred)
print(f'Mean Squared Error: {np.round(rfr_mse, 5)}')

In [26]:
# Create a base linear regression
base_regressor = LinearRegression()

# Create a bagging regressor with linear regressin
bagged_regressor = BaggingRegressor(base_regressor, n_estimators=10)

# Train the bagged regressor on your data
bagged_regressor.fit(X_train, y_train)

In [27]:
y_pred = bagged_regressor.predict(X_val)

In [28]:
# Mean Absolute Error
bgr_mae = mean_absolute_error(y_val, y_pred)
print(f'Mean Absolute Error: {np.round(bgr_mae, 5)}')

In [30]:
# R-square score
bgr_r2 = r2_score(y_val, y_pred)
print(f'R-Squared Score: {np.round(bgr_r2 * 100, 3)}')

In [31]:
# Define model
gbr_model = GradientBoostingRegressor()
gbr_model.fit(X_train, y_train)

In [32]:
# Prediction
y_pred = gbr_model.predict(X_val)

In [36]:
# Define model
svr_model = SVR()

svr_model.fit(X_train, y_train)

In [37]:
# Prediction
y_pred = svr_model.predict(X_val)

In [38]:
# Mean Absolute Error
svr_mae = mean_absolute_error(y_val, y_pred)
print(f'Mean Absolute Error: {np.round(svr_mae, 5)}')

In [39]:
# Mean Squared Error
svr_mse = mean_squared_error(y_val, y_pred)
print(f'Mean Squared Error: {np.round(svr_mse, 5)}')

In [41]:
LGBM_model = LGBMRegressor(n_estimators=100)

LGBM_model.fit(X_train, y_train)

In [42]:
# Prediction
y_pred = LGBM_model.predict(X_val)

In [43]:
# Mean Absolute Error
lgbm_mae = mean_absolute_error(y_val, y_pred)
print(f'Mean Absolute Error: {np.round(lgbm_mae, 5)}')

In [44]:
# Mean Squared Error
lgbm_mse = mean_squared_error(y_val, y_pred)
print(f'Mean Squared Error: {np.round(lgbm_mse, 5)}')

In [46]:
results = pd.DataFrame({'Model':['LinearRegression', 'SGDRegressor', 'RandomForestRegressor', 'BaggingRegressor', 'GradientBoostingRegressor', 'SVR', 'LGBM'],
                       'MAE': [lr_mae, sgd_mae, rfr_mae, bgr_mae, gbr_mae, svr_mae, lgbm_mae],
                       'MSE': [lr_mse, sgd_mse, rfr_mse, bgr_mse, gbr_mse, svr_mse, lgbm_mse],
                       'R2': [lr_r2, sgd_r2, rfr_r2, bgr_r2, gbr_r2, svr_r2, lgbm_r2]})
results

In [ ]:
!pip install lazypredict

from lazypredict.Supervised import LazyRegressor
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import iqr

# Models
from sklearn.linear_model import LinearRegression, BayesianRidge, ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor, VotingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
import xgboost as xgb

# Preprocessing
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Evaluation
from sklearn.metrics import mean_squared_error, r2_score

# Options
pd.set_option('display.max_columns',50)
plt.style.use('bmh')

In [18]:
regressors = [
    ('bayesian ridge', BayesianRidge()),
    ('elastic net',ElasticNetCV()),
    ('linear_reg', LinearRegression()),
    ('lasso',LassoCV()),
    ('ridge',RidgeCV())
]

In [19]:
evals = {}
for clf, model in regressors:
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    score = r2_score(y_test, y_pred)
    evals[clf] = score

In [20]:
evals_df = pd.DataFrame({'model':evals.keys(),'r2_score':evals.values()})
evals_df.sort_values('r2_score', inplace = True)
evals_df

In [23]:
voting_regressor = VotingRegressor(estimators=[
    ('bayesian_ridge', best_estimators['bayesian_ridge']),
    ('elastic_net', best_estimators['elastic_net']),
    ('linear_reg', best_estimators['linear_reg']),
    ('lasso', best_estimators['lasso']),
    ('ridge', best_estimators['ridge'])
])

# Fit the VotingRegressor on the dataset
voting_regressor.fit(X, y)
y_preds = voting_regressor.predict(test)

In [ ]:
!pip install -q catboost

In [25]:
xgb_model = XGBRegressor()
cat_model = CatBoostRegressor(verbose=False)
lgb_model = lgb.LGBMRegressor(verbose=-1)

meta_model = LinearRegression()

stacked = StackingRegressor(estimators=[
    ('xgb', xgb_model),
    ('cat', cat_model),
    ('lgb', lgb_model)],
    final_estimator=meta_model)

stacked.fit(X_train, y_train)
preds = stacked.predict(X_val)

print("Stacked R2:", r2_score(y_val, preds))

In [26]:
xgb_model = XGBRegressor(n_estimators = 8000, max_depth = 9,
                   learning_rate =  0.05371502553155743,
                   subsample = 0.85715838272758116,
                   colsample_bytree = 0.892390046436166,
                   gamma = 0.012984290742285246,
                   min_child_weight = 3,
                   random_state = 0)

cat_model = CatBoostRegressor(n_estimators = 8000, learning_rate = 0.011277016304363601,
                       depth = 8, min_data_in_leaf = 98, random_state = 0, devices='0', verbose=False)

lgb_model = lgb.LGBMRegressor(boosting_type = 'gbdt', n_estimators = 2000,
                    learning_rate =  0.012, num_leaves = 250,
                    subsample_for_bin = 165700, min_child_samples = 114,
                    reg_alpha = 2.075e-06, reg_lambda = 3.839e-07,
                    colsample_bytree = 0.9634, subsample = 0.9592,
                    max_depth = 10, random_state = 0, verbose = 0)

xgb_model.fit(X_train, y_train)
cat_model.fit(X_train, y_train)
lgb_model.fit(X_train, y_train)

xgb_pred = xgb_model.predict(X_val)
cat_pred = cat_model.predict(X_val)
lgb_pred = lgb_model.predict(X_val)

y_pred = 0.33*xgb_pred + 0.33*cat_pred + 0.34*lgb_pred
print("Stacked R2:", r2_score(y_val, y_pred))

In [27]:
input_layer = Input(shape=(X_train.shape[1], ))

x = Dense(512, activation='gelu')(input_layer)
x = BatchNormalization()(x)
x = Dropout(0.15)(x)

x = Dense(256, activation='gelu')(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)

x = Dense(128, activation='gelu')(x)
x = BatchNormalization()(x)
x = Dropout(0.05)(x)

x = Dense(64, activation='relu')(x)
x = Dropout(0.001)(x)

output_layer = Dense(1, activation='linear')(x)

my_model = Model(input_layer, output_layer)
opt = Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.9999, epsilon=1e-01)
my_model.compile(optimizer='adam', loss='mse')


model_checkpoint = ModelCheckpoint(
    filepath="/content/best_model.keras",
    monitor="val_loss",
    verbose=1,
    save_best_only=True)

early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=20,
    verbose=1,
    mode="auto")

history = my_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=1000,
    batch_size=512,
    verbose=1,
    callbacks=[model_checkpoint, early_stopping])

In [28]:
best_model = load_model("/content/best_model.keras")
y_pred = best_model.predict(X_val)
r2 = r2_score(y_val, y_pred)
print("R^2 score:", r2)

In [29]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()

In [15]:
ri.fit(X_train_t,y_train)
ri.score(X_train_t,y_train)

In [16]:
ri.score(X_test_t,y_test)

In [22]:
test_data_t=pipe.transform(test_data_new)
test_final=gv.predict(test_data_t)

In [15]:
lr = LinearRegression()

# Fit the instance on the data and then predict the expected value
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

# Evaluate the model
r2_score(lr_pred,y_test)

In [18]:
test_ids = test_df['id']

# Remove 'id' column from df_test
test_df.drop(columns=['id'], inplace=True)

# Create a new column 'fsum' which is the sum of all feature columns for each row
test_df['fsum'] = test_df.sum(axis=1) 

# Scale data
test_df = scaler.transform(test_df)

In [ ]:
!pip install --upgrade pip

!pip install catboost
!pip install xgboost
!pip install lightgbm

In [46]:
linear = LinearRegression()

linear.fit(X_train,y_train)
print(f"Training score {linear.score(X_train,y_train)}")
y_pred = linear.predict(X_test)
print(f"R2: {r2_score(y_test,y_pred)}")
linear_score = r2_score(y_test,y_pred)
linear_Tscore = linear.score(X_train,y_train)
# without dropping
# Training score 0.8457594591756623
# R2: 0.8454697171996348
# after dropping
# Training score 0.8459854314345968
# R2: 0.8446050432676502

# BETTER TRAIN, WORSE TEST

# Training score 0.8449632386720447
# R2: 0.8448671419805959

# Training score 0.8451228724650743
# R2: 0.8450173448266534

# linear complete data
# Training score 0.8488268363920475
# R2: 0.8484823666612045

In [47]:
cat_model = CatBoostRegressor()
cat_model.fit(X_train, y_train)

y_pred = cat_model.predict(X_test)
print(f"Training score {cat_model.score(X_train,y_train)}")
print(f"R2: {r2_score(y_test,y_pred)}")
CatBoost_score = r2_score(y_test,y_pred)
CatBoost_Tscore = cat_model.score(X_train,y_train)

# without dropping
# Training score 0.849059370869179
# R2: 0.8458341178802313
# after dropping
# Training score 0.8495838425142408
# R2: 0.8450550512464129

# BETTER TRAIN, WORSE TEST

# Training score 0.8511034268861708
# R2: 0.8484659428975101

# Training score 0.8510983395654057
# R2: 0.8482567996850483

# with high corr only
# Training score 0.8689939905594077
# R2: 0.8671051110548669

# Training score 0.8692380657333689
# R2: 0.8673866049021248

# cat complete data
# Training score 0.8712936535722917
# R2: 0.8684364777340219

# Training score 0.8713026887254586
# R2: 0.868381238518181

In [48]:
xg_model = XGBRegressor()
xg_model.fit(X_train, y_train)

y_pred = xg_model.predict(X_test)
print(f"Training score {xg_model.score(X_train,y_train)}")
print(f"R2: {r2_score(y_test,y_pred)}")
XGBoost_score = r2_score(y_test,y_pred)
XGBoost_Tscore = xg_model.score(X_train,y_train)

# without dropping
# Training score 0.8185763391684946
# R2: 0.8092899007536791
# after dropping
# Training score 0.818278893261753
# R2: 0.8075431863252942

# BETTER TRAIN, WORSE TEST

# Training score 0.8320398257448948
# R2: 0.8265681584631458

# Training score 0.8370201885233045
# R2: 0.8313670522106444

# with high corr only
# Training score 0.8694170389349443
# R2: 0.8668235370658703

# Training score 0.8698661310623312
# R2: 0.8672521384497449

# xg complete data
# Training score 0.8717124360778755
# R2: 0.8681429795144493

In [56]:
# ABC_model = AdaBoostRegressor(estimator=bc_model)
# ABC_model.fit(X_train, y_train)

# y_pred = ABC_model.predict(X_test)
# print(f"Training score {ABC_model.score(X_train,y_train)}")
# print(f"R2: {r2_score(y_test,y_pred)}")
# ABC_score = r2_score(y_test,y_pred)
# ABC_Tscore = ABC_model.score(X_train,y_train)

In [58]:
vc_model = VotingRegressor(estimators)
vc_model.fit(X_train, y_train)

y_pred = vc_model.predict(X_test)
print(f"Training score {vc_model.score(X_train,y_train)}")
print(f"R2: {r2_score(y_test,y_pred)}")
VR_score = r2_score(y_test,y_pred)
VR_Tscore = vc_model.score(X_train,y_train)

# without dropping
# Training score 0.8456583210297507
# R2: 0.8417872917459406
# after dropping
# Training score 0.8459156228601873
# R2: 0.8408403700058219

# BETTER TRAIN, WORSE TEST

# Training score 0.8477804436778823
# R2: 0.8451736968761587

# Training score 0.8486829862581831
# R2: 0.8459588292860563

# with high corr only
# Training score 0.8661221100918872
# R2: 0.8645853469530693

# Training score 0.8664405855516222
# R2: 0.8649047721345305

# voting complete data
# Training score 0.8684375143959511
# R2: 0.8661693484135519

In [59]:
SC_model = StackingRegressor(estimators)
SC_model.fit(X_train, y_train)

y_pred = SC_model.predict(X_test)
print(f"Training score {SC_model.score(X_train,y_train)}")
print(f"R2: {r2_score(y_test,y_pred)}")
StackReg_score = r2_score(y_test,y_pred)
StackReg_Tscore = SC_model.score(X_train,y_train)

# Training score 0.8486827459210073
# R2: 0.8454175629190451
# after dropping
# Training score 0.8488442450516133
# R2: 0.8450995697555888

# BETTER TRAIN, WORSE TEST

# Training score 0.8513106187494396
# R2: 0.8484909488287333

# Training score 0.8508926163373516
# R2: 0.8482738572320817

# with high corr only
# Training score 0.8692084944453877
# R2: 0.8671144372609388

# Training score 0.8695405752611133
# R2: 0.8674593045005337

# complete data
# Training score -0.8129348173408533
# R2: -0.8206480970793406

In [60]:
LG_model = LGBMRegressor(boosting_type = 'gbdt', 
                         n_estimators=2000, 
                         learning_rate =  0.012,
                         num_leaves = 250, 
                         subsample_for_bin= 165700, 
                         min_child_samples= 114, 
                         reg_alpha= 2.075e-06, 
                         reg_lambda= 3.839e-07, 
                         colsample_bytree= 0.9634,
                         subsample= 0.9592, 
                         max_depth= 10,
                         random_state=0,
                         verbosity=-1)
LG_model.fit(X_train, y_train)

y_pred = LG_model.predict(X_test)
print(f"Training score {LG_model.score(X_train,y_train)}")
print(f"R2: {r2_score(y_test,y_pred)}")
LGReg_score = r2_score(y_test,y_pred)
LGReg_Tscore = LG_model.score(X_train,y_train)

# without dropping
# Training score 0.7722033445629952
# R2: 0.7656664606302841
# after dropping
# Training score 0.7736411682470161
# R2: 0.765641172682489

# BETTER TRAIN, WORSE TEST

# 0.8355890630044472
# R2: 0.8330412134653961

# Training score 0.8179158545212489
# R2: 0.8154329421048914

# with high corr only
# Training score 0.8677071132111231
# R2: 0.8670502864448657

# Training score 0.8680294090677185
# R2: 0.8673631191473345

# LGBM complete data
# Training score 0.8693953938884129
# R2: 0.8684146412104383

# [LightGBM] [Info] Start training from score 0.504480
# Training score 0.8693418511330352
# R2: 0.868388306489848

In [61]:
# result = [
#     {"Model": "Linear", "Training score": linear_score , "R2": linear_Tscore},
#     {"Model": "CatBoost", "Training score": CatBoost_score , "R2": CatBoost_Tscore},
#     {"Model": "XGBoost", "Training score": XGBoost_score , "R2": XGBoost_Tscore ,
#     {"Model": "SVR", "Training score": SVR_score, "R2": SVR_Tscore },
#     {"Model": "KNN", "Training score": KNN_score , "R2": KNN_Tscore },
#     {"Model": "Decision Tree", "Training score": DT_score , "R2": DT_Tscore },
#     {"Model": "Random Forest", "Training score": RF_score , "R2": RF_Tscore },
#     {"Model": "Bagging", "Training score": BagReg_score , "R2": BagReg_Tscore },
#     {"Model": "Extra Trees", "Training score": ET_score , "R2": ET_Tscore },
#     {"Model": "AdaBoost", "Training score": ABC_score , "R2": ABC_Tscore },
#     {"Model": "VotingRegressor", "Training score": VR_score , "R2": VR_Tscore },
#     {"Model": "Stacking", "Training score": StackReg_score , "R2": StackReg_Tscore ,
#     {"Model": "LGBMRegressor", "Training score": LGReg_score , "R2": LGReg_Tscore 
# ]

# result_df = pd.DataFrame(result)

In [63]:
# result_df = result_df.sort_values(by='Training score', ascending=False)

In [67]:
# cat_model = CatBoostRegressor()
# cat_model.fit(X, y)

# y_pred_cat = cat_model.predict(test_set)
# print(f"Training score {cat_model.score(X,y)}")

# # Training score 0.8688006661365003

# # Training score 0.8690503412244763

In [69]:
# y_pred_SC = pd.DataFrame(y_pred_SC)
# output_SC = test_set_id
# output_SC["FloodProbability"] = pd.DataFrame(y_pred_SC)
# output_SC = output_SC.set_index("id")
# output_SC.to_csv("FloodProbability_OHE_project_FEng_SC.csv")

In [73]:
LG_model = LGBMRegressor(boosting_type = 'gbdt', 
                         n_estimators=2000, 
                         learning_rate =  0.012,
                         num_leaves = 250, 
                         subsample_for_bin= 165700, 
                         min_child_samples= 114, 
                         reg_alpha= 2.075e-06, 
                         reg_lambda= 3.839e-07, 
                         colsample_bytree= 0.9634,
                         subsample= 0.9592, 
                         max_depth= 10,
                         random_state=0,
                         verbosity=-1)
LG_model.fit(X, y)

y_pred_LG = LG_model.predict(test_set)
print(f"Training score {LG_model.score(X,y)}")

In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [7]:
model = XGBRegressor()

In [8]:
model.fit(X,y)

In [11]:
predictions = model.predict(X_test)

In [12]:
predictions.shape

In [14]:
model_1 = XGBRegressor()
model_1.fit(
    X_train, Y_train
)

In [16]:
mse = mean_absolute_error(y_test, predictions1)
mse

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import r2_score

In [9]:
model = Sequential([
    Dense(64, activation='relu', input_dim=X_train.shape[1]),
    Dropout(0.1),
    
    Dense(32, activation='relu'),
    Dropout(0.1),
    
    Dense(16, activation='relu'),
    Dense(1, activation='linear'),

])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['r2_score'])
model.summary()

In [10]:
call_back = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True , verbose=0)

In [11]:
history = model.fit(X_train, y_train, epochs=100, batch_size=64, callbacks= call_back , validation_data=(X_val, y_val), verbose=1)

In [12]:
# Define needed variables
tr_loss = history.history['loss']
val_loss = history.history['val_loss']
index_loss = np.argmin(val_loss)
val_lowest = val_loss[index_loss]

Epochs = [i+1 for i in range(len(tr_loss))]
loss_label = f'best epoch= {str(index_loss + 1)}'

# Plot training history
plt.figure(figsize= (20, 8))
plt.style.use('fivethirtyeight')

plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
plt.scatter(index_loss + 1, val_lowest, s= 150, c= 'blue', label= loss_label)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout
plt.show()


In [13]:
max(history.history['val_r2_score'])

In [15]:
call_back2 = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True , verbose=0)

In [16]:
history2 = model2.fit(X_train, y_train, epochs=100, batch_size=64, callbacks= call_back2 , validation_data=(X_val, y_val), verbose=1)

In [20]:
call_back3 = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True , verbose=0)

In [21]:
history3 = model3.fit(X_train, y_train, epochs=100, batch_size=64, callbacks= call_back3 , validation_data=(X_val, y_val), verbose=1)

In [24]:
model4 = Sequential([
    Dense(8, activation='relu', input_dim=X_train.shape[1]),    
    Dense(1, activation='linear'),

])

model4.compile(optimizer=SGD(learning_rate=0.001), loss='mean_squared_error', metrics=['r2_score'])
model4.summary()

In [28]:
max(history4.history['val_r2_score'])

In [29]:
y_pred = model3.predict(test)
y_pred


In [30]:
# make y_pred a 1d array
y_pred = y_pred.reshape(y_pred.shape[0])

In [31]:
submission = pd.DataFrame({'id': test.index.values, 'FloodProbability': y_pred}, columns=['id', 'FloodProbability'])

submission.to_csv('submission.csv', index=False)

In [10]:
y_pred=np.ndarray.flatten(y_pred)

In [14]:
# lets run polynomial regression on our data due to it's best results

# Create a polynomial features object with degree 2

poly_reg = PolynomialFeatures(degree=2)

X_poly = poly_reg.fit_transform(testing)


FloodProbability = model.predict(X_poly)

In [30]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


# Instantiate baseline regression algorithms
lin_reg = LinearRegression()
lasso = Lasso()
ridge = Ridge()
svr = SVR()
lgbm = LGBMRegressor()
xgb_reg = XGBRegressor()
kn_reg = KNeighborsRegressor()
tree = DecisionTreeRegressor(random_state = 42)
forest = RandomForestRegressor(random_state = 42)
grad_boost = GradientBoostingRegressor(random_state = 42)

# Define a function that fits and trains training data then test it on the test data 
def building_model(model, model_name, X_train, X_val):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
        
    train_score.append(round(model.score(X_train, y_train), 3))
    test_score.append(round(model.score(X_val, y_val), 3))
    r2.append(round(r2_score(y_val, y_pred), 3))
    mae.append(round(mean_absolute_error(y_val, y_pred), 3))
    rmse.append(round(np.sqrt(mean_squared_error(y_val, y_pred)), 3))

In [45]:
y_pred = pd.Series(y_pred, index = X_test.index, name = 'predicted_proba')
results = pd.concat([y_test, y_pred], axis = 1)
results

In [14]:
pipe.fit(X_train,y_train)

In [15]:
y_predict=pipe.predict(X_test)

In [30]:
XGBooster.fit(X,np.ravel(y))

In [47]:
LGBMBooster.fit(X,np.ravel(y))

In [64]:
voter.fit(X,np.ravel(y))

In [69]:
#Builing a Stacking classifier
stack = StackingRegressor(estimators = estimators, final_estimator = final_estimator, cv = 3, n_jobs = -1, verbose = 1)

In [70]:
stack.fit(X,np.ravel(y))

In [75]:
stack_result

In [76]:
stack_result.to_csv('submission.csv')

In [16]:
# Spliting Data
X = train.drop(["id", "FloodProbability"], axis=1)
y = train["FloodProbability"]
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [20]:
# List of models to test
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'XGBOOST': xgb.XGBRegressor(),
    'LGBM' : ltb.LGBMRegressor()
}

# DataFrame to store model performances
performance = pd.DataFrame(columns=['Model', 'MSE'])

# Fit models and calculate performances
for name, model in models.items():
    model.fit(X_train[feats], y_train)
    X_test[f'{name}_pred'] = model.predict(X_test[feats])
    mse = mean_squared_error(y_test, X_test[f'{name}_pred'])
    performance = pd.concat([performance, pd.DataFrame({'Model': [name], 'MSE': [mse]})], ignore_index=True)

# Display model performances
print(performance)

# Visualize residuals for each model
plt.figure(figsize=(14, 10))

for i, (name, model) in enumerate(models.items(), 1):
    plt.subplot(4, 2, i)
    X_test['residuals'] = y_test - X_test[f'{name}_pred']
    sns.boxplot(x=X_test['residuals'])
    plt.title(f'Residuals for {name}')
    plt.xlabel('Residuals')

plt.tight_layout()
plt.show()

In [21]:
# Model combination
model_1 = Ridge()
model_2 = xgb.XGBRegressor()

# Training of models on train
model_1.fit(X_train[feats], y_train)
model_2.fit(X_train[feats], y_train)

# Predictions of models on train
pred_train_1 = model_1.predict(X_train[feats])
pred_train_2 = model_2.predict(X_train[feats])
residu_train_1 = pred_train_1 - y_train

# Combination of predictions on train
stacked_predictions_train = np.column_stack((pred_train_1, pred_train_2))

# Meta motel
meta_model = ltb.LGBMRegressor()
meta_model.fit(stacked_predictions_train, y_train)

# Prediction of models on test
pred_test_1 = model_1.predict(X_test[feats])
pred_test_2 = model_2.predict(X_test[feats])
residu_test_1 = pred_test_1 - y_test

# Combinaison of prediction on test
stacked_predictions_test = np.column_stack((pred_test_1, pred_test_2))

# Laste prediction of meta model
final_predictions = meta_model.predict(stacked_predictions_test)

# Evaluation model
mse = mean_squared_error(y_test, final_predictions)
print("score", r2_score(y_test, final_predictions))
print(f'Mean Squared Error: {mse}')

In [24]:
test_id =test.copy()
model_final = meta_model
model_final.fit(X_train[feats], y_train)
y_pred = model_final.predict(X_test[feats])
print(r2_score(y_test, y_pred))
test_id.loc[:,"FloodProbability"] = model_final.predict(test[feats])
submission = test_id[["id", "FloodProbability"]]
submission.to_csv("submission.csv", index=None)

In [24]:
LRout=LinearRegression()
LRout.fit(Xout1_train,yout1_train)
LRoutPred=LRout.predict(Xout1_test)
mseout=mean_squared_error(yout1_test,LRoutPred)
rmseout=np.sqrt(mseout)
maeout=mean_absolute_error(yout1_test,LRoutPred)
r2outscore=r2_score(yout1_test,LRoutPred)
print('Mean Squared Error : ', mseout)
print('Root Mean Squared Error : ', rmseout)
print('Mean Absolute Error : ', maeout)
print('R2 Score : ' ,r2outscore)


In [27]:
Xall_train,Xall_test,yall_train,yall_test=train_test_split(Xall,yall,test_size=.3,random_state=42)
print(Xall_train.shape,Xall_test.shape,yall_train.shape)

In [29]:
LR=LinearRegression()
LR.fit(Xall_train_std,yall_train)
LR_pred=LR.predict(Xall_test_std)
mseall=mean_squared_error(yall_test,LR_pred)
rmseall=np.sqrt(mseall)
maeall=mean_absolute_error(yall_test,LR_pred)
r2scoreall = r2_score(yall_test,LR_pred)
print('Mean Squared Error : ', mseall)
print('Root Mean Squared Error : ', rmseall)
print('Mean Absolute Error : ', maeall)
print('R2 Score : ' ,r2scoreall)


In [30]:
LRallPredTrain=LR.predict(Xall_train_std)
LRallPredTest=LR.predict(Xall_test_std)

alltrain_error = mean_absolute_error(yall_train, LRallPredTrain)
alltest_error = mean_absolute_error(yall_test, LRallPredTest)

print("Training Error:", alltrain_error)
print("Validation Error:", alltest_error)

r2allscoreTrain=r2_score(yall_train,LRallPredTrain)
r2allscoreTest=r2_score(yall_test,LRallPredTest)

print("Training R2score:", r2allscoreTrain)
print("Validation R2score:", r2allscoreTest)


In [31]:
sns.histplot(yall_test, kde=True, color='green', label='Test', stat='density')
sns.histplot(LR_pred, kde=True, color='orange', label='Predicted', stat='density')
plt.title('Distribution Plot of Test and Predicted Data')
plt.legend()
plt.show()

In [32]:
from sklearn.linear_model import Ridge


In [33]:
alpha = 0.1
ridge = Ridge(alpha=alpha)
ridge.fit(Xall_train_std, yall_train)
yreg_pred = ridge.predict(Xall_test_std)
msereg=mean_squared_error(yall_test,yreg_pred)
rmsereg=np.sqrt(msereg)
maereg=mean_absolute_error(yall_test,yreg_pred)
r2scorereg = r2_score(yall_test,yreg_pred)
print('Mean Squared Error : ', msereg)
print('Root Mean Squared Error : ', rmsereg)
print('Mean Absolute Error : ', maereg)
print('R2 Score : ' ,r2scorereg)

In [34]:
from sklearn.ensemble import RandomForestRegressor

In [38]:
StdScaTest = StandardScaler()
test_std=StdScaTest.fit_transform(df_test.iloc[:,1:])


In [40]:
# Prepare the submission file
submission = pd.DataFrame({
    'id': df_test['id'],
    'FloodProbability': test_predictions
})
submission.head(10)



In [41]:
# Save the submission file
submission.to_csv('/kaggle/working/submission.csv', index=False)



In [25]:
%%time
ent,crossmi,mi_sc=get_mutual_info(dt_reg,y_reg,weight=penalty_weight)

seq_mir=[]
for f in mi_sc.index:
    if f.startswith('random_'):
        break
    seq_mir.append(f)
seq_mir=sorted(seq_mir)
cvs_mir=cross_val_score(model,dt_reg[seq_mir],y_reg,scoring='r2',cv=10).mean()
print(f'{cvs_mir:0.5f}  {", ".join(seq_mir)}')

In [1]:
!pip install -U lightautoml

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import requests

import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, r2_score
from sklearn.model_selection import train_test_split
import torch

from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
import lightautoml

In [3]:
N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 42
TEST_SIZE = 0.2
TIMEOUT = 1000 * 3600

np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

In [7]:
task = Task('reg', loss='mse', metric='r2')

In [8]:
%%time
automl = TabularAutoML(
    task = task,
    timeout = 600 * 3600,
    cpu_limit = N_THREADS,
    general_params = {'use_algos':[['linear_l2', 'lgb', 'lgb_tuned']]},
    selection_params ={'mode' : 0},
    tuning_params = {'max_tuning_time': 1200},
    reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE}
)

out_of_fold_predictions = automl.fit_predict(
    train_data,
    roles = {
        'target': 'FloodProbability',
        'drop': ['id']
    }, 
    verbose = 1
)

In [9]:
%%time 
CONFIG_PATH = os.path.join(lightautoml.automl.presets.tabular_presets._base_dir, 'tabular_configs')

configs_list = [os.path.join(CONFIG_PATH, x) for x in [
    "conf_1_sel_type_1.yml",
    "conf_3_sel_type_1_no_inter_lgbm.yml",
]]

automl_utilized = TabularUtilizedAutoML(
    task = task, 
    timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    general_params ={'use_algos':[['linear_l2', 'lgb', 'lgb_tuned']]},
    tuning_params = {'max_tuning_time': 1200},
    reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
    configs_list = configs_list,
    max_runs_per_config=4
)

out_of_fold_predictions_utilized = automl_utilized.fit_predict(
    train_data,
    roles = {
        'target': 'FloodProbability',
        'drop': ['id', 'fskew', 'fkurtosis']
            }, 
    verbose = 1
)

In [10]:
test_predictions = automl.predict(test_data)

In [11]:
test_predictions_utilized = automl_utilized.predict(test_data)

In [12]:
print(f'OOF score: {r2_score(train_data["FloodProbability"].values, out_of_fold_predictions.data[:, 0])}')
print(f'OOF score utilized: {r2_score(train_data["FloodProbability"].values, out_of_fold_predictions_utilized.data[:, 0])}')

In [13]:
submission = pd.DataFrame({
    'id': test_data.id.values,
    'FloodProbability': 0.7 * test_predictions_utilized.data[:, 0] + 0.3 * test_predictions.data[:, 0]
})

In [14]:
submission.head()

In [15]:
submission.to_csv('submission.csv', index = False)

In [8]:
final_models = [ m for m in models if m not in ["autog3", "autog5"]]

params = {"alpha": .01, "random_state": seed, "fit_intercept" : False}

all_intercepts, all_coefs, blend_scores = [], [], []

for r in range(n_repeats):
    new_model = {
        "train"       : oofs[r], 
        "test"        : preds[r], 
        "features"    : final_models,
        "my_model"    : Ridge(**params), 
    }
    res = do_ensemble(**new_model, do_feat_imp = False)
    all_intercepts.extend(res["intercept"]) ; all_coefs.append(res["coefs"])
    blend_scores.append(res["oofs_score"])
    
print(f"{bold}Final mean RMSE {bold_blue}{np.mean(blend_scores):.6f}{end}")

In [9]:
_, ax = plt.subplots(n_repeats, 1, figsize = (10, int(all_coefs[0].shape[1] * 0.5 * n_repeats)), 
                       sharex = True, tight_layout = True)

for i in range(n_repeats):
    ax[i].boxplot(all_coefs[i], vert = False, labels = all_coefs[i].columns)
    ax[i].axvline(x = 0, color = 'green')
    ax[i].set_title(f"Regression results over folds in repetition n° {i+1}")
    ax[i].set_xlabel("Coefficients values")
    ax[i].xaxis.set_ticks_position("top")
    ax[i].xaxis.set_label_position('top')
    ax[i].spines[["right", "bottom"]].set_visible(False)

In [10]:
for n in range(n_repeats):
    for df in [oofs[n], preds[n]]:
        df["cb7_9_12"] = (df["cb7"] + df["cb9"] + df["cb12"]) / 3

for m in ["cb7_9_12"]:
    if m not in models:
        models.append(m)

final_models = [ m for m in models if m not in ["autog3", "autog5", "cb7", "cb9", "cb12"]]

params = {"alpha": .01, "random_state": seed, "fit_intercept" : False}

all_intercepts, all_coefs, blend_scores = [], [], []

for r in range(n_repeats):
    new_model = {
        "train"       : oofs[r], 
        "test"        : preds[r], 
        "features"    : final_models,
        "my_model"    : Ridge(**params), 
    }
    res = do_ensemble(**new_model, do_feat_imp = False)
    all_intercepts.extend(res["intercept"]) ; all_coefs.append(res["coefs"])
    blend_scores.append(res["oofs_score"])
    
print(f"{bold}Final mean RMSE {bold_blue}{np.mean(blend_scores):.6f}{end}")

_, ax = plt.subplots(n_repeats, 1, figsize = (10, int(all_coefs[0].shape[1] * 0.5 * n_repeats)), 
                       sharex = True, tight_layout = True)

for i in range(n_repeats):
    ax[i].boxplot(all_coefs[i], vert = False, labels = all_coefs[i].columns)
    ax[i].axvline(x = 0, color = 'green')
    ax[i].set_title(f"Regression results over folds in repetition n° {i+1}")
    ax[i].set_xlabel("Coefficients values")
    ax[i].xaxis.set_ticks_position("top")
    ax[i].xaxis.set_label_position('top')
    ax[i].spines[["right", "bottom"]].set_visible(False)

In [11]:
for n in range(n_repeats):
    for df in [oofs[n], preds[n]]:
        df["cb7_9_12"] = (df["cb7"] + df["cb9"] + df["cb12"]) / 3
        df["xgb11_12_13"] = (df["xgb11"] + df["xgb12"] + df["xgb13"]) / 3
        df["cb2_3"] = (df["cb2"] + df["cb3"]) / 2
        df["lgbm5_xgb10"] = (df["lgbm5"] + df["xgb10"]) / 2
        df["cb2_cb3_lgbm5_xgb10"] = (df["cb2"] + df["cb3"] + df["lgbm5"] + df["xgb10"]) / 4

for m in ["cb7_9_12", "xgb11_12_13", "cb2_3", "lgbm5_xgb10", "cb2_cb3_lgbm5_xgb10"]:
    if m not in models:
        models.append(m)

final_models = [ m for m in models if m not in [
    "autog3", "autog5", "xgb3", "lgbm10", "lgbm6", "lgbm14", "lgbm15", "lgbm7", "xgb9", "lgbm12",
    "cb10", "lgbm17", "rf1", "cb6", "cb11", "cb4", "cb5", "xgb6", "cb7", "cb9", "cb12",
    "xgb11", "xgb12", "xgb13", "et1", "xgb4", "lgbm11", "cb2", "cb3", "lgbm5", "xgb10",
    "bestlgbm2", "cb2_3", "lgbm5_xgb10",
]]

params = {"alpha": .01, "random_state": seed, "fit_intercept" : False}

all_intercepts, all_coefs, blend_scores = [], [], []

for r in range(n_repeats):
    new_model = {
        "train"       : oofs[r], 
        "test"        : preds[r], 
        "features"    : final_models,
        "my_model"    : Ridge(**params), 
    }
    res = do_ensemble(**new_model, do_feat_imp = True)
    all_intercepts.extend(res["intercept"]) ; all_coefs.append(res["coefs"])
    blend_scores.append(res["oofs_score"])
    
print(f"{bold}Final mean RMSE {bold_blue}{np.mean(blend_scores):.6f}{end}")

_, ax = plt.subplots(n_repeats, 1, figsize = (10, int(all_coefs[0].shape[1] * 0.5 * n_repeats)), 
                       sharex = True, tight_layout = True)

for i in range(n_repeats):
    ax[i].boxplot(all_coefs[i], vert = False, labels = all_coefs[i].columns)
    ax[i].axvline(x = 0, color = 'green')
    ax[i].set_title(f"Regression results over folds in repetition n° {i+1}")
    ax[i].set_xlabel("Coefficients values")
    ax[i].xaxis.set_ticks_position("top")
    ax[i].xaxis.set_label_position('top')
    ax[i].spines[["right", "bottom"]].set_visible(False)

In [12]:
all_scores = {}
ascending = True
for r in range(n_repeats):
    for m in final_models:
        if r == 0:
            all_scores[m] = []
        all_scores[m].append(score_(train[target], oofs[r][m]))
        
all_scores["Ensemble"] = blend_scores

all_scores = pd.DataFrame(all_scores)
_col_order = list(all_scores.mean(axis=0).sort_values(ascending = ascending).index)
all_scores = all_scores[_col_order]

_t = list(all_scores.mean(axis=0).sort_values(ascending = ascending).values)
_labels = [f"{l:15} ({v:.5})" for l,v in zip(_col_order, _t)]

_, ax = plt.subplots(1, 1, figsize = (10, int(all_scores.shape[1] * 1/2)))
ax.boxplot(all_scores, vert = False, labels = _labels)
ax.set_title("OOFS R2")
ax.set_xlabel("R2 (higher is better)")
ax.xaxis.set_ticks_position("top")
ax.xaxis.set_label_position('top')
ax.spines[["right", "bottom"]].set_visible(False)

In [13]:
print(final_models)
_oofs = oofs[0][final_models].copy()
_preds = preds[0][final_models].copy()

for m in final_models:
    for n in range(n_repeats):
        _oofs[m] += oofs[n][m] / n_repeats
        _preds[m] += preds[n][m] / n_repeats
    
model = Ridge(**params)
model.fit(_oofs, train[target])
y_oof = model.predict(_oofs)
print(f"R2 score on train : {score_(train[target], y_oof):.6f}")
y_pred = pd.Series(model.predict(_preds), index=test.index, name=target)
y_pred.to_csv(f"submission_without_autogluon.csv")
!head submission_without_autogluon.csv

_, ax = plt.subplots(1, 1, figsize = (15, int(len(final_models) * 1/2)))
sns.barplot(x = model.coef_, y = final_models, ax = ax, color = "blue")
plt.gca().invert_yaxis()
ax.axvline(x = 0, color = 'green')
ax.xaxis.set_ticks_position("top")
ax.xaxis.set_label_position('top')
ax.spines[["right", "bottom"]].set_visible(False)
ax.set_title(f"Coefficient values with all mean of oofs files");

In [14]:
models = [m for m in models if m not in ["cb7_9_12", "xgb11_12_13", "cb2_3", "lgbm5_xgb10", "cb2_cb3_lgbm5_xgb10"]]

for n in range(n_repeats):
    for df in [oofs[n], preds[n]]:
        df["cb7_9_12"] = (df["cb7"] + df["cb9"] + df["cb12"]) / 3
        df["xgb12_13"] = (df["xgb12"] + df["xgb13"]) / 2
        df["autog3_5"] = (df["autog3"] + df["autog5"]) / 2
        df["cb2_3"] = (df["cb2"] + df["cb3"]) / 2
        df["lgbmcorr_xgb8_lgbm9"] = (df["bestlgbmcorr"] + df["xgb8"] + df["lgbm9"]) / 3

for m in ["cb7_9_12", "xgb12_13", "cb2_3", "autog3_5", "lgbmcorr_xgb8_lgbm9"]:
    if m not in models:
        models.append(m)

final_models = [ m for m in models if m not in 
                [
                 "lgbm12", "lgbm6", "cb4", "lgbm7", "xgb4", "cb6",
                 "cb10", "cb5", "xgb3", "xgb9", "xgb6", 
                 "xgb12", "xgb13", "rf1", "xgb10", "lgbm15", "et1", "lgbm10", # le 24 mai, avec xgb12, 13 et 12_13
                 "lgbm11", # le 25 mai avec ajout autog1
                 "cb11", "xbg11", # le 25 mai avec ajout cb12 et cb13
                 "bestlgbm2", "xgb11", "lgbm14",# le 26 mai, ajout d'un 2ème fold autogluon, et erreur sur libellé xgb11
                 
                 "cb7", "cb9", "cb12", #"cb7_9_12", 
                 "cb2", "cb3", # "cb2_3", 
                 "autog4",
                 "lgbm17", "lgbm16", # le 28 mai ajout lgbm16 et 17
#                 "autog3_5", 
                 "autog3", "autog5", 
                 "bestlgbmcorr", "xgb8", "lgbm9"
                ]]

params = {"alpha": .001, "random_state": seed, "fit_intercept" : False}#, "positive": True}
all_intercepts, all_coefs, blend_scores = [], [], []

oof_mean = pd.Series(0.0, index = train.index, name= target)

for r in range(n_repeats):
    new_model = {
        "train"       : oofs[r], 
        "test"        : preds[r], 
        "features"    : final_models,
        "my_model"    : Ridge(**params), 
    }
    res = do_ensemble(**new_model, do_feat_imp = True)
    all_intercepts.extend(res["intercept"]) ; all_coefs.append(res["coefs"])
    blend_scores.append(res["oofs_score"]) ; 
    oof_mean += res["oofs"] / n_repeats

print(f"{bold}Final mean RMSE {bold_blue}{np.mean(blend_scores):.6f}{end}")

_, ax = plt.subplots(n_repeats, 1, figsize = (10, int(all_coefs[0].shape[1] * 0.5 * n_repeats)), 
                       sharex = True, tight_layout = True)

for i in range(n_repeats):
    ax[i].boxplot(all_coefs[i], vert = False, labels = all_coefs[i].columns)
    ax[i].axvline(x = 0, color = 'green')
    ax[i].set_title(f"Regression results over folds in repetition n° {i+1}")
    ax[i].set_xlabel("Coefficients values")
    ax[i].xaxis.set_ticks_position("top")
    ax[i].xaxis.set_label_position('top')
    ax[i].spines[["right", "bottom"]].set_visible(False)

In [15]:
all_scores = {}
ascending = True
for r in range(n_repeats):
    for m in final_models:
        if r == 0:
            all_scores[m] = []
        all_scores[m].append(score_(train[target], oofs[r][m]))
        
all_scores["Ensemble"] = blend_scores

all_scores = pd.DataFrame(all_scores)
_col_order = list(all_scores.mean(axis=0).sort_values(ascending = ascending).index)
all_scores = all_scores[_col_order]

_t = list(all_scores.mean(axis=0).sort_values(ascending = ascending).values)
_labels = [f"{l:15} ({v:.5})" for l,v in zip(_col_order, _t)]

_, ax = plt.subplots(1, 1, figsize = (10, int(all_scores.shape[1] * 1/2)))
ax.boxplot(all_scores, vert = False, labels = _labels)
ax.set_title("OOFS R2")
ax.set_xlabel("R2 (higher is better)")
ax.xaxis.set_ticks_position("top")
ax.xaxis.set_label_position('top')
ax.spines[["right", "bottom"]].set_visible(False)

In [16]:
print(final_models)
_oofs = oofs[0][final_models].copy()
_preds = preds[0][final_models].copy()

for m in final_models:
    for n in range(n_repeats):
        _oofs[m] += oofs[n][m] / n_repeats
        _preds[m] += preds[n][m] / n_repeats
    
model = Ridge(**params)
model.fit(_oofs, train[target])
y_oof = model.predict(_oofs)
print(f"R2 score on train : {score_(train[target], y_oof):.6f}")
y_pred = pd.Series(model.predict(_preds), index = test.index, name = target)
y_pred.to_csv(f"submission_with_autogluon.csv")
!head submission_with_autogluon.csv

_, ax = plt.subplots(1, 1, figsize = (15, int(len(final_models) * 1/2)))
sns.barplot(x = model.coef_, y = final_models, ax = ax, color = "blue")
plt.gca().invert_yaxis()
ax.axvline(x = 0, color = 'green')
ax.xaxis.set_ticks_position("top")
ax.xaxis.set_label_position('top')
ax.spines[["right", "bottom"]].set_visible(False)
ax.set_title(f"Coefficient values with all mean of oofs files");

In [17]:
folds = StratifiedKFold(n_splits = n_splits, random_state = seed, shuffle = True)
for fold, (_, val_idx) in enumerate(folds.split(train, (train[target]*400).astype(np.int16).astype("category"))):
    print(f"   Fold {fold+1} : R2 {score_(train.loc[val_idx, target], oof_mean.loc[val_idx]):.6f}")    

print(f"Mean of oof R2 score : {score_(train[target], oof_mean):.6f}")

In [18]:
oofs_other = pd.DataFrame(index = train.index)
for m in final_models:
    oofs_other[f"{m}"] = 0 
    for r in range(n_repeats):
        oofs_other[f"{m}"] += oofs[r][m] / n_repeats
    
oofs_other[target] = train[target]

new_model = {
    "train"       : oofs_other, 
    "test"        : preds[0], 
    "features"    : final_models,
    "my_model"    : Ridge(**params), 
}
_ = do_ensemble(**new_model, do_feat_imp = True)

In [19]:
blend_scores = []
oof_mean2 = pd.Series(0.0, index = train.index, name= target)
for r in range(n_repeats):
    new_model = {
        "train"       : oofs[r], 
        "test"        : preds[r], 
        "features"    : final_models,
        "my_model"    : Ridge(**params), 
        "folds"       : StratifiedKFold(n_splits = n_splits, random_state = seed + 10, shuffle = True), 
    }
    res = do_ensemble(**new_model, do_feat_imp = True)
    all_intercepts.extend(res["intercept"]) ; all_coefs.append(res["coefs"])
    blend_scores.append(res["oofs_score"]) ; 
    oof_mean2 += res["oofs"] / n_repeats
    
print(f"{bold}Final mean RMSE {bold_blue}{np.mean(blend_scores):.6f}{end}")
print(f"R2 score oof_mean {score_(train[target], oof_mean):.6f} | oof_mean2 {score_(train[target], oof_mean2):.6f} | ", end='')
print(f"oof_mean + oof_mean2 : {score_(train[target], .5 * (oof_mean + oof_mean2)):.6f}")

In [17]:
# モデルの訓練
_ = regressor.fit(X, y)

In [18]:
# 予測
pred = regressor.predict(test)

In [20]:
# 指定形式へ変換 (sample_submissionの"Rings"列を予測値で入替)
sub[target] = pred
sub.to_csv('submit.csv', index=False)
sub.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder

In [13]:
# Fit the model on the training data
rf_model.fit(X_train, y_train)

In [14]:
# Evaluate the model using training data
y_pred = rf_model.predict(X_train)

In [31]:
from lightgbm import LGBMRegressor
            
lgb_params = {
    'boosting_type': 'gbdt', 
    'n_estimators':2000, 
    'learning_rate' :  0.012,
    'num_leaves' : 250, 
    'subsample_for_bin': 165700, 
    'min_child_samples': 114, 
    'reg_alpha': 2.075e-06, 
    'reg_lambda': 3.839e-07, 
    'colsample_bytree': 0.9634,
    'subsample': 0.9592, 
    'max_depth': 10,
    'random_state':0,
    'verbosity':-1}

lgbm = LGBMRegressor(**lgb_params)

lgbm_model = lgbm.fit(X_train , Y_train)

y_pred_lgbm = lgbm_model.predict(X_test)

lgbm_rmse_calculator = np.sqrt(mean_squared_error(np.log(Y_test), np.log(y_pred_lgbm)))
lgbm_r2_metric = r2_score(Y_test, y_pred_lgbm)

print(f"Light Gradient Boosting RMSE Metric: {lgbm_rmse_calculator}")
print(f"Light Gradient Boosting R-squared Metric: {lgbm_r2_metric}")

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [13]:
clf = LassoCV()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
score = r2_score(y_test, y_pred)
print(f'R2 Score of the Lasso Regression model is: {score}')

In [14]:
test = scaler.transform(df_test)
y_pred_test = clf.predict(test).round(2)

In [15]:
submission = pd.DataFrame({'id': df_test.index, 'FloodProbability': y_pred_test})
submission

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Load training and testing data from CSV files
train_data = pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')
test_data = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv')

# Separate features and target variable
X_train = train_data.drop(columns=['FloodProbability'])
y_train = train_data['FloodProbability']
X_test = test_data

# Handling missing values (consider other imputation methods)
X_train.fillna(X_train.median(), inplace=True)
X_test.fillna(X_test.median(), inplace=True)

# Normalizing features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape data for CNN (assuming 1D CNN)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define the CNN model
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2, padding='same'),  # Adjust padding strategy
    Conv1D(128, 3, activation='relu'),
    MaxPooling1D(pool_size=2, padding='same'),  # Adjust padding strategy
    Conv1D(256, 3, activation='relu'),
    MaxPooling1D(pool_size=2, padding='same'),  # Adjust padding strategy
    Flatten(),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=64, validation_split=0.2)

# Evaluate the model
y_pred_train = model.predict(X_train)
r2_train = r2_score(y_train, y_pred_train)
print(f"R2 Score on Training Data: {r2_train:.4f}")

# Make predictions on test data
y_test_pred = model.predict(X_test)

# Create a DataFrame for submission
submission_df = pd.DataFrame({'id': test_data['id'], 'FloodProbability': y_test_pred.flatten()})

# Save submission DataFrame to CSV
submission_df.to_csv('submission_cnn.csv', index=False)


In [10]:
plt.figure(figsize=(20,20))
sns.heatmap(df.corr(numeric_only=True),annot=True)

In [14]:
plt.figure(figsize=(12, 6))
sns.lineplot(x='Urbanization', y='FloodProbability', data=train)
plt.title('Urbanization vs Flood Probability')
plt.xlabel('Urbanization')
plt.ylabel('Flood Probability')
plt.show()

In [7]:
plt.hist(df["WetlandLoss"], label="WetLandLoss feature")
s = np.random.poisson(5, df.shape[0])
plt.hist(s, label="Poisson, lambda=5", alpha=0.5)
plt.legend()

plt.show()

In [9]:
# Plotting the distribution of each features for Train and Test
ncol = 4
nrow = int(np.ceil(len(orig_features) / 4))
plt.figure(figsize=(ncol*3, nrow*2.5))
for i,col in enumerate(orig_features):
    ax = plt.subplot(nrow, ncol, i+1)
    sb.countplot(train,x=col,ax=ax,palette='viridis')
    sb.countplot(test,x=col,ax=ax,palette='autumn')
    ax.set_title(col)
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
plt.tight_layout();

In [10]:
# Checking the target distribution
plt.figure(figsize=(8,2))
sb.histplot(train["FloodProbability"], kde=True, \
            bins=np.linspace(train["FloodProbability"].min(), train["FloodProbability"].max(), 88), color='r');
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))

In [11]:
# cheking the correlation
plt.figure(figsize=(12,8))
corr = train.iloc[:,1:].corr()
mask = np.triu(np.ones_like(corr))
sb.heatmap(corr, vmax=1, vmin=-1, center=0, mask=mask, annot=True, fmt='0.1f', cmap='inferno');

In [13]:
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(7,4))
sb.lineplot(pca.explained_variance_ratio_.cumsum())
plt.xlabel('Principal Components')
plt.ylabel('Explained Variance Ratio')
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))

In [8]:
import matplotlib.pyplot as plt
plt.scatter(df["MonsoonIntensity"],y)

In [19]:


# Assuming 'df' is your DataFrame and 'column' is the column you want to filter
# Q1 = df.quantile(0.05)
# Q3 = df.quantile(0.75)
# IQR = Q3 - Q1

# # Define bounds for what is considered an outlier
# lower_bound = Q1 - 1.5 * IQR
# upper_bound = Q3 + 1.5 * IQR

# # Filter out outliers and keep only the valid values
# df_filtered = (df >= lower_bound) & (df <= upper_bound)
# test=(test>=lower_bound)&(test<=upper_bound)

In [6]:
columns = list(train_df.columns)[1:-1]

fig = plt.figure(figsize=(15, 10))
for i in range(len(columns)):
    ax = plt.subplot(5, 4, i + 1)
    sns.histplot(train_df[columns[i]], kde=True, ax=ax)
    ax.set_xlabel(str(columns[i]).capitalize())
    ax.grid(False)
    
    ax.set_yticklabels([])
    ax.set_ylabel('')
    
fig.text(0.04, 0.5, 'Frequency', va='center', rotation='vertical', fontsize=14)

plt.tight_layout(rect=[0.05, 0, 1, 1])

plt.show()

In [7]:
# Compute the correlation matrix
corr_matrix = train_df.drop(columns=['id']).corr()

# Plot the heatmap
plt.figure(figsize=(15, 15))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.show()

In [6]:
for col in cols:
    fig, ax = plt.subplots(figsize=(6,2))
    max_val = round(train[col].max()) + 1
    train[col].hist(density=True,bins = np.arange(0,max_val,1), ax=ax)
    plt.xticks(np.arange(0,20,1))
    plt.title(col)
    plt.show

In [8]:
round(test.agg(['min','mean','median','max','var','std','skew']),2).T

In [9]:
corr = train.drop('FloodProbability', axis=1).corr()
mask = np.triu(np.ones_like(corr, dtype = bool))

sns.heatmap(corr, mask = mask,linewidth=0.1)
plt.show()

In [10]:
corr = train.corr()
mask = np.triu(np.ones_like(corr, dtype = bool))

sns.heatmap(corr, mask = mask,linewidth=0.1)
plt.show()

In [11]:
train.drop('FloodProbability', axis=1).plot(kind='box',vert=False)
plt.title('Boxplot of train variables')
plt.show()

In [12]:
test.plot(kind='box',vert=False)
plt.title('Boxplot of test variables')
plt.show()

In [13]:
# Plotting the Skewness of each column
skewness_df = pd.DataFrame(skew).T
skewness_df.columns = ['Skewness']

skewness_df.plot(kind='bar', color='skyblue', figsize=(10, 6))
plt.title('Skewness of Numeric Columns')
plt.show()

In [14]:
plt.figure(figsize=(4 * 5, 5 * 4))

for i, col in enumerate(num_cols, 1):
    plt.subplot(4, 5, i)
    sns.countplot(data=data, x=col);
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [9]:
plt.figure(figsize=(12,10))
sns.heatmap(co_mat)

In [10]:
sns.boxplot(x=train_df['MonsoonIntensity'])
plt.show()

In [16]:
ax = plt.axes()
ax.scatter(y_test, lr_pred, alpha=.5)

ax.set(xlabel='Ground truth', 
       ylabel='Predictions',
       title='Flood Predictions vs Truth, using Linear Regression');

In [11]:
plt.figure(figsize=(15, 120))
countplot_idx = 1
boxplot_idx = 2
for column in cat_columns:
    ax = plt.subplot(20, 2, countplot_idx)
    sns.countplot(x=train_set[column], palette="rocket", hue=train_set[column], dodge=False)
    ax.set_title(f"Count Plot of {column}")
    countplot_idx += 2
    
    plt.subplot(20, 2, boxplot_idx)
    sns.boxplot(x=train_set[column])
    plt.title(f'Box plot for {column}')
    boxplot_idx += 2        

plt.tight_layout()
plt.show()

In [12]:
pd.DataFrame(train_set[cat_columns].skew(), columns= ["skew"])

In [16]:
plt.figure(figsize=(21,8))
sns.heatmap(train_set.corr(), annot=True, )
plt.show()

In [43]:
plt.figure(figsize=(21,8))
sns.heatmap(X[corr_more_02].join(y).corr(), annot=True, )
plt.show()

In [4]:
sns.boxplot(flood_train.drop(["id", "FloodProbability"], axis = 1))
plt.xticks(rotation = 90)

In [17]:
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
# histogram for all columns with seaborn

import seaborn as sns
import matplotlib.pyplot as plt

for column in Data.columns:
    plt.figure(figsize=(10, 7))
    sns.histplot(Data[column])
    plt.title(f"Histogram of {column}")
    plt.show()


In [14]:
# Define a function that plots a count plot of the features
def count_plot(df, col, re):
    plt.subplot(df.shape[1], 1, re + 1)
    sns.countplot(data = df, x = col, palette = palette)
    plt.xlabel(col)
    plt.ylabel('count')
    plt.title(f'Count of the Values of the {col} Feature')
    

In [15]:
for col_name, re in zip(X_train.columns, range(X_train.shape[1])):
    plt.figure(figsize = (12, 50))
    count_plot(X_train, col_name, re)

plt.tight_layout()
plt.show()

In [16]:
# Define a cat plot of the features vs the flood probability
def cat_plot(df, col, re):
    sns.catplot(data = df, x = col, y = 'FloodProbability', kind = 'boxen', palette = palette)
    plt.xlabel(col)
    plt.ylabel('count')
    plt.title(f'Categorical Plot for the Values of the {col} Feature vs. Flood Probability')

In [18]:
# Construct a triangular heatmap to study the correlation in the training data
corr = train_df.corr()
mask = np.triu(np.ones_like(corr, dtype = bool))

plt.figure(figsize=(12, 10))

sns.heatmap(corr, mask=mask, center=0, annot=True,
            fmt='.2f', square=True, cmap='coolwarm')

In [26]:
# Construct a regression plot
plt.figure(figsize = (15, 10))

for col_name, re in zip(features, range(len(features))):
    plt.subplot(2, 2, re + 1)
    sns.regplot(data = X_train_1, x = col_name, y = y_train)
    plt.xlabel(col_name)
    plt.ylabel('flood probability')
    plt.title(f'Flood Probability by {col_name}')

plt.tight_layout()
plt.show()

In [27]:
# Construct a triangular heat map
train_new = pd.concat([X_train_1[features], y_train], axis = 1)

corr = train_new.corr()
mask = np.triu(np.ones_like(corr, dtype = bool))

plt.figure(figsize=(12, 10))

sns.heatmap(corr, mask=mask, center=0, annot=True,
            fmt='.2f', square=True, cmap='coolwarm')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [9]:
print("Test Histogramme")
sns.set(rc={'figure.figsize': (20, 16)})
test.hist(color='blue');

In [10]:
cols_test = test.columns
fig, ax = plt.subplots(nrows=len(cols_test)//3+1, ncols=3, figsize=(24,20), constrained_layout=True)
ax = ax.ravel()
for i,j in enumerate(cols_test):
    sns.boxplot(train, ax = ax[i], x=test[j])

In [11]:
cols_train = train.columns
fig, ax = plt.subplots(nrows=len(cols_train)//3+1, ncols=3, figsize=(24,20), constrained_layout=True)
ax = ax.ravel()
for i,j in enumerate(cols_train):
    sns.boxplot(train, ax = ax[i], x=train[j])

In [12]:
j=0
k=0
fig,ax=plt.subplots(nrows=7,ncols=3,figsize=(18,30))
for i,fea in enumerate(NumCols):
    if i==0:
        j=0
        k=0
    elif i%3==0:
        j=j+1
        k=0
    else:
        k=k+1

    ax[j,k].set_xlabel(fea)
    sns.scatterplot(data=df_train,x=df_train[fea].value_counts(),y=df_train['FloodProbability'],ax=ax[j,k])

In [13]:
fig,ax=plt.subplots(nrows=21,ncols=2,figsize=(8,80))
for i,fea in enumerate(NumCols):
    sns.distplot(df_train[fea],ax=ax[i,0])
    sns.boxplot(df_train.iloc[:,i:i+1],ax=ax[i,1])


In [22]:
sns.histplot(y_reg)
plt.show()
plt.close()

In [31]:
sns.scatterplot(y=(dt_gen.sum(axis=1)),x=y_gen)
plt.xlabel('y')
plt.ylabel('$\sum$ features')
plt.grid()

In [6]:
class FeatureImportance():
    
    def __init__(self, n_permutations = None, lib_metric = None, digit = 4):
        
        self.n_permutations = n_permutations
        self.lib_metric = lib_metric
        self.differences = {}
        self.digit = digit
        
        self.metric_max = ["Accuracy", "AUC", "Recall@20", "cohen_kappa_score", "auc", "f1", "R2"]
        self.metric_min = ["Logloss", "rmse", "RMSE", "mae", "Balanced log loss", "RMSLE"]
        if lib_metric not in self.metric_max + self.metric_min:
            raise ValueError(f"Unknown lib_metric : {lib_metric}.")
        
    def append(self, feature, metric_value, metric_ref):
        
        if feature not in self.differences.keys():
            self.differences[feature] = []
            
        self.differences[feature].append(metric_value - metric_ref)
        
    def build_dataframe(self):
        
        if self.differences == {}: return
        
        df_diff = pd.DataFrame(self.differences).transpose()
        df_diff["mean"] = df_diff.mean(axis=1)
        df_diff.sort_values("mean", ascending = self.lib_metric in self.metric_max, inplace=True)
        
        return df_diff
        
    def plot(self, title=""):
        
        if self.differences == {}: return
        
        df_diff = self.build_dataframe()
        
        fk = next(iter(self.differences))
        len_fk = len(self.differences[fk])
        
        fig, ax = plt.subplots(1, 2, figsize = (15, int(df_diff.shape[0] * 1/2)))
            
        ax[0].boxplot(df_diff[[c for c in range(len_fk)]][::-1].T,
                          vert=False, labels=df_diff.index[::-1])
        ax[0].axvline(x = 0, color = 'green')
        ax[0].set_xlabel(f"Distribution {self.lib_metric} difference after permutation of values")
            
        sns.barplot(x = "mean", y = df_diff.index, data = df_diff, ax = ax[1], color = "green")
        ax[1].bar_label(ax[1].containers[0], fmt = f'%.{self.digit}f', padding = 2)
        ax[1].set_xlabel(f"Mean {self.lib_metric} difference after permutation of values")
        ax[1].set_yticklabels([])
        ax[1].set_yticks([])
        
        for ax_ in ax:
            ax_.set_title(title)
            ax_.xaxis.set_ticks_position("top")
            ax_.xaxis.set_label_position('top')
            ax_.spines[["right", "bottom"]].set_visible(False)
        ax[1].spines[["left"]].set_visible(False)

In [11]:
# 目的変数に対する説明変数の分布

cat_features = [
    "MonsoonIntensity",
    "TopographyDrainage",
    "RiverManagement",
    "Deforestation",
    "Urbanization",
    "ClimateChange",
    "DamsQuality",
    "Siltation",
    "AgriculturalPractices",
    "Encroachments",
    "IneffectiveDisasterPreparedness",
    "DrainageSystems",
    "CoastalVulnerability",
    "Landslides",
    "Watersheds",
    "DeterioratingInfrastructure",
    "PopulationScore",
    "WetlandLoss",
    "InadequatePlanning",
    "PoliticalFactors",
]
target = "FloodProbability"

fig, ax = plt.subplots(len(cat_features), 1, figsize=(25, 4*len(cat_features)), dpi=150)
ax = ax.flatten()
for i, ft in enumerate(cat_features):
    sns.histplot(
        data=train,
        x=target,
        hue=ft,
        multiple='stack',
        edgecolor='.3',
        linewidth=.5,
        log_scale=False,
        ax=ax[i],
    )
    ax[i].get_legend().remove()
fig.suptitle(f'Train Categorical Features vs {target}\n\n\n', ha='center', fontweight='bold', fontsize=25)
plt.tight_layout()
plt.show()

In [12]:
def plot_correlation_heatmap(df, title_name):
    corr = df.corr()
    fig, axes = plt.subplots(figsize=(20, 15))
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(mask)] = True
    sns.heatmap(corr, mask=mask, linewidths=.5, annot=True)
    plt.title(title_name)
    plt.show()

plot_correlation_heatmap(train, 'Train Dataset Correlation')
plot_correlation_heatmap(test, 'Test Dataset Correlation')

In [8]:
plt.figure(figsize=(8, 6))
sns.boxplot(train_data['FloodProbability'])
plt.title('Boxplot of FloodProbability')
plt.xlabel('FloodProbability')
plt.show()